<a href="https://colab.research.google.com/github/cryptopatience/Main-code/blob/main/Mag_7_%2B_IBIT_COIN_%EA%B3%B5%EB%A7%A4%EB%8F%84%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 필요한 라이브러리 설치 및 임포트
!pip install yfinance pandas numpy plotly requests -q

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
import requests
from io import StringIO
import time

warnings.filterwarnings('ignore')
pio.renderers.default = 'colab'

# ==================== FINRA 공매도 데이터 수집 함수 ====================

def get_finra_short_interest(ticker, retries=3):
    """
    FINRA에서 특정 종목의 공매도 잔고(Short Interest) 데이터 수집
    월 2회 업데이트 (Settlement Date 기준)
    """
    for attempt in range(retries):
        try:
            # FINRA Short Interest URL
            url = f"https://api.finra.org/data/group/otcMarket/name/consolidatedShortInterest"

            # 최근 6개월 데이터 요청
            params = {
                'limit': 100,
                'offset': 0,
                'symbolCode': ticker
            }

            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
            }

            response = requests.get(url, params=params, headers=headers, timeout=10)

            if response.status_code == 200:
                data = response.json()

                if data and len(data) > 0:
                    df = pd.DataFrame(data)

                    # 최근 데이터 추출
                    if 'settlementDate' in df.columns and 'shortInterest' in df.columns:
                        df['settlementDate'] = pd.to_datetime(df['settlementDate'])
                        df = df.sort_values('settlementDate', ascending=False)

                        latest = df.iloc[0]

                        return {
                            'ticker': ticker,
                            'settlement_date': latest['settlementDate'].strftime('%Y-%m-%d'),
                            'short_interest': latest.get('shortInterest', 0),
                            'avg_daily_volume': latest.get('averageDailyVolume', 0),
                            'days_to_cover': latest.get('daysToCover', 0),
                            'change_pct': latest.get('changePercent', 0)
                        }

            # API 실패 시 웹 스크래핑 시도
            return get_finra_short_interest_web(ticker)

        except Exception as e:
            if attempt < retries - 1:
                time.sleep(2)
                continue
            print(f"  ⚠️ FINRA API 오류 ({ticker}): {str(e)}")
            return None

    return None


def get_finra_short_interest_web(ticker):
    """
    FINRA 웹사이트에서 직접 공매도 데이터 스크래핑 (대체 방법)
    """
    try:
        # FINRA의 공매도 잔고 데이터 CSV 엔드포인트
        # 실제 URL은 FINRA 웹사이트 구조에 따라 변경될 수 있음
        base_url = "https://cdn.finra.org/equity/regsho/daily"

        # 최근 날짜 시도
        today = datetime.now()
        for days_back in range(30):  # 최근 30일 시도
            check_date = today - timedelta(days=days_back)
            date_str = check_date.strftime('%Y%m%d')

            url = f"{base_url}/CNMSshvol{date_str}.txt"

            try:
                response = requests.get(url, timeout=10)
                if response.status_code == 200:
                    df = pd.read_csv(StringIO(response.text), sep='|')

                    if 'Symbol' in df.columns:
                        ticker_data = df[df['Symbol'] == ticker]

                        if not ticker_data.empty:
                            row = ticker_data.iloc[0]

                            return {
                                'ticker': ticker,
                                'date': check_date.strftime('%Y-%m-%d'),
                                'short_volume': row.get('ShortVolume', 0),
                                'total_volume': row.get('TotalVolume', 0),
                                'short_volume_ratio': (row.get('ShortVolume', 0) / row.get('TotalVolume', 1) * 100) if row.get('TotalVolume', 0) > 0 else 0
                            }
            except:
                continue

        return None

    except Exception as e:
        return None


def get_short_interest_from_yfinance(ticker):
    """
    Yahoo Finance에서 공매도 관련 데이터 수집 (대체 방법)
    """
    try:
        stock = yf.Ticker(ticker)
        info = stock.info

        short_data = {
            'ticker': ticker,
            'short_ratio': info.get('shortRatio', 0),  # Days to Cover
            'short_percent_float': info.get('shortPercentOfFloat', 0) * 100 if info.get('shortPercentOfFloat') else 0,
            'shares_short': info.get('sharesShort', 0),
            'shares_short_prior_month': info.get('sharesShortPriorMonth', 0),
            'short_percent_shares_out': info.get('sharesPercentSharesOut', 0) * 100 if info.get('sharesPercentSharesOut') else 0
        }

        # 변화율 계산
        if short_data['shares_short_prior_month'] > 0:
            short_data['short_change_pct'] = ((short_data['shares_short'] - short_data['shares_short_prior_month']) /
                                               short_data['shares_short_prior_month'] * 100)
        else:
            short_data['short_change_pct'] = 0

        return short_data

    except Exception as e:
        print(f"  ⚠️ Yahoo Finance 오류 ({ticker}): {str(e)}")
        return None


def get_comprehensive_short_data(ticker):
    """
    여러 소스에서 공매도 데이터 종합 수집
    """
    print(f"  📊 {ticker} 공매도 데이터 수집...", end=" ")

    # Yahoo Finance 데이터 (가장 신뢰성 높음)
    yf_data = get_short_interest_from_yfinance(ticker)

    # FINRA 데이터 시도
    finra_data = get_finra_short_interest(ticker)

    # 데이터 통합
    combined_data = {
        'ticker': ticker,
        'short_ratio_days': 0,
        'short_percent_float': 0,
        'shares_short_millions': 0,
        'short_change_pct': 0,
        'data_source': 'N/A'
    }

    if yf_data:
        combined_data.update({
            'short_ratio_days': round(yf_data.get('short_ratio', 0), 2),
            'short_percent_float': round(yf_data.get('short_percent_float', 0), 2),
            'shares_short_millions': round(yf_data.get('shares_short', 0) / 1e6, 2),
            'short_change_pct': round(yf_data.get('short_change_pct', 0), 2),
            'data_source': 'Yahoo Finance'
        })
        print("✓ (YF)", end="")

    if finra_data:
        # FINRA 데이터가 있으면 추가 정보로 활용
        if 'days_to_cover' in finra_data and finra_data['days_to_cover'] > 0:
            combined_data['short_ratio_days'] = round(finra_data['days_to_cover'], 2)
        combined_data['data_source'] += ' + FINRA'
        print(" + FINRA", end="")

    print(" ✓")
    return combined_data


# ==================== Magnificent Seven 정의 ====================

MAG7_STOCKS = {
    'AAPL': {
        'name': 'Apple Inc.',
        'description': '아이폰, 생태계, 온디바이스 AI',
        'sector': 'Technology',
        'industry': 'Consumer Electronics'
    },
    'MSFT': {
        'name': 'Microsoft Corporation',
        'description': '클라우드(Azure), 생성형 AI (OpenAI 대주주)',
        'sector': 'Technology',
        'industry': 'Software'
    },
    'GOOGL': {
        'name': 'Alphabet Inc.',
        'description': '구글 검색, 유튜브, AI (Gemini)',
        'sector': 'Communication Services',
        'industry': 'Internet Content & Information'
    },
    'AMZN': {
        'name': 'Amazon.com Inc.',
        'description': '전자상거래, 클라우드(AWS) 1위',
        'sector': 'Consumer Cyclical',
        'industry': 'Internet Retail'
    },
    'NVDA': {
        'name': 'NVIDIA Corporation',
        'description': 'AI 반도체(GPU) 독점적 지배자',
        'sector': 'Technology',
        'industry': 'Semiconductors'
    },
    'META': {
        'name': 'Meta Platforms Inc.',
        'description': '페이스북, 인스타그램, AI(Llama)',
        'sector': 'Communication Services',
        'industry': 'Internet Content & Information'
    },
    'TSLA': {
        'name': 'Tesla Inc.',
        'description': '전기차, 자율주행, 로봇',
        'sector': 'Consumer Cyclical',
        'industry': 'Auto Manufacturers'
    }
}

print("=" * 80)
print("🌟 MAGNIFICENT SEVEN (MAG 7) 실시간 종합 분석 + 공매도 데이터")
print("=" * 80)
print("\n빅테크 7대 기업 리스트:")
for ticker, info in MAG7_STOCKS.items():
    print(f"  • {ticker:6} - {info['name']:30} | {info['description']}")
print("=" * 80)


def get_current_quarter_start():
    """현재 분기 시작일 계산"""
    now = datetime.now()
    quarter = (now.month - 1) // 3
    quarter_start_month = quarter * 3 + 1
    quarter_start = datetime(now.year, quarter_start_month, 1)
    return quarter_start


def calculate_anchored_vwap(df):
    """Anchored VWAP 계산"""
    df = df.copy()
    df['Typical_Price'] = (df['High'] + df['Low'] + df['Close']) / 3
    df['TP_Volume'] = df['Typical_Price'] * df['Volume']
    df['Cumulative_TP_Volume'] = df['TP_Volume'].cumsum()
    df['Cumulative_Volume'] = df['Volume'].cumsum()
    df['Anchored_VWAP'] = df['Cumulative_TP_Volume'] / df['Cumulative_Volume']
    return df


def get_quarterly_vwap_analysis(ticker):
    """분기별 Anchored VWAP 분석"""
    try:
        quarter_start = get_current_quarter_start()
        end_date = datetime.now()
        quarter_num = (quarter_start.month - 1) // 3 + 1

        stock = yf.Ticker(ticker)
        df = stock.history(start=quarter_start, end=end_date)

        if df.empty or len(df) < 5:
            return None

        df = calculate_anchored_vwap(df)

        current_price = df['Close'].iloc[-1]
        current_vwap = df['Anchored_VWAP'].iloc[-1]
        above_vwap_ratio = (df['Close'] > df['Anchored_VWAP']).sum() / len(df) * 100
        recent_5days_avg = df['Close'].tail(5).mean()
        recent_10days_avg = df['Close'].tail(10).mean()

        recent_20 = df['Close'].tail(min(20, len(df)))
        uptrend_strength = (recent_20.diff() > 0).sum() / len(recent_20) * 100 if len(recent_20) > 1 else 50

        recent_volume = df['Volume'].tail(5).mean()
        avg_volume = df['Volume'].mean()
        volume_ratio = recent_volume / avg_volume if avg_volume > 0 else 1

        info = stock.info
        quarter_start_price = df['Close'].iloc[0]
        quarter_return = ((current_price - quarter_start_price) / quarter_start_price * 100)

        return {
            'Ticker': ticker,
            'Company': MAG7_STOCKS[ticker]['name'],
            'Description': MAG7_STOCKS[ticker]['description'],
            'Sector': MAG7_STOCKS[ticker]['sector'],
            'Quarter': f'{quarter_start.year} Q{quarter_num}',
            'Quarter_Start_Date': quarter_start.strftime('%Y-%m-%d'),
            'Trading_Days': len(df),
            'Current_Price': round(current_price, 2),
            'Anchored_VWAP': round(current_vwap, 2),
            'Quarter_Start_Price': round(quarter_start_price, 2),
            'Quarter_Return_%': round(quarter_return, 2),
            'Price_vs_VWAP_%': round((current_price - current_vwap) / current_vwap * 100, 2),
            'Above_VWAP_Days_%': round(above_vwap_ratio, 1),
            'Recent_5D_Avg': round(recent_5days_avg, 2),
            'Recent_10D_Avg': round(recent_10days_avg, 2),
            'Uptrend_Strength_%': round(uptrend_strength, 1),
            'Volume_Ratio': round(volume_ratio, 2),
            'Is_Above_VWAP': current_price > current_vwap,
            'Market_Cap': info.get('marketCap', 0),
            'Buy_Signal_Score': 0
        }

    except Exception as e:
        print(f"Error processing {ticker}: {str(e)}")
        return None


# ==================== 메인 분석 시작 ====================

quarter_start = get_current_quarter_start()
quarter_num = (quarter_start.month - 1) // 3 + 1

print("\n" + "=" * 80)
print(f"🎯 Anchored VWAP 분석: {quarter_start.year} Q{quarter_num}")
print(f"📍 Anchor Point: {quarter_start.strftime('%Y-%m-%d')}")
print(f"📅 현재까지 거래일: {(datetime.now() - quarter_start).days}일")
print("=" * 80)

# MAG 7 분석
results = []
mag7_tickers = list(MAG7_STOCKS.keys())

print("\n🔍 MAG 7 종목 분석 중...\n")

for ticker in mag7_tickers:
    print(f"분석 중: {ticker} ({MAG7_STOCKS[ticker]['name']})...", end=" ")
    result = get_quarterly_vwap_analysis(ticker)
    if result:
        results.append(result)
        print("✓")
    else:
        print("✗")

df_results = pd.DataFrame(results)

# 매수 신호 점수 계산
def calculate_buy_score(row):
    score = 0
    if row['Is_Above_VWAP']:
        score += 30

    price_diff = row['Price_vs_VWAP_%']
    if 0 < price_diff <= 5:
        score += 20
    elif 5 < price_diff <= 10:
        score += 10
    elif price_diff > 10:
        score += 5

    if row['Above_VWAP_Days_%'] >= 80:
        score += 20
    elif row['Above_VWAP_Days_%'] >= 60:
        score += 15
    elif row['Above_VWAP_Days_%'] >= 40:
        score += 10

    if row['Uptrend_Strength_%'] >= 60:
        score += 15
    elif row['Uptrend_Strength_%'] >= 50:
        score += 10

    if row['Volume_Ratio'] >= 1.2:
        score += 15
    elif row['Volume_Ratio'] >= 1.0:
        score += 10

    return min(score, 100)

df_results['Buy_Signal_Score'] = df_results.apply(calculate_buy_score, axis=1)

# 시가총액 추가
df_results['Market_Cap_Trillion'] = (df_results['Market_Cap'] / 1e12).round(3)

# ==================== 공매도 데이터 수집 ====================

print("\n" + "=" * 80)
print("📊 공매도(Short Interest) 데이터 수집 중...")
print("=" * 80)

short_data_list = []

for ticker in mag7_tickers:
    short_data = get_comprehensive_short_data(ticker)
    if short_data:
        short_data_list.append(short_data)

df_short = pd.DataFrame(short_data_list)

# 공매도 데이터를 메인 데이터프레임에 병합
df_results = df_results.merge(df_short, left_on='Ticker', right_on='ticker', how='left')

# 공매도 스코어 계산 (낮을수록 좋음)
def calculate_short_score(row):
    """
    공매도 스코어: 공매도 비율이 낮을수록 높은 점수
    - 5% 미만: 매우 좋음 (20점)
    - 5-10%: 양호 (15점)
    - 10-20%: 보통 (10점)
    - 20% 이상: 주의 (5점)
    """
    short_pct = row.get('short_percent_float', 0)

    if short_pct < 5:
        return 20
    elif short_pct < 10:
        return 15
    elif short_pct < 20:
        return 10
    else:
        return 5

df_results['Short_Score'] = df_results.apply(calculate_short_score, axis=1)

# 종합 투자 점수 (기존 Buy Signal + Short Score)
df_results['Total_Investment_Score'] = df_results['Buy_Signal_Score'] + df_results['Short_Score']
df_results = df_results.sort_values('Total_Investment_Score', ascending=False)

# ==================== 결과 출력 ====================

print("\n" + "=" * 80)
print("📊 MAGNIFICENT SEVEN - 종합 분석 결과 (공매도 포함)")
print("=" * 80)

display_cols = ['Ticker', 'Company', 'Current_Price', 'Anchored_VWAP', 'Price_vs_VWAP_%',
                'Quarter_Return_%', 'short_percent_float', 'short_ratio_days',
                'Buy_Signal_Score', 'Short_Score', 'Total_Investment_Score']

print(df_results[display_cols].to_string(index=False))

print("\n" + "=" * 80)
print("🏆 MAG 7 투자 추천 순위 (공매도 분석 포함)")
print("=" * 80)

for idx, row in df_results.iterrows():
    rank = df_results.index.get_loc(idx) + 1

    if rank == 1:
        medal = "🥇"
    elif rank == 2:
        medal = "🥈"
    elif rank == 3:
        medal = "🥉"
    else:
        medal = f"  {rank}"

    print(f"\n{medal} {row['Ticker']} - {row['Company']}")
    print(f"    🎯 {row['Description']}")
    print(f"    💰 시가총액: ${row['Market_Cap_Trillion']:.2f}T")
    print(f"    📈 현재가: ${row['Current_Price']} | VWAP: ${row['Anchored_VWAP']}")
    print(f"    📊 VWAP 대비: {row['Price_vs_VWAP_%']:+.2f}% | 분기 수익률: {row['Quarter_Return_%']:+.2f}%")
    print(f"    🎲 VWAP 위 거래일: {row['Above_VWAP_Days_%']}% | 추세 강도: {row['Uptrend_Strength_%']}%")

    # 공매도 정보
    short_pct = row.get('short_percent_float', 0)
    short_days = row.get('short_ratio_days', 0)
    print(f"    🔴 공매도 비율: {short_pct:.2f}% of float | 커버 소요일: {short_days:.1f}일")

    if short_pct < 5:
        short_status = "✅ 매우 낮음 (긍정)"
    elif short_pct < 10:
        short_status = "🟢 양호"
    elif short_pct < 20:
        short_status = "🟡 보통"
    else:
        short_status = "🔴 높음 (주의)"

    print(f"    🎯 공매도 상태: {short_status}")

    print(f"    ⭐ 기술적 점수: {row['Buy_Signal_Score']}/100 | 공매도 점수: {row['Short_Score']}/20", end="")
    print(f" | 종합: {row['Total_Investment_Score']}/120")

    if row['Total_Investment_Score'] >= 90:
        print("       → 💚 최우선 매수 추천")
    elif row['Total_Investment_Score'] >= 75:
        print("       → 💛 강력 매수")
    elif row['Total_Investment_Score'] >= 60:
        print("       → 💙 눌림목 대기")
    else:
        print("       → ⚪ 관찰 필요")

# ==================== 공매도 관련 차트 ====================

print("\n📈 공매도 분석 차트 생성 중...")

# 차트 1: 공매도 비율 비교
fig_short = go.Figure()

colors_short = ['#00C853' if x < 5 else '#FFD600' if x < 10 else '#FF5722'
                for x in df_results['short_percent_float']]

fig_short.add_trace(go.Bar(
    y=df_results['Ticker'],
    x=df_results['short_percent_float'],
    orientation='h',
    marker=dict(color=colors_short),
    text=df_results['short_percent_float'].round(2),
    textposition='auto',
    hovertemplate='<b>%{y}</b><br>공매도 비율: %{x:.2f}%<extra></extra>'
))

fig_short.update_layout(
    title='🔴 MAG 7 - 공매도 비율 (Short % of Float)',
    xaxis_title='공매도 비율 (%)',
    yaxis_title='종목',
    height=500,
    template='plotly_white'
)

# 참고선 추가
fig_short.add_vline(x=5, line_dash="dash", line_color="green",
                    annotation_text="낮음", annotation_position="top")
fig_short.add_vline(x=10, line_dash="dash", line_color="orange",
                    annotation_text="보통", annotation_position="top")
fig_short.add_vline(x=20, line_dash="dash", line_color="red",
                    annotation_text="높음", annotation_position="top")

fig_short.show()

# 차트 2: 종합 투자 점수 (기술적 + 공매도)
fig_total = make_subplots(
    rows=1, cols=2,
    subplot_titles=('기술적 분석 점수', '종합 투자 점수'),
    specs=[[{"type": "bar"}, {"type": "bar"}]]
)

fig_total.add_trace(
    go.Bar(
        y=df_results['Ticker'],
        x=df_results['Buy_Signal_Score'],
        orientation='h',
        name='기술적 점수',
        marker_color='#2196F3',
        text=df_results['Buy_Signal_Score'],
        textposition='auto'
    ),
    row=1, col=1
)

fig_total.add_trace(
    go.Bar(
        y=df_results['Ticker'],
        x=df_results['Total_Investment_Score'],
        orientation='h',
        name='종합 점수',
        marker_color='#4CAF50',
        text=df_results['Total_Investment_Score'],
        textposition='auto'
    ),
    row=1, col=2
)

fig_total.update_xaxes(title_text="점수", row=1, col=1)
fig_total.update_xaxes(title_text="점수", row=1, col=2)
fig_total.update_yaxes(title_text="종목", row=1, col=1)

fig_total.update_layout(
    title_text='📊 MAG 7 - 투자 점수 비교 (기술적 vs 종합)',
    height=500,
    showlegend=False,
    template='plotly_white'
)

fig_total.show()

# 차트 3: 공매도 vs 분기 수익률
fig_scatter = px.scatter(
    df_results,
    x='short_percent_float',
    y='Quarter_Return_%',
    size='Market_Cap_Trillion',
    color='Total_Investment_Score',
    hover_data=['Ticker', 'Company'],
    text='Ticker',
    color_continuous_scale='RdYlGn',
    title='📊 MAG 7 - 공매도 비율 vs 분기 수익률',
    labels={
        'short_percent_float': '공매도 비율 (%)',
        'Quarter_Return_%': '분기 수익률 (%)',
        'Total_Investment_Score': '종합 점수'
    }
)

fig_scatter.update_traces(textposition='top center', textfont_size=12)
fig_scatter.update_layout(height=600)
fig_scatter.show()

# ==================== 최종 요약 ====================

print("\n" + "=" * 80)
print("📈 MAG 7 투자 전략 최종 요약 (공매도 분석 포함)")
print("=" * 80)

print(f"\n📍 분석 기준: Anchored VWAP ({quarter_start.year} Q{quarter_num})")
print(f"📅 Anchor Point: {quarter_start.strftime('%Y-%m-%d')}")
print(f"🌟 분석 대상: MAGNIFICENT SEVEN (7개 종목)")

# 공매도 분석 요약
low_short = df_results[df_results['short_percent_float'] < 5]
print(f"\n✅ 낮은 공매도 (<5%): {len(low_short)}개")
if not low_short.empty:
    print(f"   → {', '.join(low_short['Ticker'].tolist())}")

high_short = df_results[df_results['short_percent_float'] >= 10]
print(f"⚠️ 높은 공매도 (≥10%): {len(high_short)}개")
if not high_short.empty:
    print(f"   → {', '.join(high_short['Ticker'].tolist())}")

# 종합 추천
top_score = df_results[df_results['Total_Investment_Score'] >= 90]
print(f"\n💚 최우선 매수 (90점↑): {len(top_score)}개")
if not top_score.empty:
    print(f"   → {', '.join(top_score['Ticker'].tolist())}")

strong_buy = df_results[(df_results['Total_Investment_Score'] >= 75) &
                        (df_results['Total_Investment_Score'] < 90)]
print(f"💛 강력 매수 (75-90점): {len(strong_buy)}개")
if not strong_buy.empty:
    print(f"   → {', '.join(strong_buy['Ticker'].tolist())}")

print("\n🏆 TOP 3 추천 (공매도 고려):")
for idx in range(min(3, len(df_results))):
    row = df_results.iloc[idx]
    print(f"  {idx+1}. {row['Ticker']} - {row['Company']}")
    print(f"     💰 시총: ${row['Market_Cap_Trillion']:.2f}T | " +
          f"종합점수: {row['Total_Investment_Score']:.0f}/120")
    print(f"     📊 VWAP: {row['Price_vs_VWAP_%']:+.2f}% | " +
          f"분기수익: {row['Quarter_Return_%']:+.2f}% | " +
          f"공매도: {row['short_percent_float']:.2f}%")

print("\n💡 투자 가이드 (공매도 포함):")
print("""
1. 💚 최우선 매수 (90점 이상):
   - VWAP 위 + 낮은 공매도 비율
   - 즉시 매수 검토, 강한 상승 모멘텀 예상

2. 💛 강력 매수 (75-90점):
   - 기술적으로 양호하나 공매도 약간 존재
   - VWAP 근처 조정 시 매수 기회

3. 💙 눌림목 대기 (60-75점):
   - 공매도 비율 확인 필수
   - VWAP 돌파 확인 후 매수

4. ⚠️ 공매도 주의사항:
   - 10% 이상: Short Squeeze 가능성 주의
   - 5% 미만: 시장의 신뢰 높음
   - Days to Cover 3일 이상: 변동성 증가 가능
""")

print("\n📊 공매도 데이터 출처:")
for _, row in df_results.iterrows():
    if pd.notna(row.get('data_source')):
        print(f"  • {row['Ticker']}: {row['data_source']}")

print("\n" + "=" * 80)
print("✨ MAGNIFICENT SEVEN 종합 분석 완료!")
print("=" * 80)

🌟 MAGNIFICENT SEVEN (MAG 7) 실시간 종합 분석 + 공매도 데이터

빅테크 7대 기업 리스트:
  • AAPL   - Apple Inc.                     | 아이폰, 생태계, 온디바이스 AI
  • MSFT   - Microsoft Corporation          | 클라우드(Azure), 생성형 AI (OpenAI 대주주)
  • GOOGL  - Alphabet Inc.                  | 구글 검색, 유튜브, AI (Gemini)
  • AMZN   - Amazon.com Inc.                | 전자상거래, 클라우드(AWS) 1위
  • NVDA   - NVIDIA Corporation             | AI 반도체(GPU) 독점적 지배자
  • META   - Meta Platforms Inc.            | 페이스북, 인스타그램, AI(Llama)
  • TSLA   - Tesla Inc.                     | 전기차, 자율주행, 로봇

🎯 Anchored VWAP 분석: 2025 Q4
📍 Anchor Point: 2025-10-01
📅 현재까지 거래일: 58일

🔍 MAG 7 종목 분석 중...

분석 중: AAPL (Apple Inc.)... ✓
분석 중: MSFT (Microsoft Corporation)... ✓
분석 중: GOOGL (Alphabet Inc.)... ✓
분석 중: AMZN (Amazon.com Inc.)... ✓
분석 중: NVDA (NVIDIA Corporation)... ✓
분석 중: META (Meta Platforms Inc.)... ✓
분석 중: TSLA (Tesla Inc.)... ✓

📊 공매도(Short Interest) 데이터 수집 중...
  📊 AAPL 공매도 데이터 수집...   ⚠️ FINRA API 오류 (AAPL): Expecting value: line 1 column 1 (char 0)
✓ 


📈 MAG 7 투자 전략 최종 요약 (공매도 분석 포함)

📍 분석 기준: Anchored VWAP (2025 Q4)
📅 Anchor Point: 2025-10-01
🌟 분석 대상: MAGNIFICENT SEVEN (7개 종목)

✅ 낮은 공매도 (<5%): 7개
   → GOOGL, AAPL, TSLA, NVDA, AMZN, MSFT, META
⚠️ 높은 공매도 (≥10%): 0개

💚 최우선 매수 (90점↑): 2개
   → GOOGL, AAPL
💛 강력 매수 (75-90점): 0개

🏆 TOP 3 추천 (공매도 고려):
  1. GOOGL - Alphabet Inc.
     💰 시총: $3.88T | 종합점수: 100/120
     📊 VWAP: +15.13% | 분기수익: +30.65% | 공매도: 1.07%
  2. AAPL - Apple Inc.
     💰 시총: $4.12T | 종합점수: 95/120
     📊 VWAP: +4.94% | 분기수익: +8.76% | 공매도: 0.78%
  3. TSLA - Tesla Inc.
     💰 시총: $1.42T | 종합점수: 50/120
     📊 VWAP: -1.47% | 분기수익: -7.16% | 공매도: 2.45%

💡 투자 가이드 (공매도 포함):

1. 💚 최우선 매수 (90점 이상):
   - VWAP 위 + 낮은 공매도 비율
   - 즉시 매수 검토, 강한 상승 모멘텀 예상

2. 💛 강력 매수 (75-90점):
   - 기술적으로 양호하나 공매도 약간 존재
   - VWAP 근처 조정 시 매수 기회

3. 💙 눌림목 대기 (60-75점):
   - 공매도 비율 확인 필수
   - VWAP 돌파 확인 후 매수

4. ⚠️ 공매도 주의사항:
   - 10% 이상: Short Squeeze 가능성 주의
   - 5% 미만: 시장의 신뢰 높음
   - Days to Cover 3일 이상: 변동성 증가 가능


📊 공매도 데이터 출처:
  • GOOGL: Yahoo Finance
  • 

In [3]:
# 필요한 라이브러리 설치 및 임포트
!pip install yfinance pandas numpy plotly requests -q

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
import requests
from io import StringIO
import time

warnings.filterwarnings('ignore')
pio.renderers.default = 'colab'

# ==================== FINRA 공매도 데이터 수집 함수 ====================

def get_finra_short_interest(ticker, retries=2):
    """
    FINRA에서 특정 종목의 공매도 잔고(Short Interest) 데이터 수집
    실제 CSV 파일 다운로드 방식 사용
    """
    try:
        # 1. 일별 공매도 거래량 데이터 수집
        daily_data = get_finra_short_volume_csv(ticker, days_back=10)

        # 2. 월별 공매도 잔고 데이터 수집
        monthly_data = get_finra_monthly_short_interest(ticker)

        # 데이터 통합
        result = {}

        if daily_data:
            result.update({
                'daily_short_ratio': daily_data['latest_short_ratio'],
                'avg_daily_short_ratio': daily_data['avg_short_ratio_10d'],
                'daily_data_date': daily_data['latest_date'],
                'daily_data_points': daily_data['data_points']
            })

        if monthly_data:
            result.update({
                'monthly_short_interest': monthly_data.get('short_interest', 0),
                'days_to_cover': monthly_data.get('days_to_cover', 0),
                'settlement_date': monthly_data.get('settlement_date', 'N/A')
            })

        return result if result else None

    except Exception as e:
        return None


def get_finra_short_volume_csv(ticker, days_back=10):
    """
    FINRA에서 일별 공매도 거래량 CSV 파일 다운로드 및 분석
    URL: https://cdn.finra.org/equity/regsho/daily/CNMSshvol[YYYYMMDD].txt
    """
    try:
        today = datetime.now()
        short_volume_data = []

        for days in range(days_back):
            check_date = today - timedelta(days=days)

            # 주말 건너뛰기
            if check_date.weekday() >= 5:  # 토요일(5), 일요일(6)
                continue

            date_str = check_date.strftime('%Y%m%d')

            # FINRA Daily Short Volume URL
            url = f"https://cdn.finra.org/equity/regsho/daily/CNMSshvol{date_str}.txt"

            try:
                response = requests.get(url, timeout=10)

                if response.status_code == 200:
                    # 파이프(|) 구분자로 CSV 읽기
                    df = pd.read_csv(StringIO(response.text), sep='|')

                    # 컬럼명 확인 및 정리
                    if 'Symbol' in df.columns or 'symbol' in df.columns:
                        # 대소문자 무관하게 처리
                        df.columns = df.columns.str.strip()
                        symbol_col = 'Symbol' if 'Symbol' in df.columns else 'symbol'

                        # 해당 종목 데이터 찾기
                        ticker_data = df[df[symbol_col].str.upper() == ticker.upper()]

                        if not ticker_data.empty:
                            row = ticker_data.iloc[0]

                            short_vol = row.get('ShortVolume', row.get('shortVolume', 0))
                            total_vol = row.get('TotalVolume', row.get('totalVolume', 0))

                            if pd.notna(short_vol) and pd.notna(total_vol) and total_vol > 0:
                                short_volume_data.append({
                                    'date': check_date.strftime('%Y-%m-%d'),
                                    'short_volume': int(short_vol),
                                    'total_volume': int(total_vol),
                                    'short_ratio': round(short_vol / total_vol * 100, 2)
                                })
            except:
                continue

        if short_volume_data:
            # 데이터프레임으로 변환
            df_short = pd.DataFrame(short_volume_data)

            # 최근 데이터 통계
            avg_short_ratio = df_short['short_ratio'].mean()
            latest_short_ratio = df_short.iloc[0]['short_ratio'] if len(df_short) > 0 else 0

            return {
                'ticker': ticker,
                'latest_date': df_short.iloc[0]['date'] if len(df_short) > 0 else 'N/A',
                'latest_short_ratio': latest_short_ratio,
                'avg_short_ratio_10d': round(avg_short_ratio, 2),
                'data_points': len(df_short),
                'historical_data': df_short
            }

        return None

    except Exception as e:
        return None


def get_finra_monthly_short_interest(ticker):
    """
    FINRA 월별 공매도 잔고 데이터 수집
    URL: https://cdn.finra.org/equity/regsho/monthly/[FILENAME].txt
    """
    try:
        # 최근 6개월 시도
        today = datetime.now()

        for months_back in range(6):
            check_date = today - timedelta(days=30 * months_back)

            # 월별 파일명 형식: CNMSshvol{YYYYMM}a.txt, CNMSshvol{YYYYMM}b.txt
            year_month = check_date.strftime('%Y%m')

            for suffix in ['a', 'b']:
                url = f"https://cdn.finra.org/equity/regsho/monthly/CNMSshvol{year_month}{suffix}.txt"

                try:
                    response = requests.get(url, timeout=10)

                    if response.status_code == 200:
                        df = pd.read_csv(StringIO(response.text), sep='|')

                        if 'Symbol' in df.columns or 'symbol' in df.columns:
                            df.columns = df.columns.str.strip()
                            symbol_col = 'Symbol' if 'Symbol' in df.columns else 'symbol'

                            ticker_data = df[df[symbol_col].str.upper() == ticker.upper()]

                            if not ticker_data.empty:
                                row = ticker_data.iloc[0]

                                return {
                                    'ticker': ticker,
                                    'settlement_date': row.get('SettlementDate', 'N/A'),
                                    'short_interest': row.get('ShortInterest', 0),
                                    'avg_daily_volume': row.get('AverageDailyVolume', 0),
                                    'days_to_cover': row.get('DaysToCover', 0)
                                }
                except:
                    continue

        return None

    except Exception as e:
        return None


def get_short_interest_from_yfinance(ticker):
    """
    Yahoo Finance에서 공매도 관련 데이터 수집 (대체 방법)
    """
    try:
        stock = yf.Ticker(ticker)
        info = stock.info

        short_data = {
            'ticker': ticker,
            'short_ratio': info.get('shortRatio', 0),  # Days to Cover
            'short_percent_float': info.get('shortPercentOfFloat', 0) * 100 if info.get('shortPercentOfFloat') else 0,
            'shares_short': info.get('sharesShort', 0),
            'shares_short_prior_month': info.get('sharesShortPriorMonth', 0),
            'short_percent_shares_out': info.get('sharesPercentSharesOut', 0) * 100 if info.get('sharesPercentSharesOut') else 0
        }

        # 변화율 계산
        if short_data['shares_short_prior_month'] > 0:
            short_data['short_change_pct'] = ((short_data['shares_short'] - short_data['shares_short_prior_month']) /
                                               short_data['shares_short_prior_month'] * 100)
        else:
            short_data['short_change_pct'] = 0

        return short_data

    except Exception as e:
        print(f"  ⚠️ Yahoo Finance 오류 ({ticker}): {str(e)}")
        return None


def get_comprehensive_short_data(ticker):
    """
    여러 소스에서 공매도 데이터 종합 수집
    - Yahoo Finance: 공매도 잔고 (Short Interest)
    - FINRA CSV: 일별 공매도 거래량 비율
    """
    # Yahoo Finance 데이터 (공매도 잔고)
    yf_data = get_short_interest_from_yfinance(ticker)

    # FINRA CSV 데이터 (일별 거래량 비율)
    finra_data = get_finra_short_interest(ticker)

    # 데이터 통합
    combined_data = {
        'ticker': ticker,
        'short_ratio_days': 0,
        'short_percent_float': 0,
        'shares_short_millions': 0,
        'short_change_pct': 0,
        'daily_short_ratio': 0,
        'avg_daily_short_ratio_10d': 0,
        'data_source': []
    }

    if yf_data:
        combined_data.update({
            'short_ratio_days': round(yf_data.get('short_ratio', 0), 2),
            'short_percent_float': round(yf_data.get('short_percent_float', 0), 2),
            'shares_short_millions': round(yf_data.get('shares_short', 0) / 1e6, 2),
            'short_change_pct': round(yf_data.get('short_change_pct', 0), 2),
        })
        combined_data['data_source'].append('Yahoo Finance')

    if finra_data:
        # FINRA 일별 데이터 추가
        if 'daily_short_ratio' in finra_data:
            combined_data['daily_short_ratio'] = finra_data['daily_short_ratio']
            combined_data['avg_daily_short_ratio_10d'] = finra_data['avg_daily_short_ratio']
            combined_data['data_source'].append(f"FINRA Daily ({finra_data.get('daily_data_points', 0)}일)")

        # FINRA 월별 데이터로 Days to Cover 업데이트
        if 'days_to_cover' in finra_data and finra_data['days_to_cover'] > 0:
            combined_data['short_ratio_days'] = round(finra_data['days_to_cover'], 2)
            combined_data['data_source'].append('FINRA Monthly')

    # 데이터 소스를 문자열로 변환
    combined_data['data_source'] = ' + '.join(combined_data['data_source']) if combined_data['data_source'] else 'N/A'

    return combined_data


# ==================== Magnificent Seven 정의 ====================

MAG7_STOCKS = {
    'AAPL': {
        'name': 'Apple Inc.',
        'description': '아이폰, 생태계, 온디바이스 AI',
        'sector': 'Technology',
        'industry': 'Consumer Electronics'
    },
    'MSFT': {
        'name': 'Microsoft Corporation',
        'description': '클라우드(Azure), 생성형 AI (OpenAI 대주주)',
        'sector': 'Technology',
        'industry': 'Software'
    },
    'GOOGL': {
        'name': 'Alphabet Inc.',
        'description': '구글 검색, 유튜브, AI (Gemini)',
        'sector': 'Communication Services',
        'industry': 'Internet Content & Information'
    },
    'AMZN': {
        'name': 'Amazon.com Inc.',
        'description': '전자상거래, 클라우드(AWS) 1위',
        'sector': 'Consumer Cyclical',
        'industry': 'Internet Retail'
    },
    'NVDA': {
        'name': 'NVIDIA Corporation',
        'description': 'AI 반도체(GPU) 독점적 지배자',
        'sector': 'Technology',
        'industry': 'Semiconductors'
    },
    'META': {
        'name': 'Meta Platforms Inc.',
        'description': '페이스북, 인스타그램, AI(Llama)',
        'sector': 'Communication Services',
        'industry': 'Internet Content & Information'
    },
    'TSLA': {
        'name': 'Tesla Inc.',
        'description': '전기차, 자율주행, 로봇',
        'sector': 'Consumer Cyclical',
        'industry': 'Auto Manufacturers'
    }
}

print("=" * 80)
print("🌟 MAGNIFICENT SEVEN (MAG 7) 실시간 종합 분석 + 공매도 데이터")
print("=" * 80)
print("\n빅테크 7대 기업 리스트:")
for ticker, info in MAG7_STOCKS.items():
    print(f"  • {ticker:6} - {info['name']:30} | {info['description']}")
print("=" * 80)


def get_current_quarter_start():
    """현재 분기 시작일 계산"""
    now = datetime.now()
    quarter = (now.month - 1) // 3
    quarter_start_month = quarter * 3 + 1
    quarter_start = datetime(now.year, quarter_start_month, 1)
    return quarter_start


def calculate_anchored_vwap(df):
    """Anchored VWAP 계산"""
    df = df.copy()
    df['Typical_Price'] = (df['High'] + df['Low'] + df['Close']) / 3
    df['TP_Volume'] = df['Typical_Price'] * df['Volume']
    df['Cumulative_TP_Volume'] = df['TP_Volume'].cumsum()
    df['Cumulative_Volume'] = df['Volume'].cumsum()
    df['Anchored_VWAP'] = df['Cumulative_TP_Volume'] / df['Cumulative_Volume']
    return df


def get_quarterly_vwap_analysis(ticker):
    """분기별 Anchored VWAP 분석"""
    try:
        quarter_start = get_current_quarter_start()
        end_date = datetime.now()
        quarter_num = (quarter_start.month - 1) // 3 + 1

        stock = yf.Ticker(ticker)
        df = stock.history(start=quarter_start, end=end_date)

        if df.empty or len(df) < 5:
            return None

        df = calculate_anchored_vwap(df)

        current_price = df['Close'].iloc[-1]
        current_vwap = df['Anchored_VWAP'].iloc[-1]
        above_vwap_ratio = (df['Close'] > df['Anchored_VWAP']).sum() / len(df) * 100
        recent_5days_avg = df['Close'].tail(5).mean()
        recent_10days_avg = df['Close'].tail(10).mean()

        recent_20 = df['Close'].tail(min(20, len(df)))
        uptrend_strength = (recent_20.diff() > 0).sum() / len(recent_20) * 100 if len(recent_20) > 1 else 50

        recent_volume = df['Volume'].tail(5).mean()
        avg_volume = df['Volume'].mean()
        volume_ratio = recent_volume / avg_volume if avg_volume > 0 else 1

        info = stock.info
        quarter_start_price = df['Close'].iloc[0]
        quarter_return = ((current_price - quarter_start_price) / quarter_start_price * 100)

        return {
            'Ticker': ticker,
            'Company': MAG7_STOCKS[ticker]['name'],
            'Description': MAG7_STOCKS[ticker]['description'],
            'Sector': MAG7_STOCKS[ticker]['sector'],
            'Quarter': f'{quarter_start.year} Q{quarter_num}',
            'Quarter_Start_Date': quarter_start.strftime('%Y-%m-%d'),
            'Trading_Days': len(df),
            'Current_Price': round(current_price, 2),
            'Anchored_VWAP': round(current_vwap, 2),
            'Quarter_Start_Price': round(quarter_start_price, 2),
            'Quarter_Return_%': round(quarter_return, 2),
            'Price_vs_VWAP_%': round((current_price - current_vwap) / current_vwap * 100, 2),
            'Above_VWAP_Days_%': round(above_vwap_ratio, 1),
            'Recent_5D_Avg': round(recent_5days_avg, 2),
            'Recent_10D_Avg': round(recent_10days_avg, 2),
            'Uptrend_Strength_%': round(uptrend_strength, 1),
            'Volume_Ratio': round(volume_ratio, 2),
            'Is_Above_VWAP': current_price > current_vwap,
            'Market_Cap': info.get('marketCap', 0),
            'Buy_Signal_Score': 0
        }

    except Exception as e:
        print(f"Error processing {ticker}: {str(e)}")
        return None


# ==================== 메인 분석 시작 ====================

quarter_start = get_current_quarter_start()
quarter_num = (quarter_start.month - 1) // 3 + 1

print("\n" + "=" * 80)
print(f"🎯 Anchored VWAP 분석: {quarter_start.year} Q{quarter_num}")
print(f"📍 Anchor Point: {quarter_start.strftime('%Y-%m-%d')}")
print(f"📅 현재까지 거래일: {(datetime.now() - quarter_start).days}일")
print("=" * 80)

# MAG 7 분석
results = []
mag7_tickers = list(MAG7_STOCKS.keys())

print("\n🔍 MAG 7 종목 분석 중...\n")

for ticker in mag7_tickers:
    print(f"분석 중: {ticker} ({MAG7_STOCKS[ticker]['name']})...", end=" ")
    result = get_quarterly_vwap_analysis(ticker)
    if result:
        results.append(result)
        print("✓")
    else:
        print("✗")

df_results = pd.DataFrame(results)

# 매수 신호 점수 계산
def calculate_buy_score(row):
    score = 0
    if row['Is_Above_VWAP']:
        score += 30

    price_diff = row['Price_vs_VWAP_%']
    if 0 < price_diff <= 5:
        score += 20
    elif 5 < price_diff <= 10:
        score += 10
    elif price_diff > 10:
        score += 5

    if row['Above_VWAP_Days_%'] >= 80:
        score += 20
    elif row['Above_VWAP_Days_%'] >= 60:
        score += 15
    elif row['Above_VWAP_Days_%'] >= 40:
        score += 10

    if row['Uptrend_Strength_%'] >= 60:
        score += 15
    elif row['Uptrend_Strength_%'] >= 50:
        score += 10

    if row['Volume_Ratio'] >= 1.2:
        score += 15
    elif row['Volume_Ratio'] >= 1.0:
        score += 10

    return min(score, 100)

df_results['Buy_Signal_Score'] = df_results.apply(calculate_buy_score, axis=1)

# 시가총액 추가
df_results['Market_Cap_Trillion'] = (df_results['Market_Cap'] / 1e12).round(3)

# ==================== 공매도 데이터 수집 ====================

print("\n" + "=" * 80)
print("📊 공매도(Short Interest) 데이터 수집 중...")
print("=" * 80)

short_data_list = []

for ticker in mag7_tickers:
    print(f"  📊 {ticker} 공매도 데이터 수집...", end=" ")
    short_data = get_comprehensive_short_data(ticker)
    if short_data:
        short_data_list.append(short_data)
        print("✓")
    else:
        print("✗")

df_short = pd.DataFrame(short_data_list)

# 공매도 데이터를 메인 데이터프레임에 병합
df_results = df_results.merge(df_short, left_on='Ticker', right_on='ticker', how='left')

# 공매도 스코어 계산 (낮을수록 좋음)
def calculate_short_score(row):
    """
    공매도 스코어: 공매도 비율이 낮을수록 높은 점수
    - 5% 미만: 매우 좋음 (20점)
    - 5-10%: 양호 (15점)
    - 10-20%: 보통 (10점)
    - 20% 이상: 주의 (5점)
    """
    short_pct = row.get('short_percent_float', 0)

    if short_pct < 5:
        return 20
    elif short_pct < 10:
        return 15
    elif short_pct < 20:
        return 10
    else:
        return 5

df_results['Short_Score'] = df_results.apply(calculate_short_score, axis=1)

# 종합 투자 점수 (기존 Buy Signal + Short Score)
df_results['Total_Investment_Score'] = df_results['Buy_Signal_Score'] + df_results['Short_Score']
df_results = df_results.sort_values('Total_Investment_Score', ascending=False)

# ==================== 결과 출력 ====================

print("\n" + "=" * 80)
print("📊 MAGNIFICENT SEVEN - 종합 분석 결과 (공매도 포함)")
print("=" * 80)

display_cols = ['Ticker', 'Company', 'Current_Price', 'Anchored_VWAP', 'Price_vs_VWAP_%',
                'Quarter_Return_%', 'short_percent_float', 'short_ratio_days',
                'daily_short_ratio', 'Buy_Signal_Score', 'Short_Score', 'Total_Investment_Score']

print(df_results[display_cols].to_string(index=False))

print("\n" + "=" * 80)
print("🏆 MAG 7 투자 추천 순위 (공매도 분석 포함)")
print("=" * 80)

for idx, row in df_results.iterrows():
    rank = df_results.index.get_loc(idx) + 1

    if rank == 1:
        medal = "🥇"
    elif rank == 2:
        medal = "🥈"
    elif rank == 3:
        medal = "🥉"
    else:
        medal = f"  {rank}"

    print(f"\n{medal} {row['Ticker']} - {row['Company']}")
    print(f"    🎯 {row['Description']}")
    print(f"    💰 시가총액: ${row['Market_Cap_Trillion']:.2f}T")
    print(f"    📈 현재가: ${row['Current_Price']} | VWAP: ${row['Anchored_VWAP']}")
    print(f"    📊 VWAP 대비: {row['Price_vs_VWAP_%']:+.2f}% | 분기 수익률: {row['Quarter_Return_%']:+.2f}%")
    print(f"    🎲 VWAP 위 거래일: {row['Above_VWAP_Days_%']}% | 추세 강도: {row['Uptrend_Strength_%']}%")

    # 공매도 정보
    short_pct = row.get('short_percent_float', 0)
    short_days = row.get('short_ratio_days', 0)
    daily_short = row.get('daily_short_ratio', 0)

    print(f"    🔴 공매도 비율: {short_pct:.2f}% of float | 커버 소요일: {short_days:.1f}일")

    if daily_short > 0:
        print(f"    📊 최근 일평균 공매도 거래비율: {daily_short:.1f}%")

    if short_pct < 5:
        short_status = "✅ 매우 낮음 (긍정)"
    elif short_pct < 10:
        short_status = "🟢 양호"
    elif short_pct < 20:
        short_status = "🟡 보통"
    else:
        short_status = "🔴 높음 (주의)"

    print(f"    🎯 공매도 상태: {short_status}")

    print(f"    ⭐ 기술적 점수: {row['Buy_Signal_Score']}/100 | 공매도 점수: {row['Short_Score']}/20", end="")
    print(f" | 종합: {row['Total_Investment_Score']}/120")

    if row['Total_Investment_Score'] >= 90:
        print("       → 💚 최우선 매수 추천")
    elif row['Total_Investment_Score'] >= 75:
        print("       → 💛 강력 매수")
    elif row['Total_Investment_Score'] >= 60:
        print("       → 💙 눌림목 대기")
    else:
        print("       → ⚪ 관찰 필요")

# ==================== 공매도 관련 차트 ====================

print("\n📈 공매도 분석 차트 생성 중...")

# 차트 1: 공매도 비율 비교
fig_short = go.Figure()

colors_short = ['#00C853' if x < 5 else '#FFD600' if x < 10 else '#FF5722'
                for x in df_results['short_percent_float']]

fig_short.add_trace(go.Bar(
    y=df_results['Ticker'],
    x=df_results['short_percent_float'],
    orientation='h',
    marker=dict(color=colors_short),
    text=df_results['short_percent_float'].round(2),
    textposition='auto',
    hovertemplate='<b>%{y}</b><br>공매도 비율: %{x:.2f}%<extra></extra>'
))

fig_short.update_layout(
    title='🔴 MAG 7 - 공매도 비율 (Short % of Float)',
    xaxis_title='공매도 비율 (%)',
    yaxis_title='종목',
    height=500,
    template='plotly_white'
)

# 참고선 추가
fig_short.add_vline(x=5, line_dash="dash", line_color="green",
                    annotation_text="낮음", annotation_position="top")
fig_short.add_vline(x=10, line_dash="dash", line_color="orange",
                    annotation_text="보통", annotation_position="top")
fig_short.add_vline(x=20, line_dash="dash", line_color="red",
                    annotation_text="높음", annotation_position="top")

fig_short.show()

# 차트 2: 종합 투자 점수 (기술적 + 공매도)
fig_total = make_subplots(
    rows=1, cols=2,
    subplot_titles=('기술적 분석 점수', '종합 투자 점수'),
    specs=[[{"type": "bar"}, {"type": "bar"}]]
)

fig_total.add_trace(
    go.Bar(
        y=df_results['Ticker'],
        x=df_results['Buy_Signal_Score'],
        orientation='h',
        name='기술적 점수',
        marker_color='#2196F3',
        text=df_results['Buy_Signal_Score'],
        textposition='auto'
    ),
    row=1, col=1
)

fig_total.add_trace(
    go.Bar(
        y=df_results['Ticker'],
        x=df_results['Total_Investment_Score'],
        orientation='h',
        name='종합 점수',
        marker_color='#4CAF50',
        text=df_results['Total_Investment_Score'],
        textposition='auto'
    ),
    row=1, col=2
)

fig_total.update_xaxes(title_text="점수", row=1, col=1)
fig_total.update_xaxes(title_text="점수", row=1, col=2)
fig_total.update_yaxes(title_text="종목", row=1, col=1)

fig_total.update_layout(
    title_text='📊 MAG 7 - 투자 점수 비교 (기술적 vs 종합)',
    height=500,
    showlegend=False,
    template='plotly_white'
)

fig_total.show()

# 차트 3: 공매도 vs 분기 수익률
fig_scatter = px.scatter(
    df_results,
    x='short_percent_float',
    y='Quarter_Return_%',
    size='Market_Cap_Trillion',
    color='Total_Investment_Score',
    hover_data=['Ticker', 'Company'],
    text='Ticker',
    color_continuous_scale='RdYlGn',
    title='📊 MAG 7 - 공매도 비율 vs 분기 수익률',
    labels={
        'short_percent_float': '공매도 비율 (%)',
        'Quarter_Return_%': '분기 수익률 (%)',
        'Total_Investment_Score': '종합 점수'
    }
)

fig_scatter.update_traces(textposition='top center', textfont_size=12)
fig_scatter.update_layout(height=600)
fig_scatter.show()

# ==================== 최종 요약 ====================

print("\n" + "=" * 80)
print("📈 MAG 7 투자 전략 최종 요약 (공매도 분석 포함)")
print("=" * 80)

print(f"\n📍 분석 기준: Anchored VWAP ({quarter_start.year} Q{quarter_num})")
print(f"📅 Anchor Point: {quarter_start.strftime('%Y-%m-%d')}")
print(f"🌟 분석 대상: MAGNIFICENT SEVEN (7개 종목)")

# 공매도 분석 요약
low_short = df_results[df_results['short_percent_float'] < 5]
print(f"\n✅ 낮은 공매도 (<5%): {len(low_short)}개")
if not low_short.empty:
    print(f"   → {', '.join(low_short['Ticker'].tolist())}")

high_short = df_results[df_results['short_percent_float'] >= 10]
print(f"⚠️ 높은 공매도 (≥10%): {len(high_short)}개")
if not high_short.empty:
    print(f"   → {', '.join(high_short['Ticker'].tolist())}")

# 종합 추천
top_score = df_results[df_results['Total_Investment_Score'] >= 90]
print(f"\n💚 최우선 매수 (90점↑): {len(top_score)}개")
if not top_score.empty:
    print(f"   → {', '.join(top_score['Ticker'].tolist())}")

strong_buy = df_results[(df_results['Total_Investment_Score'] >= 75) &
                        (df_results['Total_Investment_Score'] < 90)]
print(f"💛 강력 매수 (75-90점): {len(strong_buy)}개")
if not strong_buy.empty:
    print(f"   → {', '.join(strong_buy['Ticker'].tolist())}")

print("\n🏆 TOP 3 추천 (공매도 고려):")
for idx in range(min(3, len(df_results))):
    row = df_results.iloc[idx]
    print(f"  {idx+1}. {row['Ticker']} - {row['Company']}")
    print(f"     💰 시총: ${row['Market_Cap_Trillion']:.2f}T | " +
          f"종합점수: {row['Total_Investment_Score']:.0f}/120")
    print(f"     📊 VWAP: {row['Price_vs_VWAP_%']:+.2f}% | " +
          f"분기수익: {row['Quarter_Return_%']:+.2f}% | " +
          f"공매도: {row['short_percent_float']:.2f}%")

print("\n💡 투자 가이드 (공매도 포함):")
print("""
1. 💚 최우선 매수 (90점 이상):
   - VWAP 위 + 낮은 공매도 비율
   - 즉시 매수 검토, 강한 상승 모멘텀 예상

2. 💛 강력 매수 (75-90점):
   - 기술적으로 양호하나 공매도 약간 존재
   - VWAP 근처 조정 시 매수 기회

3. 💙 눌림목 대기 (60-75점):
   - 공매도 비율 확인 필수
   - VWAP 돌파 확인 후 매수

4. ⚠️ 공매도 주의사항:
   - 10% 이상: Short Squeeze 가능성 주의
   - 5% 미만: 시장의 신뢰 높음
   - Days to Cover 3일 이상: 변동성 증가 가능
""")

print("\n📊 공매도 데이터 출처:")
for _, row in df_results.iterrows():
    if pd.notna(row.get('data_source')):
        print(f"  • {row['Ticker']}: {row['data_source']}")

print("\n" + "=" * 80)
print("✨ MAGNIFICENT SEVEN 종합 분석 완료!")
print("=" * 80)

🌟 MAGNIFICENT SEVEN (MAG 7) 실시간 종합 분석 + 공매도 데이터

빅테크 7대 기업 리스트:
  • AAPL   - Apple Inc.                     | 아이폰, 생태계, 온디바이스 AI
  • MSFT   - Microsoft Corporation          | 클라우드(Azure), 생성형 AI (OpenAI 대주주)
  • GOOGL  - Alphabet Inc.                  | 구글 검색, 유튜브, AI (Gemini)
  • AMZN   - Amazon.com Inc.                | 전자상거래, 클라우드(AWS) 1위
  • NVDA   - NVIDIA Corporation             | AI 반도체(GPU) 독점적 지배자
  • META   - Meta Platforms Inc.            | 페이스북, 인스타그램, AI(Llama)
  • TSLA   - Tesla Inc.                     | 전기차, 자율주행, 로봇

🎯 Anchored VWAP 분석: 2025 Q4
📍 Anchor Point: 2025-10-01
📅 현재까지 거래일: 58일

🔍 MAG 7 종목 분석 중...

분석 중: AAPL (Apple Inc.)... ✓
분석 중: MSFT (Microsoft Corporation)... ✓
분석 중: GOOGL (Alphabet Inc.)... ✓
분석 중: AMZN (Amazon.com Inc.)... ✓
분석 중: NVDA (NVIDIA Corporation)... ✓
분석 중: META (Meta Platforms Inc.)... ✓
분석 중: TSLA (Tesla Inc.)... ✓

📊 공매도(Short Interest) 데이터 수집 중...
  📊 AAPL 공매도 데이터 수집... ✓
  📊 MSFT 공매도 데이터 수집... ✓
  📊 GOOGL 공매도 데이터 수집... ✓
  📊 AMZN 공매도 데이터 


📈 MAG 7 투자 전략 최종 요약 (공매도 분석 포함)

📍 분석 기준: Anchored VWAP (2025 Q4)
📅 Anchor Point: 2025-10-01
🌟 분석 대상: MAGNIFICENT SEVEN (7개 종목)

✅ 낮은 공매도 (<5%): 7개
   → GOOGL, AAPL, TSLA, NVDA, AMZN, MSFT, META
⚠️ 높은 공매도 (≥10%): 0개

💚 최우선 매수 (90점↑): 2개
   → GOOGL, AAPL
💛 강력 매수 (75-90점): 0개

🏆 TOP 3 추천 (공매도 고려):
  1. GOOGL - Alphabet Inc.
     💰 시총: $3.88T | 종합점수: 100/120
     📊 VWAP: +15.13% | 분기수익: +30.65% | 공매도: 1.07%
  2. AAPL - Apple Inc.
     💰 시총: $4.12T | 종합점수: 95/120
     📊 VWAP: +4.94% | 분기수익: +8.76% | 공매도: 0.78%
  3. TSLA - Tesla Inc.
     💰 시총: $1.42T | 종합점수: 50/120
     📊 VWAP: -1.47% | 분기수익: -7.16% | 공매도: 2.45%

💡 투자 가이드 (공매도 포함):

1. 💚 최우선 매수 (90점 이상):
   - VWAP 위 + 낮은 공매도 비율
   - 즉시 매수 검토, 강한 상승 모멘텀 예상

2. 💛 강력 매수 (75-90점):
   - 기술적으로 양호하나 공매도 약간 존재
   - VWAP 근처 조정 시 매수 기회

3. 💙 눌림목 대기 (60-75점):
   - 공매도 비율 확인 필수
   - VWAP 돌파 확인 후 매수

4. ⚠️ 공매도 주의사항:
   - 10% 이상: Short Squeeze 가능성 주의
   - 5% 미만: 시장의 신뢰 높음
   - Days to Cover 3일 이상: 변동성 증가 가능


📊 공매도 데이터 출처:
  • GOOGL: Yahoo Finance + FI

In [4]:
# 필요한 라이브러리 설치 및 임포트
!pip install yfinance pandas numpy plotly requests -q

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
import requests
from io import StringIO
import time

warnings.filterwarnings('ignore')
pio.renderers.default = 'colab'

# ==================== FINRA 공매도 데이터 수집 함수 ====================

def get_finra_short_interest(ticker, retries=2):
    """
    FINRA에서 특정 종목의 공매도 잔고(Short Interest) 데이터 수집
    실제 CSV 파일 다운로드 방식 사용
    """
    try:
        # 1. 일별 공매도 거래량 데이터 수집
        daily_data = get_finra_short_volume_csv(ticker, days_back=10)

        # 2. 월별 공매도 잔고 데이터 수집
        monthly_data = get_finra_monthly_short_interest(ticker)

        # 데이터 통합
        result = {}

        if daily_data:
            result.update({
                'daily_short_ratio': daily_data['latest_short_ratio'],
                'avg_daily_short_ratio': daily_data['avg_short_ratio_10d'],
                'daily_data_date': daily_data['latest_date'],
                'daily_data_points': daily_data['data_points']
            })

        if monthly_data:
            result.update({
                'monthly_short_interest': monthly_data.get('short_interest', 0),
                'days_to_cover': monthly_data.get('days_to_cover', 0),
                'settlement_date': monthly_data.get('settlement_date', 'N/A')
            })

        return result if result else None

    except Exception as e:
        return None


def get_finra_short_volume_csv(ticker, days_back=10):
    """
    FINRA에서 일별 공매도 거래량 CSV 파일 다운로드 및 분석
    URL: https://cdn.finra.org/equity/regsho/daily/CNMSshvol[YYYYMMDD].txt
    """
    try:
        today = datetime.now()
        short_volume_data = []

        for days in range(days_back):
            check_date = today - timedelta(days=days)

            # 주말 건너뛰기
            if check_date.weekday() >= 5:  # 토요일(5), 일요일(6)
                continue

            date_str = check_date.strftime('%Y%m%d')

            # FINRA Daily Short Volume URL
            url = f"https://cdn.finra.org/equity/regsho/daily/CNMSshvol{date_str}.txt"

            try:
                response = requests.get(url, timeout=10)

                if response.status_code == 200:
                    # 파이프(|) 구분자로 CSV 읽기
                    df = pd.read_csv(StringIO(response.text), sep='|')

                    # 컬럼명 확인 및 정리
                    if 'Symbol' in df.columns or 'symbol' in df.columns:
                        # 대소문자 무관하게 처리
                        df.columns = df.columns.str.strip()
                        symbol_col = 'Symbol' if 'Symbol' in df.columns else 'symbol'

                        # 해당 종목 데이터 찾기
                        ticker_data = df[df[symbol_col].str.upper() == ticker.upper()]

                        if not ticker_data.empty:
                            row = ticker_data.iloc[0]

                            short_vol = row.get('ShortVolume', row.get('shortVolume', 0))
                            total_vol = row.get('TotalVolume', row.get('totalVolume', 0))

                            if pd.notna(short_vol) and pd.notna(total_vol) and total_vol > 0:
                                short_volume_data.append({
                                    'date': check_date.strftime('%Y-%m-%d'),
                                    'short_volume': int(short_vol),
                                    'total_volume': int(total_vol),
                                    'short_ratio': round(short_vol / total_vol * 100, 2)
                                })
            except:
                continue

        if short_volume_data:
            # 데이터프레임으로 변환
            df_short = pd.DataFrame(short_volume_data)

            # 최근 데이터 통계
            avg_short_ratio = df_short['short_ratio'].mean()
            latest_short_ratio = df_short.iloc[0]['short_ratio'] if len(df_short) > 0 else 0

            return {
                'ticker': ticker,
                'latest_date': df_short.iloc[0]['date'] if len(df_short) > 0 else 'N/A',
                'latest_short_ratio': latest_short_ratio,
                'avg_short_ratio_10d': round(avg_short_ratio, 2),
                'data_points': len(df_short),
                'historical_data': df_short
            }

        return None

    except Exception as e:
        return None


def get_finra_monthly_short_interest(ticker):
    """
    FINRA 월별 공매도 잔고 데이터 수집
    URL: https://cdn.finra.org/equity/regsho/monthly/[FILENAME].txt
    """
    try:
        # 최근 6개월 시도
        today = datetime.now()

        for months_back in range(6):
            check_date = today - timedelta(days=30 * months_back)

            # 월별 파일명 형식: CNMSshvol{YYYYMM}a.txt, CNMSshvol{YYYYMM}b.txt
            year_month = check_date.strftime('%Y%m')

            for suffix in ['a', 'b']:
                url = f"https://cdn.finra.org/equity/regsho/monthly/CNMSshvol{year_month}{suffix}.txt"

                try:
                    response = requests.get(url, timeout=10)

                    if response.status_code == 200:
                        df = pd.read_csv(StringIO(response.text), sep='|')

                        if 'Symbol' in df.columns or 'symbol' in df.columns:
                            df.columns = df.columns.str.strip()
                            symbol_col = 'Symbol' if 'Symbol' in df.columns else 'symbol'

                            ticker_data = df[df[symbol_col].str.upper() == ticker.upper()]

                            if not ticker_data.empty:
                                row = ticker_data.iloc[0]

                                return {
                                    'ticker': ticker,
                                    'settlement_date': row.get('SettlementDate', 'N/A'),
                                    'short_interest': row.get('ShortInterest', 0),
                                    'avg_daily_volume': row.get('AverageDailyVolume', 0),
                                    'days_to_cover': row.get('DaysToCover', 0)
                                }
                except:
                    continue

        return None

    except Exception as e:
        return None


def get_short_interest_from_yfinance(ticker):
    """
    Yahoo Finance에서 공매도 관련 데이터 수집 (대체 방법)
    """
    try:
        stock = yf.Ticker(ticker)
        info = stock.info

        short_data = {
            'ticker': ticker,
            'short_ratio': info.get('shortRatio', 0),  # Days to Cover
            'short_percent_float': info.get('shortPercentOfFloat', 0) * 100 if info.get('shortPercentOfFloat') else 0,
            'shares_short': info.get('sharesShort', 0),
            'shares_short_prior_month': info.get('sharesShortPriorMonth', 0),
            'short_percent_shares_out': info.get('sharesPercentSharesOut', 0) * 100 if info.get('sharesPercentSharesOut') else 0
        }

        # 변화율 계산
        if short_data['shares_short_prior_month'] > 0:
            short_data['short_change_pct'] = ((short_data['shares_short'] - short_data['shares_short_prior_month']) /
                                               short_data['shares_short_prior_month'] * 100)
        else:
            short_data['short_change_pct'] = 0

        return short_data

    except Exception as e:
        print(f"  ⚠️ Yahoo Finance 오류 ({ticker}): {str(e)}")
        return None


def get_comprehensive_short_data(ticker):
    """
    여러 소스에서 공매도 데이터 종합 수집
    - Yahoo Finance: 공매도 잔고 (Short Interest)
    - FINRA CSV: 일별 공매도 거래량 비율
    """
    # Yahoo Finance 데이터 (공매도 잔고)
    yf_data = get_short_interest_from_yfinance(ticker)

    # FINRA CSV 데이터 (일별 거래량 비율)
    finra_data = get_finra_short_interest(ticker)

    # 데이터 통합
    combined_data = {
        'ticker': ticker,
        'short_ratio_days': 0,
        'short_percent_float': 0,
        'shares_short_millions': 0,
        'short_change_pct': 0,
        'daily_short_ratio': 0,
        'avg_daily_short_ratio_10d': 0,
        'data_source': []
    }

    if yf_data:
        combined_data.update({
            'short_ratio_days': round(yf_data.get('short_ratio', 0), 2),
            'short_percent_float': round(yf_data.get('short_percent_float', 0), 2),
            'shares_short_millions': round(yf_data.get('shares_short', 0) / 1e6, 2),
            'short_change_pct': round(yf_data.get('short_change_pct', 0), 2),
        })
        combined_data['data_source'].append('Yahoo Finance')

    if finra_data:
        # FINRA 일별 데이터 추가
        if 'daily_short_ratio' in finra_data:
            combined_data['daily_short_ratio'] = finra_data['daily_short_ratio']
            combined_data['avg_daily_short_ratio_10d'] = finra_data['avg_daily_short_ratio']
            combined_data['data_source'].append(f"FINRA Daily ({finra_data.get('daily_data_points', 0)}일)")

        # FINRA 월별 데이터로 Days to Cover 업데이트
        if 'days_to_cover' in finra_data and finra_data['days_to_cover'] > 0:
            combined_data['short_ratio_days'] = round(finra_data['days_to_cover'], 2)
            combined_data['data_source'].append('FINRA Monthly')

    # 데이터 소스를 문자열로 변환
    combined_data['data_source'] = ' + '.join(combined_data['data_source']) if combined_data['data_source'] else 'N/A'

    return combined_data


# ==================== Magnificent Seven 정의 ====================

MAG7_STOCKS = {
    'AAPL': {
        'name': 'Apple Inc.',
        'description': '아이폰, 생태계, 온디바이스 AI',
        'sector': 'Technology',
        'industry': 'Consumer Electronics'
    },
    'MSFT': {
        'name': 'Microsoft Corporation',
        'description': '클라우드(Azure), 생성형 AI (OpenAI 대주주)',
        'sector': 'Technology',
        'industry': 'Software'
    },
    'GOOGL': {
        'name': 'Alphabet Inc.',
        'description': '구글 검색, 유튜브, AI (Gemini)',
        'sector': 'Communication Services',
        'industry': 'Internet Content & Information'
    },
    'AMZN': {
        'name': 'Amazon.com Inc.',
        'description': '전자상거래, 클라우드(AWS) 1위',
        'sector': 'Consumer Cyclical',
        'industry': 'Internet Retail'
    },
    'NVDA': {
        'name': 'NVIDIA Corporation',
        'description': 'AI 반도체(GPU) 독점적 지배자',
        'sector': 'Technology',
        'industry': 'Semiconductors'
    },
    'META': {
        'name': 'Meta Platforms Inc.',
        'description': '페이스북, 인스타그램, AI(Llama)',
        'sector': 'Communication Services',
        'industry': 'Internet Content & Information'
    },
    'TSLA': {
        'name': 'Tesla Inc.',
        'description': '전기차, 자율주행, 로봇',
        'sector': 'Consumer Cyclical',
        'industry': 'Auto Manufacturers'
    }
}

print("=" * 80)
print("🌟 MAGNIFICENT SEVEN (MAG 7) 실시간 종합 분석 + 공매도 데이터")
print("=" * 80)
print("\n빅테크 7대 기업 리스트:")
for ticker, info in MAG7_STOCKS.items():
    print(f"  • {ticker:6} - {info['name']:30} | {info['description']}")
print("=" * 80)


def get_current_quarter_start():
    """현재 분기 시작일 계산"""
    now = datetime.now()
    quarter = (now.month - 1) // 3
    quarter_start_month = quarter * 3 + 1
    quarter_start = datetime(now.year, quarter_start_month, 1)
    return quarter_start


def calculate_anchored_vwap(df):
    """Anchored VWAP 계산"""
    df = df.copy()
    df['Typical_Price'] = (df['High'] + df['Low'] + df['Close']) / 3
    df['TP_Volume'] = df['Typical_Price'] * df['Volume']
    df['Cumulative_TP_Volume'] = df['TP_Volume'].cumsum()
    df['Cumulative_Volume'] = df['Volume'].cumsum()
    df['Anchored_VWAP'] = df['Cumulative_TP_Volume'] / df['Cumulative_Volume']
    return df


def get_quarterly_vwap_analysis(ticker):
    """분기별 Anchored VWAP 분석"""
    try:
        quarter_start = get_current_quarter_start()
        end_date = datetime.now()
        quarter_num = (quarter_start.month - 1) // 3 + 1

        stock = yf.Ticker(ticker)
        df = stock.history(start=quarter_start, end=end_date)

        if df.empty or len(df) < 5:
            return None

        df = calculate_anchored_vwap(df)

        current_price = df['Close'].iloc[-1]
        current_vwap = df['Anchored_VWAP'].iloc[-1]
        above_vwap_ratio = (df['Close'] > df['Anchored_VWAP']).sum() / len(df) * 100
        recent_5days_avg = df['Close'].tail(5).mean()
        recent_10days_avg = df['Close'].tail(10).mean()

        recent_20 = df['Close'].tail(min(20, len(df)))
        uptrend_strength = (recent_20.diff() > 0).sum() / len(recent_20) * 100 if len(recent_20) > 1 else 50

        recent_volume = df['Volume'].tail(5).mean()
        avg_volume = df['Volume'].mean()
        volume_ratio = recent_volume / avg_volume if avg_volume > 0 else 1

        info = stock.info
        quarter_start_price = df['Close'].iloc[0]
        quarter_return = ((current_price - quarter_start_price) / quarter_start_price * 100)

        return {
            'Ticker': ticker,
            'Company': MAG7_STOCKS[ticker]['name'],
            'Description': MAG7_STOCKS[ticker]['description'],
            'Sector': MAG7_STOCKS[ticker]['sector'],
            'Quarter': f'{quarter_start.year} Q{quarter_num}',
            'Quarter_Start_Date': quarter_start.strftime('%Y-%m-%d'),
            'Trading_Days': len(df),
            'Current_Price': round(current_price, 2),
            'Anchored_VWAP': round(current_vwap, 2),
            'Quarter_Start_Price': round(quarter_start_price, 2),
            'Quarter_Return_%': round(quarter_return, 2),
            'Price_vs_VWAP_%': round((current_price - current_vwap) / current_vwap * 100, 2),
            'Above_VWAP_Days_%': round(above_vwap_ratio, 1),
            'Recent_5D_Avg': round(recent_5days_avg, 2),
            'Recent_10D_Avg': round(recent_10days_avg, 2),
            'Uptrend_Strength_%': round(uptrend_strength, 1),
            'Volume_Ratio': round(volume_ratio, 2),
            'Is_Above_VWAP': current_price > current_vwap,
            'Market_Cap': info.get('marketCap', 0),
            'Buy_Signal_Score': 0
        }

    except Exception as e:
        print(f"Error processing {ticker}: {str(e)}")
        return None


# ==================== 메인 분석 시작 ====================

quarter_start = get_current_quarter_start()
quarter_num = (quarter_start.month - 1) // 3 + 1

print("\n" + "=" * 80)
print(f"🎯 Anchored VWAP 분석: {quarter_start.year} Q{quarter_num}")
print(f"📍 Anchor Point: {quarter_start.strftime('%Y-%m-%d')}")
print(f"📅 현재까지 거래일: {(datetime.now() - quarter_start).days}일")
print("=" * 80)

# MAG 7 분석
results = []
mag7_tickers = list(MAG7_STOCKS.keys())

print("\n🔍 MAG 7 종목 분석 중...\n")

for ticker in mag7_tickers:
    print(f"분석 중: {ticker} ({MAG7_STOCKS[ticker]['name']})...", end=" ")
    result = get_quarterly_vwap_analysis(ticker)
    if result:
        results.append(result)
        print("✓")
    else:
        print("✗")

df_results = pd.DataFrame(results)

# 매수 신호 점수 계산
def calculate_buy_score(row):
    score = 0
    if row['Is_Above_VWAP']:
        score += 30

    price_diff = row['Price_vs_VWAP_%']
    if 0 < price_diff <= 5:
        score += 20
    elif 5 < price_diff <= 10:
        score += 10
    elif price_diff > 10:
        score += 5

    if row['Above_VWAP_Days_%'] >= 80:
        score += 20
    elif row['Above_VWAP_Days_%'] >= 60:
        score += 15
    elif row['Above_VWAP_Days_%'] >= 40:
        score += 10

    if row['Uptrend_Strength_%'] >= 60:
        score += 15
    elif row['Uptrend_Strength_%'] >= 50:
        score += 10

    if row['Volume_Ratio'] >= 1.2:
        score += 15
    elif row['Volume_Ratio'] >= 1.0:
        score += 10

    return min(score, 100)

df_results['Buy_Signal_Score'] = df_results.apply(calculate_buy_score, axis=1)

# 시가총액 추가
df_results['Market_Cap_Trillion'] = (df_results['Market_Cap'] / 1e12).round(3)

# ==================== 공매도 데이터 수집 ====================

print("\n" + "=" * 80)
print("📊 공매도(Short Interest) 데이터 수집 중...")
print("=" * 80)

short_data_list = []

for ticker in mag7_tickers:
    print(f"  📊 {ticker} 공매도 데이터 수집...", end=" ")
    short_data = get_comprehensive_short_data(ticker)
    if short_data:
        short_data_list.append(short_data)
        print("✓")
    else:
        print("✗")

df_short = pd.DataFrame(short_data_list)

# 공매도 데이터를 메인 데이터프레임에 병합
df_results = df_results.merge(df_short, left_on='Ticker', right_on='ticker', how='left')

# 공매도 스코어 계산 (낮을수록 좋음)
def calculate_short_score(row):
    """
    공매도 스코어: 공매도 비율이 낮을수록 높은 점수
    - 5% 미만: 매우 좋음 (20점)
    - 5-10%: 양호 (15점)
    - 10-20%: 보통 (10점)
    - 20% 이상: 주의 (5점)
    """
    short_pct = row.get('short_percent_float', 0)

    if short_pct < 5:
        return 20
    elif short_pct < 10:
        return 15
    elif short_pct < 20:
        return 10
    else:
        return 5

df_results['Short_Score'] = df_results.apply(calculate_short_score, axis=1)

# 종합 투자 점수 (기존 Buy Signal + Short Score)
df_results['Total_Investment_Score'] = df_results['Buy_Signal_Score'] + df_results['Short_Score']
df_results = df_results.sort_values('Total_Investment_Score', ascending=False)

# ==================== 결과 출력 ====================

print("\n" + "=" * 80)
print("📊 MAGNIFICENT SEVEN - 종합 분석 결과 (공매도 포함)")
print("=" * 80)

display_cols = ['Ticker', 'Company', 'Current_Price', 'Anchored_VWAP', 'Price_vs_VWAP_%',
                'Quarter_Return_%', 'short_percent_float', 'short_ratio_days',
                'daily_short_ratio', 'Buy_Signal_Score', 'Short_Score', 'Total_Investment_Score']

print(df_results[display_cols].to_string(index=False))

print("\n" + "=" * 80)
print("🏆 MAG 7 투자 추천 순위 (공매도 분석 포함)")
print("=" * 80)

for idx, row in df_results.iterrows():
    rank = df_results.index.get_loc(idx) + 1

    if rank == 1:
        medal = "🥇"
    elif rank == 2:
        medal = "🥈"
    elif rank == 3:
        medal = "🥉"
    else:
        medal = f"  {rank}"

    print(f"\n{medal} {row['Ticker']} - {row['Company']}")
    print(f"    🎯 {row['Description']}")
    print(f"    💰 시가총액: ${row['Market_Cap_Trillion']:.2f}T")
    print(f"    📈 현재가: ${row['Current_Price']} | VWAP: ${row['Anchored_VWAP']}")
    print(f"    📊 VWAP 대비: {row['Price_vs_VWAP_%']:+.2f}% | 분기 수익률: {row['Quarter_Return_%']:+.2f}%")
    print(f"    🎲 VWAP 위 거래일: {row['Above_VWAP_Days_%']}% | 추세 강도: {row['Uptrend_Strength_%']}%")

    # 공매도 정보
    short_pct = row.get('short_percent_float', 0)
    short_days = row.get('short_ratio_days', 0)
    daily_short = row.get('daily_short_ratio', 0)

    print(f"    🔴 공매도 비율: {short_pct:.2f}% of float | 커버 소요일: {short_days:.1f}일")

    if daily_short > 0:
        print(f"    📊 최근 일평균 공매도 거래비율: {daily_short:.1f}%")

    if short_pct < 5:
        short_status = "✅ 매우 낮음 (긍정)"
    elif short_pct < 10:
        short_status = "🟢 양호"
    elif short_pct < 20:
        short_status = "🟡 보통"
    else:
        short_status = "🔴 높음 (주의)"

    print(f"    🎯 공매도 상태: {short_status}")

    print(f"    ⭐ 기술적 점수: {row['Buy_Signal_Score']}/100 | 공매도 점수: {row['Short_Score']}/20", end="")
    print(f" | 종합: {row['Total_Investment_Score']}/120")

    if row['Total_Investment_Score'] >= 90:
        print("       → 💚 최우선 매수 추천")
    elif row['Total_Investment_Score'] >= 75:
        print("       → 💛 강력 매수")
    elif row['Total_Investment_Score'] >= 60:
        print("       → 💙 눌림목 대기")
    else:
        print("       → ⚪ 관찰 필요")

# ==================== Yahoo Finance vs FINRA 비교 테이블 ====================

print("\n" + "=" * 80)
print("📊 Yahoo Finance vs FINRA 공매도 데이터 상세 비교")
print("=" * 80)

comparison_data = []

for idx, row in df_results.iterrows():
    ticker = row['Ticker']

    comparison_data.append({
        'Ticker': ticker,
        'Company': row['Company'][:25],

        # Yahoo Finance 데이터
        'YF_Short_%_Float': f"{row.get('short_percent_float', 0):.2f}%",
        'YF_Days_to_Cover': f"{row.get('short_ratio_days', 0):.2f}",
        'YF_Shares_Short_M': f"{row.get('shares_short_millions', 0):.1f}M",
        'YF_Change_MoM': f"{row.get('short_change_pct', 0):+.1f}%",

        # FINRA 데이터
        'FINRA_Daily_Short_%': f"{row.get('daily_short_ratio', 0):.1f}%" if row.get('daily_short_ratio', 0) > 0 else "N/A",
        'FINRA_10D_Avg_%': f"{row.get('avg_daily_short_ratio_10d', 0):.1f}%" if row.get('avg_daily_short_ratio_10d', 0) > 0 else "N/A",

        # 데이터 소스
        'Data_Source': row.get('data_source', 'N/A')
    })

df_comparison = pd.DataFrame(comparison_data)

print("\n📋 상세 비교표:")
print("=" * 120)
print(df_comparison.to_string(index=False))

print("\n" + "=" * 120)
print("📖 컬럼 설명:")
print("=" * 120)
print("""
[Yahoo Finance 데이터 - 공매도 잔고 (Short Interest)]
  • YF_Short_%_Float     : 유통주식(Float) 대비 공매도 비율
                           → 낮을수록 좋음 (일반적으로 5% 미만이 건강)

  • YF_Days_to_Cover     : 공매도 청산 소요일 (Short Interest / Avg Daily Volume)
                           → 3일 이상이면 Short Squeeze 가능성

  • YF_Shares_Short_M    : 현재 공매도된 총 주식 수 (백만 주)

  • YF_Change_MoM        : 전월 대비 공매도 변화율
                           → (+)는 공매도 증가 (약세 신호)
                           → (-)는 공매도 감소 (강세 신호)

[FINRA 데이터 - 일별 공매도 거래량 (Daily Short Volume)]
  • FINRA_Daily_Short_%  : 최근 거래일의 공매도 거래량 비율
                           → 해당일 전체 거래량 중 공매도 거래 비중

  • FINRA_10D_Avg_%      : 최근 10거래일 평균 공매도 거래 비율
                           → 30-40%는 정상 (마켓메이킹 포함)
                           → 50% 이상은 강한 약세 압력

[중요한 차이점]
  ✓ Yahoo Finance : 공매도 "잔고" (누적된 공매도 포지션)
  ✓ FINRA Daily   : 공매도 "거래량" (해당일의 신규 공매도 거래)

  → 두 지표는 다른 의미이므로 함께 분석해야 정확!
""")

# ==================== 데이터 품질 분석 ====================

print("\n" + "=" * 80)
print("🔍 데이터 품질 및 신뢰도 분석")
print("=" * 80)

quality_analysis = []

for idx, row in df_results.iterrows():
    ticker = row['Ticker']

    # 데이터 완성도 평가
    yf_complete = (
        row.get('short_percent_float', 0) > 0 and
        row.get('shares_short_millions', 0) > 0
    )

    finra_complete = row.get('daily_short_ratio', 0) > 0

    data_quality = "⭐⭐⭐" if (yf_complete and finra_complete) else \
                   "⭐⭐" if (yf_complete or finra_complete) else "⭐"

    # 공매도 신호 강도
    short_pct = row.get('short_percent_float', 0)
    daily_short = row.get('daily_short_ratio', 0)

    if short_pct < 3 and daily_short < 40:
        signal = "💚 매우 긍정"
    elif short_pct < 5 and daily_short < 45:
        signal = "🟢 긍정"
    elif short_pct < 10 and daily_short < 50:
        signal = "🟡 중립"
    else:
        signal = "🔴 약세 압력"

    quality_analysis.append({
        'Ticker': ticker,
        'YF_Complete': '✓' if yf_complete else '✗',
        'FINRA_Complete': '✓' if finra_complete else '✗',
        'Data_Quality': data_quality,
        'Short_Signal': signal,
        'Interpretation': f"잔고 {short_pct:.1f}% / 일거래 {daily_short:.0f}%"
    })

df_quality = pd.DataFrame(quality_analysis)

print("\n")
print(df_quality.to_string(index=False))

print("\n" + "=" * 80)
print("💡 통합 해석 가이드")
print("=" * 80)
print("""
[최적 투자 신호] 💚
  ✓ Yahoo: Short % < 3% + FINRA: Daily < 40%
  → 공매도 잔고도 낮고, 일일 공매도 거래도 적음
  → 시장의 강한 신뢰, 매수 추천

[양호한 신호] 🟢
  ✓ Yahoo: Short % < 5% + FINRA: Daily < 45%
  → 전반적으로 건강한 수준
  → 적극 매수 고려

[주의 필요] 🟡
  ✓ Yahoo: Short % 5-10% + FINRA: Daily 45-50%
  → 약간의 약세 포지션 존재
  → 기술적 분석 병행 필수

[약세 압력] 🔴
  ✓ Yahoo: Short % > 10% 또는 FINRA: Daily > 50%
  → 강한 공매도 압력
  → Short Squeeze 가능성은 있으나 위험도 높음
""")

# ==================== 공매도 관련 차트 ====================

print("\n📈 공매도 분석 차트 생성 중...")

# 차트 0: Yahoo Finance vs FINRA 비교 차트
fig_compare = make_subplots(
    rows=2, cols=1,
    subplot_titles=(
        'Yahoo Finance: Short % of Float (공매도 잔고)',
        'FINRA: Daily Short Volume Ratio (일일 공매도 거래 비율)'
    ),
    vertical_spacing=0.15,
    specs=[[{"type": "bar"}], [{"type": "bar"}]]
)

# Yahoo Finance 공매도 비율
fig_compare.add_trace(
    go.Bar(
        x=df_results['Ticker'],
        y=df_results['short_percent_float'],
        name='YF Short %',
        marker=dict(
            color=df_results['short_percent_float'],
            colorscale='RdYlGn_r',
            showscale=False
        ),
        text=df_results['short_percent_float'].round(2),
        textposition='auto',
        hovertemplate='<b>%{x}</b><br>공매도 잔고: %{y:.2f}%<extra></extra>'
    ),
    row=1, col=1
)

# FINRA 일일 공매도 비율
fig_compare.add_trace(
    go.Bar(
        x=df_results['Ticker'],
        y=df_results['daily_short_ratio'],
        name='FINRA Daily %',
        marker=dict(
            color=df_results['daily_short_ratio'],
            colorscale='Blues',
            showscale=False
        ),
        text=df_results['daily_short_ratio'].round(1),
        textposition='auto',
        hovertemplate='<b>%{x}</b><br>일일 공매도: %{y:.1f}%<extra></extra>'
    ),
    row=2, col=1
)

# 참고선 추가
fig_compare.add_hline(y=5, line_dash="dash", line_color="green", row=1, col=1,
                      annotation_text="건강 기준선 (5%)", annotation_position="right")
fig_compare.add_hline(y=40, line_dash="dash", line_color="orange", row=2, col=1,
                      annotation_text="정상 범위 (40%)", annotation_position="right")

fig_compare.update_xaxes(title_text="종목", row=1, col=1)
fig_compare.update_xaxes(title_text="종목", row=2, col=1)
fig_compare.update_yaxes(title_text="Short % of Float", row=1, col=1)
fig_compare.update_yaxes(title_text="Daily Short %", row=2, col=1)

fig_compare.update_layout(
    title_text='📊 Yahoo Finance vs FINRA 공매도 데이터 비교',
    height=800,
    showlegend=False,
    template='plotly_white'
)

fig_compare.show()

# 차트 0-2: 산점도로 두 지표 관계 분석
fig_scatter_short = px.scatter(
    df_results,
    x='short_percent_float',
    y='daily_short_ratio',
    size='Market_Cap_Trillion',
    color='Total_Investment_Score',
    hover_data=['Ticker', 'Company'],
    text='Ticker',
    color_continuous_scale='RdYlGn',
    title='📊 공매도 잔고 vs 일일 공매도 거래 비율 관계',
    labels={
        'short_percent_float': 'Yahoo Finance: Short % of Float',
        'daily_short_ratio': 'FINRA: Daily Short Volume %',
        'Total_Investment_Score': '종합 점수'
    }
)

# 참고선 추가
fig_scatter_short.add_hline(y=40, line_dash="dash", line_color="gray",
                            annotation_text="FINRA 정상 기준", annotation_position="right")
fig_scatter_short.add_vline(x=5, line_dash="dash", line_color="gray",
                            annotation_text="YF 건강 기준", annotation_position="top")

fig_scatter_short.update_traces(textposition='top center', textfont_size=12)
fig_scatter_short.update_layout(height=600)
fig_scatter_short.show()

# 차트 1: 공매도 비율 비교
fig_short = go.Figure()

colors_short = ['#00C853' if x < 5 else '#FFD600' if x < 10 else '#FF5722'
                for x in df_results['short_percent_float']]

fig_short.add_trace(go.Bar(
    y=df_results['Ticker'],
    x=df_results['short_percent_float'],
    orientation='h',
    marker=dict(color=colors_short),
    text=df_results['short_percent_float'].round(2),
    textposition='auto',
    hovertemplate='<b>%{y}</b><br>공매도 비율: %{x:.2f}%<extra></extra>'
))

fig_short.update_layout(
    title='🔴 MAG 7 - 공매도 비율 (Short % of Float)',
    xaxis_title='공매도 비율 (%)',
    yaxis_title='종목',
    height=500,
    template='plotly_white'
)

# 참고선 추가
fig_short.add_vline(x=5, line_dash="dash", line_color="green",
                    annotation_text="낮음", annotation_position="top")
fig_short.add_vline(x=10, line_dash="dash", line_color="orange",
                    annotation_text="보통", annotation_position="top")
fig_short.add_vline(x=20, line_dash="dash", line_color="red",
                    annotation_text="높음", annotation_position="top")

fig_short.show()

# 차트 2: 종합 투자 점수 (기술적 + 공매도)
fig_total = make_subplots(
    rows=1, cols=2,
    subplot_titles=('기술적 분석 점수', '종합 투자 점수'),
    specs=[[{"type": "bar"}, {"type": "bar"}]]
)

fig_total.add_trace(
    go.Bar(
        y=df_results['Ticker'],
        x=df_results['Buy_Signal_Score'],
        orientation='h',
        name='기술적 점수',
        marker_color='#2196F3',
        text=df_results['Buy_Signal_Score'],
        textposition='auto'
    ),
    row=1, col=1
)

fig_total.add_trace(
    go.Bar(
        y=df_results['Ticker'],
        x=df_results['Total_Investment_Score'],
        orientation='h',
        name='종합 점수',
        marker_color='#4CAF50',
        text=df_results['Total_Investment_Score'],
        textposition='auto'
    ),
    row=1, col=2
)

fig_total.update_xaxes(title_text="점수", row=1, col=1)
fig_total.update_xaxes(title_text="점수", row=1, col=2)
fig_total.update_yaxes(title_text="종목", row=1, col=1)

fig_total.update_layout(
    title_text='📊 MAG 7 - 투자 점수 비교 (기술적 vs 종합)',
    height=500,
    showlegend=False,
    template='plotly_white'
)

fig_total.show()

# 차트 3: 공매도 vs 분기 수익률
fig_scatter = px.scatter(
    df_results,
    x='short_percent_float',
    y='Quarter_Return_%',
    size='Market_Cap_Trillion',
    color='Total_Investment_Score',
    hover_data=['Ticker', 'Company'],
    text='Ticker',
    color_continuous_scale='RdYlGn',
    title='📊 MAG 7 - 공매도 비율 vs 분기 수익률',
    labels={
        'short_percent_float': '공매도 비율 (%)',
        'Quarter_Return_%': '분기 수익률 (%)',
        'Total_Investment_Score': '종합 점수'
    }
)

fig_scatter.update_traces(textposition='top center', textfont_size=12)
fig_scatter.update_layout(height=600)
fig_scatter.show()

# ==================== 최종 요약 ====================

print("\n" + "=" * 80)
print("📈 MAG 7 투자 전략 최종 요약 (공매도 분석 포함)")
print("=" * 80)

print(f"\n📍 분석 기준: Anchored VWAP ({quarter_start.year} Q{quarter_num})")
print(f"📅 Anchor Point: {quarter_start.strftime('%Y-%m-%d')}")
print(f"🌟 분석 대상: MAGNIFICENT SEVEN (7개 종목)")

# 공매도 분석 요약
low_short = df_results[df_results['short_percent_float'] < 5]
print(f"\n✅ 낮은 공매도 (<5%): {len(low_short)}개")
if not low_short.empty:
    print(f"   → {', '.join(low_short['Ticker'].tolist())}")

high_short = df_results[df_results['short_percent_float'] >= 10]
print(f"⚠️ 높은 공매도 (≥10%): {len(high_short)}개")
if not high_short.empty:
    print(f"   → {', '.join(high_short['Ticker'].tolist())}")

# 종합 추천
top_score = df_results[df_results['Total_Investment_Score'] >= 90]
print(f"\n💚 최우선 매수 (90점↑): {len(top_score)}개")
if not top_score.empty:
    print(f"   → {', '.join(top_score['Ticker'].tolist())}")

strong_buy = df_results[(df_results['Total_Investment_Score'] >= 75) &
                        (df_results['Total_Investment_Score'] < 90)]
print(f"💛 강력 매수 (75-90점): {len(strong_buy)}개")
if not strong_buy.empty:
    print(f"   → {', '.join(strong_buy['Ticker'].tolist())}")

print("\n🏆 TOP 3 추천 (공매도 고려):")
for idx in range(min(3, len(df_results))):
    row = df_results.iloc[idx]
    print(f"  {idx+1}. {row['Ticker']} - {row['Company']}")
    print(f"     💰 시총: ${row['Market_Cap_Trillion']:.2f}T | " +
          f"종합점수: {row['Total_Investment_Score']:.0f}/120")
    print(f"     📊 VWAP: {row['Price_vs_VWAP_%']:+.2f}% | " +
          f"분기수익: {row['Quarter_Return_%']:+.2f}% | " +
          f"공매도: {row['short_percent_float']:.2f}%")

print("\n💡 투자 가이드 (공매도 포함):")
print("""
1. 💚 최우선 매수 (90점 이상):
   - VWAP 위 + 낮은 공매도 비율
   - 즉시 매수 검토, 강한 상승 모멘텀 예상

2. 💛 강력 매수 (75-90점):
   - 기술적으로 양호하나 공매도 약간 존재
   - VWAP 근처 조정 시 매수 기회

3. 💙 눌림목 대기 (60-75점):
   - 공매도 비율 확인 필수
   - VWAP 돌파 확인 후 매수

4. ⚠️ 공매도 주의사항:
   - 10% 이상: Short Squeeze 가능성 주의
   - 5% 미만: 시장의 신뢰 높음
   - Days to Cover 3일 이상: 변동성 증가 가능
""")

print("\n📊 공매도 데이터 출처:")
for _, row in df_results.iterrows():
    if pd.notna(row.get('data_source')):
        print(f"  • {row['Ticker']}: {row['data_source']}")

print("\n" + "=" * 80)
print("✨ MAGNIFICENT SEVEN 종합 분석 완료!")
print("=" * 80)

🌟 MAGNIFICENT SEVEN (MAG 7) 실시간 종합 분석 + 공매도 데이터

빅테크 7대 기업 리스트:
  • AAPL   - Apple Inc.                     | 아이폰, 생태계, 온디바이스 AI
  • MSFT   - Microsoft Corporation          | 클라우드(Azure), 생성형 AI (OpenAI 대주주)
  • GOOGL  - Alphabet Inc.                  | 구글 검색, 유튜브, AI (Gemini)
  • AMZN   - Amazon.com Inc.                | 전자상거래, 클라우드(AWS) 1위
  • NVDA   - NVIDIA Corporation             | AI 반도체(GPU) 독점적 지배자
  • META   - Meta Platforms Inc.            | 페이스북, 인스타그램, AI(Llama)
  • TSLA   - Tesla Inc.                     | 전기차, 자율주행, 로봇

🎯 Anchored VWAP 분석: 2025 Q4
📍 Anchor Point: 2025-10-01
📅 현재까지 거래일: 58일

🔍 MAG 7 종목 분석 중...

분석 중: AAPL (Apple Inc.)... ✓
분석 중: MSFT (Microsoft Corporation)... ✓
분석 중: GOOGL (Alphabet Inc.)... ✓
분석 중: AMZN (Amazon.com Inc.)... ✓
분석 중: NVDA (NVIDIA Corporation)... ✓
분석 중: META (Meta Platforms Inc.)... ✓
분석 중: TSLA (Tesla Inc.)... ✓

📊 공매도(Short Interest) 데이터 수집 중...
  📊 AAPL 공매도 데이터 수집... ✓
  📊 MSFT 공매도 데이터 수집... ✓
  📊 GOOGL 공매도 데이터 수집... ✓
  📊 AMZN 공매도 데이터 


📈 MAG 7 투자 전략 최종 요약 (공매도 분석 포함)

📍 분석 기준: Anchored VWAP (2025 Q4)
📅 Anchor Point: 2025-10-01
🌟 분석 대상: MAGNIFICENT SEVEN (7개 종목)

✅ 낮은 공매도 (<5%): 7개
   → GOOGL, AAPL, TSLA, NVDA, AMZN, MSFT, META
⚠️ 높은 공매도 (≥10%): 0개

💚 최우선 매수 (90점↑): 2개
   → GOOGL, AAPL
💛 강력 매수 (75-90점): 0개

🏆 TOP 3 추천 (공매도 고려):
  1. GOOGL - Alphabet Inc.
     💰 시총: $3.88T | 종합점수: 100/120
     📊 VWAP: +15.13% | 분기수익: +30.65% | 공매도: 1.07%
  2. AAPL - Apple Inc.
     💰 시총: $4.12T | 종합점수: 95/120
     📊 VWAP: +4.94% | 분기수익: +8.76% | 공매도: 0.78%
  3. TSLA - Tesla Inc.
     💰 시총: $1.42T | 종합점수: 50/120
     📊 VWAP: -1.47% | 분기수익: -7.16% | 공매도: 2.45%

💡 투자 가이드 (공매도 포함):

1. 💚 최우선 매수 (90점 이상):
   - VWAP 위 + 낮은 공매도 비율
   - 즉시 매수 검토, 강한 상승 모멘텀 예상

2. 💛 강력 매수 (75-90점):
   - 기술적으로 양호하나 공매도 약간 존재
   - VWAP 근처 조정 시 매수 기회

3. 💙 눌림목 대기 (60-75점):
   - 공매도 비율 확인 필수
   - VWAP 돌파 확인 후 매수

4. ⚠️ 공매도 주의사항:
   - 10% 이상: Short Squeeze 가능성 주의
   - 5% 미만: 시장의 신뢰 높음
   - Days to Cover 3일 이상: 변동성 증가 가능


📊 공매도 데이터 출처:
  • GOOGL: Yahoo Finance + FI

In [5]:
# 필요한 라이브러리 설치 및 임포트
!pip install yfinance pandas numpy plotly requests -q

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
import requests
from io import StringIO
import time

warnings.filterwarnings('ignore')
pio.renderers.default = 'colab'

# ==================== FINRA 공매도 데이터 수집 함수 ====================

def get_finra_short_interest(ticker, retries=2):
    """
    FINRA에서 특정 종목의 공매도 잔고(Short Interest) 데이터 수집
    실제 CSV 파일 다운로드 방식 사용
    """
    try:
        # 1. 일별 공매도 거래량 데이터 수집
        daily_data = get_finra_short_volume_csv(ticker, days_back=10)

        # 2. 월별 공매도 잔고 데이터 수집
        monthly_data = get_finra_monthly_short_interest(ticker)

        # 데이터 통합
        result = {}

        if daily_data:
            result.update({
                'daily_short_ratio': daily_data['latest_short_ratio'],
                'avg_daily_short_ratio': daily_data['avg_short_ratio_10d'],
                'daily_data_date': daily_data['latest_date'],
                'daily_data_points': daily_data['data_points']
            })

        if monthly_data:
            result.update({
                'monthly_short_interest': monthly_data.get('short_interest', 0),
                'days_to_cover': monthly_data.get('days_to_cover', 0),
                'settlement_date': monthly_data.get('settlement_date', 'N/A')
            })

        return result if result else None

    except Exception as e:
        return None


def get_finra_short_volume_csv(ticker, days_back=10):
    """
    FINRA에서 일별 공매도 거래량 CSV 파일 다운로드 및 분석
    URL: https://cdn.finra.org/equity/regsho/daily/CNMSshvol[YYYYMMDD].txt
    """
    try:
        today = datetime.now()
        short_volume_data = []

        for days in range(days_back):
            check_date = today - timedelta(days=days)

            # 주말 건너뛰기
            if check_date.weekday() >= 5:  # 토요일(5), 일요일(6)
                continue

            date_str = check_date.strftime('%Y%m%d')

            # FINRA Daily Short Volume URL
            url = f"https://cdn.finra.org/equity/regsho/daily/CNMSshvol{date_str}.txt"

            try:
                response = requests.get(url, timeout=10)

                if response.status_code == 200:
                    # 파이프(|) 구분자로 CSV 읽기
                    df = pd.read_csv(StringIO(response.text), sep='|')

                    # 컬럼명 확인 및 정리
                    if 'Symbol' in df.columns or 'symbol' in df.columns:
                        # 대소문자 무관하게 처리
                        df.columns = df.columns.str.strip()
                        symbol_col = 'Symbol' if 'Symbol' in df.columns else 'symbol'

                        # 해당 종목 데이터 찾기
                        ticker_data = df[df[symbol_col].str.upper() == ticker.upper()]

                        if not ticker_data.empty:
                            row = ticker_data.iloc[0]

                            short_vol = row.get('ShortVolume', row.get('shortVolume', 0))
                            total_vol = row.get('TotalVolume', row.get('totalVolume', 0))

                            if pd.notna(short_vol) and pd.notna(total_vol) and total_vol > 0:
                                short_volume_data.append({
                                    'date': check_date.strftime('%Y-%m-%d'),
                                    'short_volume': int(short_vol),
                                    'total_volume': int(total_vol),
                                    'short_ratio': round(short_vol / total_vol * 100, 2)
                                })
            except:
                continue

        if short_volume_data:
            # 데이터프레임으로 변환
            df_short = pd.DataFrame(short_volume_data)

            # 최근 데이터 통계
            avg_short_ratio = df_short['short_ratio'].mean()
            latest_short_ratio = df_short.iloc[0]['short_ratio'] if len(df_short) > 0 else 0

            return {
                'ticker': ticker,
                'latest_date': df_short.iloc[0]['date'] if len(df_short) > 0 else 'N/A',
                'latest_short_ratio': latest_short_ratio,
                'avg_short_ratio_10d': round(avg_short_ratio, 2),
                'data_points': len(df_short),
                'historical_data': df_short
            }

        return None

    except Exception as e:
        return None


def get_finra_monthly_short_interest(ticker):
    """
    FINRA 월별 공매도 잔고 데이터 수집
    URL: https://cdn.finra.org/equity/regsho/monthly/[FILENAME].txt
    """
    try:
        # 최근 6개월 시도
        today = datetime.now()

        for months_back in range(6):
            check_date = today - timedelta(days=30 * months_back)

            # 월별 파일명 형식: CNMSshvol{YYYYMM}a.txt, CNMSshvol{YYYYMM}b.txt
            year_month = check_date.strftime('%Y%m')

            for suffix in ['a', 'b']:
                url = f"https://cdn.finra.org/equity/regsho/monthly/CNMSshvol{year_month}{suffix}.txt"

                try:
                    response = requests.get(url, timeout=10)

                    if response.status_code == 200:
                        df = pd.read_csv(StringIO(response.text), sep='|')

                        if 'Symbol' in df.columns or 'symbol' in df.columns:
                            df.columns = df.columns.str.strip()
                            symbol_col = 'Symbol' if 'Symbol' in df.columns else 'symbol'

                            ticker_data = df[df[symbol_col].str.upper() == ticker.upper()]

                            if not ticker_data.empty:
                                row = ticker_data.iloc[0]

                                return {
                                    'ticker': ticker,
                                    'settlement_date': row.get('SettlementDate', 'N/A'),
                                    'short_interest': row.get('ShortInterest', 0),
                                    'avg_daily_volume': row.get('AverageDailyVolume', 0),
                                    'days_to_cover': row.get('DaysToCover', 0)
                                }
                except:
                    continue

        return None

    except Exception as e:
        return None


def get_short_interest_from_yfinance(ticker):
    """
    Yahoo Finance에서 공매도 관련 데이터 수집 (대체 방법)
    """
    try:
        stock = yf.Ticker(ticker)
        info = stock.info

        short_data = {
            'ticker': ticker,
            'short_ratio': info.get('shortRatio', 0),  # Days to Cover
            'short_percent_float': info.get('shortPercentOfFloat', 0) * 100 if info.get('shortPercentOfFloat') else 0,
            'shares_short': info.get('sharesShort', 0),
            'shares_short_prior_month': info.get('sharesShortPriorMonth', 0),
            'short_percent_shares_out': info.get('sharesPercentSharesOut', 0) * 100 if info.get('sharesPercentSharesOut') else 0
        }

        # 변화율 계산
        if short_data['shares_short_prior_month'] > 0:
            short_data['short_change_pct'] = ((short_data['shares_short'] - short_data['shares_short_prior_month']) /
                                               short_data['shares_short_prior_month'] * 100)
        else:
            short_data['short_change_pct'] = 0

        return short_data

    except Exception as e:
        print(f"  ⚠️ Yahoo Finance 오류 ({ticker}): {str(e)}")
        return None


def get_comprehensive_short_data(ticker):
    """
    여러 소스에서 공매도 데이터 종합 수집
    - Yahoo Finance: 공매도 잔고 (Short Interest)
    - FINRA CSV: 일별 공매도 거래량 비율
    """
    # Yahoo Finance 데이터 (공매도 잔고)
    yf_data = get_short_interest_from_yfinance(ticker)

    # FINRA CSV 데이터 (일별 거래량 비율)
    finra_data = get_finra_short_interest(ticker)

    # 데이터 통합
    combined_data = {
        'ticker': ticker,
        'short_ratio_days': 0,
        'short_percent_float': 0,
        'shares_short_millions': 0,
        'short_change_pct': 0,
        'daily_short_ratio': 0,
        'avg_daily_short_ratio_10d': 0,
        'data_source': []
    }

    if yf_data:
        combined_data.update({
            'short_ratio_days': round(yf_data.get('short_ratio', 0), 2),
            'short_percent_float': round(yf_data.get('short_percent_float', 0), 2),
            'shares_short_millions': round(yf_data.get('shares_short', 0) / 1e6, 2),
            'short_change_pct': round(yf_data.get('short_change_pct', 0), 2),
        })
        combined_data['data_source'].append('Yahoo Finance')

    if finra_data:
        # FINRA 일별 데이터 추가
        if 'daily_short_ratio' in finra_data:
            combined_data['daily_short_ratio'] = finra_data['daily_short_ratio']
            combined_data['avg_daily_short_ratio_10d'] = finra_data['avg_daily_short_ratio']
            combined_data['data_source'].append(f"FINRA Daily ({finra_data.get('daily_data_points', 0)}일)")

        # FINRA 월별 데이터로 Days to Cover 업데이트
        if 'days_to_cover' in finra_data and finra_data['days_to_cover'] > 0:
            combined_data['short_ratio_days'] = round(finra_data['days_to_cover'], 2)
            combined_data['data_source'].append('FINRA Monthly')

    # 데이터 소스를 문자열로 변환
    combined_data['data_source'] = ' + '.join(combined_data['data_source']) if combined_data['data_source'] else 'N/A'

    return combined_data


# ==================== Magnificent Seven 정의 ====================

MAG7_STOCKS = {
    'AAPL': {
        'name': 'Apple Inc.',
        'description': '아이폰, 생태계, 온디바이스 AI',
        'sector': 'Technology',
        'industry': 'Consumer Electronics'
    },
    'MSFT': {
        'name': 'Microsoft Corporation',
        'description': '클라우드(Azure), 생성형 AI (OpenAI 대주주)',
        'sector': 'Technology',
        'industry': 'Software'
    },
    'GOOGL': {
        'name': 'Alphabet Inc.',
        'description': '구글 검색, 유튜브, AI (Gemini)',
        'sector': 'Communication Services',
        'industry': 'Internet Content & Information'
    },
    'AMZN': {
        'name': 'Amazon.com Inc.',
        'description': '전자상거래, 클라우드(AWS) 1위',
        'sector': 'Consumer Cyclical',
        'industry': 'Internet Retail'
    },
    'NVDA': {
        'name': 'NVIDIA Corporation',
        'description': 'AI 반도체(GPU) 독점적 지배자',
        'sector': 'Technology',
        'industry': 'Semiconductors'
    },
    'META': {
        'name': 'Meta Platforms Inc.',
        'description': '페이스북, 인스타그램, AI(Llama)',
        'sector': 'Communication Services',
        'industry': 'Internet Content & Information'
    },
    'TSLA': {
        'name': 'Tesla Inc.',
        'description': '전기차, 자율주행, 로봇',
        'sector': 'Consumer Cyclical',
        'industry': 'Auto Manufacturers'
    }
}

print("=" * 80)
print("🌟 MAGNIFICENT SEVEN (MAG 7) 실시간 종합 분석 + 공매도 데이터")
print("=" * 80)
print("\n빅테크 7대 기업 리스트:")
for ticker, info in MAG7_STOCKS.items():
    print(f"  • {ticker:6} - {info['name']:30} | {info['description']}")
print("=" * 80)


def get_current_quarter_start():
    """현재 분기 시작일 계산"""
    now = datetime.now()
    quarter = (now.month - 1) // 3
    quarter_start_month = quarter * 3 + 1
    quarter_start = datetime(now.year, quarter_start_month, 1)
    return quarter_start


def calculate_anchored_vwap(df):
    """Anchored VWAP 계산"""
    df = df.copy()
    df['Typical_Price'] = (df['High'] + df['Low'] + df['Close']) / 3
    df['TP_Volume'] = df['Typical_Price'] * df['Volume']
    df['Cumulative_TP_Volume'] = df['TP_Volume'].cumsum()
    df['Cumulative_Volume'] = df['Volume'].cumsum()
    df['Anchored_VWAP'] = df['Cumulative_TP_Volume'] / df['Cumulative_Volume']
    return df


def get_quarterly_vwap_analysis(ticker):
    """분기별 Anchored VWAP 분석"""
    try:
        quarter_start = get_current_quarter_start()
        end_date = datetime.now()
        quarter_num = (quarter_start.month - 1) // 3 + 1

        stock = yf.Ticker(ticker)
        df = stock.history(start=quarter_start, end=end_date)

        if df.empty or len(df) < 5:
            return None

        df = calculate_anchored_vwap(df)

        current_price = df['Close'].iloc[-1]
        current_vwap = df['Anchored_VWAP'].iloc[-1]
        above_vwap_ratio = (df['Close'] > df['Anchored_VWAP']).sum() / len(df) * 100
        recent_5days_avg = df['Close'].tail(5).mean()
        recent_10days_avg = df['Close'].tail(10).mean()

        recent_20 = df['Close'].tail(min(20, len(df)))
        uptrend_strength = (recent_20.diff() > 0).sum() / len(recent_20) * 100 if len(recent_20) > 1 else 50

        recent_volume = df['Volume'].tail(5).mean()
        avg_volume = df['Volume'].mean()
        volume_ratio = recent_volume / avg_volume if avg_volume > 0 else 1

        info = stock.info
        quarter_start_price = df['Close'].iloc[0]
        quarter_return = ((current_price - quarter_start_price) / quarter_start_price * 100)

        return {
            'Ticker': ticker,
            'Company': MAG7_STOCKS[ticker]['name'],
            'Description': MAG7_STOCKS[ticker]['description'],
            'Sector': MAG7_STOCKS[ticker]['sector'],
            'Quarter': f'{quarter_start.year} Q{quarter_num}',
            'Quarter_Start_Date': quarter_start.strftime('%Y-%m-%d'),
            'Trading_Days': len(df),
            'Current_Price': round(current_price, 2),
            'Anchored_VWAP': round(current_vwap, 2),
            'Quarter_Start_Price': round(quarter_start_price, 2),
            'Quarter_Return_%': round(quarter_return, 2),
            'Price_vs_VWAP_%': round((current_price - current_vwap) / current_vwap * 100, 2),
            'Above_VWAP_Days_%': round(above_vwap_ratio, 1),
            'Recent_5D_Avg': round(recent_5days_avg, 2),
            'Recent_10D_Avg': round(recent_10days_avg, 2),
            'Uptrend_Strength_%': round(uptrend_strength, 1),
            'Volume_Ratio': round(volume_ratio, 2),
            'Is_Above_VWAP': current_price > current_vwap,
            'Market_Cap': info.get('marketCap', 0),
            'Buy_Signal_Score': 0
        }

    except Exception as e:
        print(f"Error processing {ticker}: {str(e)}")
        return None


# ==================== 메인 분석 시작 ====================

quarter_start = get_current_quarter_start()
quarter_num = (quarter_start.month - 1) // 3 + 1

print("\n" + "=" * 80)
print(f"🎯 Anchored VWAP 분석: {quarter_start.year} Q{quarter_num}")
print(f"📍 Anchor Point: {quarter_start.strftime('%Y-%m-%d')}")
print(f"📅 현재까지 거래일: {(datetime.now() - quarter_start).days}일")
print("=" * 80)

# MAG 7 분석
results = []
mag7_tickers = list(MAG7_STOCKS.keys())

print("\n🔍 MAG 7 종목 분석 중...\n")

for ticker in mag7_tickers:
    print(f"분석 중: {ticker} ({MAG7_STOCKS[ticker]['name']})...", end=" ")
    result = get_quarterly_vwap_analysis(ticker)
    if result:
        results.append(result)
        print("✓")
    else:
        print("✗")

df_results = pd.DataFrame(results)

# 매수 신호 점수 계산
def calculate_buy_score(row):
    score = 0
    if row['Is_Above_VWAP']:
        score += 30

    price_diff = row['Price_vs_VWAP_%']
    if 0 < price_diff <= 5:
        score += 20
    elif 5 < price_diff <= 10:
        score += 10
    elif price_diff > 10:
        score += 5

    if row['Above_VWAP_Days_%'] >= 80:
        score += 20
    elif row['Above_VWAP_Days_%'] >= 60:
        score += 15
    elif row['Above_VWAP_Days_%'] >= 40:
        score += 10

    if row['Uptrend_Strength_%'] >= 60:
        score += 15
    elif row['Uptrend_Strength_%'] >= 50:
        score += 10

    if row['Volume_Ratio'] >= 1.2:
        score += 15
    elif row['Volume_Ratio'] >= 1.0:
        score += 10

    return min(score, 100)

df_results['Buy_Signal_Score'] = df_results.apply(calculate_buy_score, axis=1)

# 시가총액 추가
df_results['Market_Cap_Trillion'] = (df_results['Market_Cap'] / 1e12).round(3)

# ==================== 공매도 데이터 수집 ====================

print("\n" + "=" * 80)
print("📊 공매도(Short Interest) 데이터 수집 중...")
print("=" * 80)

short_data_list = []

for ticker in mag7_tickers:
    print(f"  📊 {ticker} 공매도 데이터 수집...", end=" ")
    short_data = get_comprehensive_short_data(ticker)
    if short_data:
        short_data_list.append(short_data)
        print("✓")
    else:
        print("✗")

df_short = pd.DataFrame(short_data_list)

# 공매도 데이터를 메인 데이터프레임에 병합
df_results = df_results.merge(df_short, left_on='Ticker', right_on='ticker', how='left')

# 공매도 스코어 계산 (낮을수록 좋음)
def calculate_short_score(row):
    """
    공매도 스코어: 공매도 비율이 낮을수록 높은 점수
    - 5% 미만: 매우 좋음 (20점)
    - 5-10%: 양호 (15점)
    - 10-20%: 보통 (10점)
    - 20% 이상: 주의 (5점)
    """
    short_pct = row.get('short_percent_float', 0)

    if short_pct < 5:
        return 20
    elif short_pct < 10:
        return 15
    elif short_pct < 20:
        return 10
    else:
        return 5

df_results['Short_Score'] = df_results.apply(calculate_short_score, axis=1)

# 종합 투자 점수 (기존 Buy Signal + Short Score)
df_results['Total_Investment_Score'] = df_results['Buy_Signal_Score'] + df_results['Short_Score']
df_results = df_results.sort_values('Total_Investment_Score', ascending=False)

# ==================== 결과 출력 ====================

print("\n" + "=" * 80)
print("📊 MAGNIFICENT SEVEN - 종합 분석 결과 (공매도 포함)")
print("=" * 80)

display_cols = ['Ticker', 'Company', 'Current_Price', 'Anchored_VWAP', 'Price_vs_VWAP_%',
                'Quarter_Return_%', 'short_percent_float', 'short_ratio_days',
                'daily_short_ratio', 'Buy_Signal_Score', 'Short_Score', 'Total_Investment_Score']

print(df_results[display_cols].to_string(index=False))

print("\n" + "=" * 80)
print("🏆 MAG 7 투자 추천 순위 (공매도 분석 포함)")
print("=" * 80)

for idx, row in df_results.iterrows():
    rank = df_results.index.get_loc(idx) + 1

    if rank == 1:
        medal = "🥇"
    elif rank == 2:
        medal = "🥈"
    elif rank == 3:
        medal = "🥉"
    else:
        medal = f"  {rank}"

    print(f"\n{medal} {row['Ticker']} - {row['Company']}")
    print(f"    🎯 {row['Description']}")
    print(f"    💰 시가총액: ${row['Market_Cap_Trillion']:.2f}T")
    print(f"    📈 현재가: ${row['Current_Price']} | VWAP: ${row['Anchored_VWAP']}")
    print(f"    📊 VWAP 대비: {row['Price_vs_VWAP_%']:+.2f}% | 분기 수익률: {row['Quarter_Return_%']:+.2f}%")
    print(f"    🎲 VWAP 위 거래일: {row['Above_VWAP_Days_%']}% | 추세 강도: {row['Uptrend_Strength_%']}%")

    # 공매도 정보
    short_pct = row.get('short_percent_float', 0)
    short_days = row.get('short_ratio_days', 0)
    daily_short = row.get('daily_short_ratio', 0)

    print(f"    🔴 공매도 비율: {short_pct:.2f}% of float | 커버 소요일: {short_days:.1f}일")

    if daily_short > 0:
        print(f"    📊 최근 일평균 공매도 거래비율: {daily_short:.1f}%")

    if short_pct < 5:
        short_status = "✅ 매우 낮음 (긍정)"
    elif short_pct < 10:
        short_status = "🟢 양호"
    elif short_pct < 20:
        short_status = "🟡 보통"
    else:
        short_status = "🔴 높음 (주의)"

    print(f"    🎯 공매도 상태: {short_status}")

    print(f"    ⭐ 기술적 점수: {row['Buy_Signal_Score']}/100 | 공매도 점수: {row['Short_Score']}/20", end="")
    print(f" | 종합: {row['Total_Investment_Score']}/120")

    if row['Total_Investment_Score'] >= 90:
        print("       → 💚 최우선 매수 추천")
    elif row['Total_Investment_Score'] >= 75:
        print("       → 💛 강력 매수")
    elif row['Total_Investment_Score'] >= 60:
        print("       → 💙 눌림목 대기")
    else:
        print("       → ⚪ 관찰 필요")

# ==================== Yahoo Finance vs FINRA 비교 테이블 ====================

print("\n" + "=" * 80)
print("📊 Yahoo Finance vs FINRA 공매도 데이터 상세 비교")
print("=" * 80)

comparison_data = []

for idx, row in df_results.iterrows():
    ticker = row['Ticker']

    comparison_data.append({
        'Ticker': ticker,
        'Company': row['Company'][:25],

        # Yahoo Finance 데이터
        'YF_Short_%_Float': f"{row.get('short_percent_float', 0):.2f}%",
        'YF_Days_to_Cover': f"{row.get('short_ratio_days', 0):.2f}",
        'YF_Shares_Short_M': f"{row.get('shares_short_millions', 0):.1f}M",
        'YF_Change_MoM': f"{row.get('short_change_pct', 0):+.1f}%",

        # FINRA 데이터
        'FINRA_Daily_Short_%': f"{row.get('daily_short_ratio', 0):.1f}%" if row.get('daily_short_ratio', 0) > 0 else "N/A",
        'FINRA_10D_Avg_%': f"{row.get('avg_daily_short_ratio_10d', 0):.1f}%" if row.get('avg_daily_short_ratio_10d', 0) > 0 else "N/A",

        # 데이터 소스
        'Data_Source': row.get('data_source', 'N/A')
    })

df_comparison = pd.DataFrame(comparison_data)

print("\n📋 상세 비교표:")
print("=" * 150)

# 컬럼 순서 및 너비 조정을 위한 포맷팅
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 150)
pd.set_option('display.max_colwidth', 30)

print(df_comparison.to_string(index=False))
print("=" * 150)

print("\n" + "=" * 120)
print("📖 컬럼 설명:")
print("=" * 120)
print("""
[Yahoo Finance 데이터 - 공매도 잔고 (Short Interest)]
  • YF_Short_%_Float     : 유통주식(Float) 대비 공매도 비율
                           → 낮을수록 좋음 (일반적으로 5% 미만이 건강)

  • YF_Days_to_Cover     : 공매도 청산 소요일 (Short Interest / Avg Daily Volume)
                           → 3일 이상이면 Short Squeeze 가능성

  • YF_Shares_Short_M    : 현재 공매도된 총 주식 수 (백만 주)

  • YF_Change_MoM        : 전월 대비 공매도 변화율
                           → (+)는 공매도 증가 (약세 신호)
                           → (-)는 공매도 감소 (강세 신호)

[FINRA 데이터 - 일별 공매도 거래량 (Daily Short Volume)]
  • FINRA_Daily_Short_%  : 최근 거래일의 공매도 거래량 비율
                           → 해당일 전체 거래량 중 공매도 거래 비중

  • FINRA_10D_Avg_%      : 최근 10거래일 평균 공매도 거래 비율
                           → 30-40%는 정상 (마켓메이킹 포함)
                           → 50% 이상은 강한 약세 압력

[중요한 차이점]
  ✓ Yahoo Finance : 공매도 "잔고" (누적된 공매도 포지션)
  ✓ FINRA Daily   : 공매도 "거래량" (해당일의 신규 공매도 거래)

  → 두 지표는 다른 의미이므로 함께 분석해야 정확!
""")

# ==================== 데이터 품질 분석 ====================

print("\n" + "=" * 80)
print("🔍 데이터 품질 및 신뢰도 분석")
print("=" * 80)

quality_analysis = []

for idx, row in df_results.iterrows():
    ticker = row['Ticker']

    # 데이터 완성도 평가
    yf_complete = (
        row.get('short_percent_float', 0) > 0 and
        row.get('shares_short_millions', 0) > 0
    )

    finra_complete = row.get('daily_short_ratio', 0) > 0

    data_quality = "⭐⭐⭐" if (yf_complete and finra_complete) else \
                   "⭐⭐" if (yf_complete or finra_complete) else "⭐"

    # 공매도 신호 강도
    short_pct = row.get('short_percent_float', 0)
    daily_short = row.get('daily_short_ratio', 0)

    if short_pct < 3 and daily_short < 40:
        signal = "💚 매우 긍정"
    elif short_pct < 5 and daily_short < 45:
        signal = "🟢 긍정"
    elif short_pct < 10 and daily_short < 50:
        signal = "🟡 중립"
    else:
        signal = "🔴 약세 압력"

    quality_analysis.append({
        'Ticker': ticker,
        'YF_Complete': '✓' if yf_complete else '✗',
        'FINRA_Complete': '✓' if finra_complete else '✗',
        'Data_Quality': data_quality,
        'Short_Signal': signal,
        'Interpretation': f"잔고 {short_pct:.1f}% / 일거래 {daily_short:.0f}%"
    })

df_quality = pd.DataFrame(quality_analysis)

print("\n📊 종목별 데이터 품질 및 공매도 신호:")
print("=" * 100)
print(df_quality.to_string(index=False))
print("=" * 100)

print("\n" + "=" * 80)
print("💡 통합 해석 가이드")
print("=" * 80)
print("""
[최적 투자 신호] 💚
  ✓ Yahoo: Short % < 3% + FINRA: Daily < 40%
  → 공매도 잔고도 낮고, 일일 공매도 거래도 적음
  → 시장의 강한 신뢰, 매수 추천

[양호한 신호] 🟢
  ✓ Yahoo: Short % < 5% + FINRA: Daily < 45%
  → 전반적으로 건강한 수준
  → 적극 매수 고려

[주의 필요] 🟡
  ✓ Yahoo: Short % 5-10% + FINRA: Daily 45-50%
  → 약간의 약세 포지션 존재
  → 기술적 분석 병행 필수

[약세 압력] 🔴
  ✓ Yahoo: Short % > 10% 또는 FINRA: Daily > 50%
  → 강한 공매도 압력
  → Short Squeeze 가능성은 있으나 위험도 높음
""")

# ==================== 공매도 관련 차트 ====================

print("\n📈 공매도 분석 차트 생성 중...")

# 차트 0: Yahoo Finance vs FINRA 비교 차트
fig_compare = make_subplots(
    rows=2, cols=1,
    subplot_titles=(
        'Yahoo Finance: Short % of Float (공매도 잔고)',
        'FINRA: Daily Short Volume Ratio (일일 공매도 거래 비율)'
    ),
    vertical_spacing=0.15,
    specs=[[{"type": "bar"}], [{"type": "bar"}]]
)

# Yahoo Finance 공매도 비율
fig_compare.add_trace(
    go.Bar(
        x=df_results['Ticker'],
        y=df_results['short_percent_float'],
        name='YF Short %',
        marker=dict(
            color=df_results['short_percent_float'],
            colorscale='RdYlGn_r',
            showscale=False
        ),
        text=df_results['short_percent_float'].round(2),
        textposition='auto',
        hovertemplate='<b>%{x}</b><br>공매도 잔고: %{y:.2f}%<extra></extra>'
    ),
    row=1, col=1
)

# FINRA 일일 공매도 비율
fig_compare.add_trace(
    go.Bar(
        x=df_results['Ticker'],
        y=df_results['daily_short_ratio'],
        name='FINRA Daily %',
        marker=dict(
            color=df_results['daily_short_ratio'],
            colorscale='Blues',
            showscale=False
        ),
        text=df_results['daily_short_ratio'].round(1),
        textposition='auto',
        hovertemplate='<b>%{x}</b><br>일일 공매도: %{y:.1f}%<extra></extra>'
    ),
    row=2, col=1
)

# 참고선 추가
fig_compare.add_hline(y=5, line_dash="dash", line_color="green", row=1, col=1,
                      annotation_text="건강 기준선 (5%)", annotation_position="right")
fig_compare.add_hline(y=40, line_dash="dash", line_color="orange", row=2, col=1,
                      annotation_text="정상 범위 (40%)", annotation_position="right")

fig_compare.update_xaxes(title_text="종목", row=1, col=1)
fig_compare.update_xaxes(title_text="종목", row=2, col=1)
fig_compare.update_yaxes(title_text="Short % of Float", row=1, col=1)
fig_compare.update_yaxes(title_text="Daily Short %", row=2, col=1)

fig_compare.update_layout(
    title_text='📊 Yahoo Finance vs FINRA 공매도 데이터 비교',
    height=800,
    showlegend=False,
    template='plotly_white'
)

fig_compare.show()

# 차트 0-2: 산점도로 두 지표 관계 분석
fig_scatter_short = px.scatter(
    df_results,
    x='short_percent_float',
    y='daily_short_ratio',
    size='Market_Cap_Trillion',
    color='Total_Investment_Score',
    hover_data=['Ticker', 'Company'],
    text='Ticker',
    color_continuous_scale='RdYlGn',
    title='📊 공매도 잔고 vs 일일 공매도 거래 비율 관계',
    labels={
        'short_percent_float': 'Yahoo Finance: Short % of Float',
        'daily_short_ratio': 'FINRA: Daily Short Volume %',
        'Total_Investment_Score': '종합 점수'
    }
)

# 참고선 추가
fig_scatter_short.add_hline(y=40, line_dash="dash", line_color="gray",
                            annotation_text="FINRA 정상 기준", annotation_position="right")
fig_scatter_short.add_vline(x=5, line_dash="dash", line_color="gray",
                            annotation_text="YF 건강 기준", annotation_position="top")

fig_scatter_short.update_traces(textposition='top center', textfont_size=12)
fig_scatter_short.update_layout(height=600)
fig_scatter_short.show()

# 차트 1: 공매도 비율 비교
fig_short = go.Figure()

colors_short = ['#00C853' if x < 5 else '#FFD600' if x < 10 else '#FF5722'
                for x in df_results['short_percent_float']]

fig_short.add_trace(go.Bar(
    y=df_results['Ticker'],
    x=df_results['short_percent_float'],
    orientation='h',
    marker=dict(color=colors_short),
    text=df_results['short_percent_float'].round(2),
    textposition='auto',
    hovertemplate='<b>%{y}</b><br>공매도 비율: %{x:.2f}%<extra></extra>'
))

fig_short.update_layout(
    title='🔴 MAG 7 - 공매도 비율 (Short % of Float)',
    xaxis_title='공매도 비율 (%)',
    yaxis_title='종목',
    height=500,
    template='plotly_white'
)

# 참고선 추가
fig_short.add_vline(x=5, line_dash="dash", line_color="green",
                    annotation_text="낮음", annotation_position="top")
fig_short.add_vline(x=10, line_dash="dash", line_color="orange",
                    annotation_text="보통", annotation_position="top")
fig_short.add_vline(x=20, line_dash="dash", line_color="red",
                    annotation_text="높음", annotation_position="top")

fig_short.show()

# 차트 2: 종합 투자 점수 (기술적 + 공매도)
fig_total = make_subplots(
    rows=1, cols=2,
    subplot_titles=('기술적 분석 점수', '종합 투자 점수'),
    specs=[[{"type": "bar"}, {"type": "bar"}]]
)

fig_total.add_trace(
    go.Bar(
        y=df_results['Ticker'],
        x=df_results['Buy_Signal_Score'],
        orientation='h',
        name='기술적 점수',
        marker_color='#2196F3',
        text=df_results['Buy_Signal_Score'],
        textposition='auto'
    ),
    row=1, col=1
)

fig_total.add_trace(
    go.Bar(
        y=df_results['Ticker'],
        x=df_results['Total_Investment_Score'],
        orientation='h',
        name='종합 점수',
        marker_color='#4CAF50',
        text=df_results['Total_Investment_Score'],
        textposition='auto'
    ),
    row=1, col=2
)

fig_total.update_xaxes(title_text="점수", row=1, col=1)
fig_total.update_xaxes(title_text="점수", row=1, col=2)
fig_total.update_yaxes(title_text="종목", row=1, col=1)

fig_total.update_layout(
    title_text='📊 MAG 7 - 투자 점수 비교 (기술적 vs 종합)',
    height=500,
    showlegend=False,
    template='plotly_white'
)

fig_total.show()

# 차트 3: 공매도 vs 분기 수익률
fig_scatter = px.scatter(
    df_results,
    x='short_percent_float',
    y='Quarter_Return_%',
    size='Market_Cap_Trillion',
    color='Total_Investment_Score',
    hover_data=['Ticker', 'Company'],
    text='Ticker',
    color_continuous_scale='RdYlGn',
    title='📊 MAG 7 - 공매도 비율 vs 분기 수익률',
    labels={
        'short_percent_float': '공매도 비율 (%)',
        'Quarter_Return_%': '분기 수익률 (%)',
        'Total_Investment_Score': '종합 점수'
    }
)

fig_scatter.update_traces(textposition='top center', textfont_size=12)
fig_scatter.update_layout(height=600)
fig_scatter.show()

# ==================== 최종 요약 ====================

print("\n" + "=" * 80)
print("📈 MAG 7 투자 전략 최종 요약 (공매도 분석 포함)")
print("=" * 80)

print(f"\n📍 분석 기준: Anchored VWAP ({quarter_start.year} Q{quarter_num})")
print(f"📅 Anchor Point: {quarter_start.strftime('%Y-%m-%d')}")
print(f"🌟 분석 대상: MAGNIFICENT SEVEN (7개 종목)")

# ==================== 최종 요약 비교표 ====================

print("\n" + "=" * 150)
print("📊 MAG 7 종합 요약표 (기술적 분석 + 공매도)")
print("=" * 150)

summary_table = []

for idx, row in df_results.iterrows():
    rank = df_results.index.get_loc(idx) + 1

    summary_table.append({
        'Rank': rank,
        'Ticker': row['Ticker'],
        'Price': f"${row['Current_Price']:.2f}",
        'VWAP_Diff': f"{row['Price_vs_VWAP_%']:+.1f}%",
        'Q_Return': f"{row['Quarter_Return_%']:+.1f}%",
        'YF_Short%': f"{row.get('short_percent_float', 0):.2f}%",
        'Days_Cover': f"{row.get('short_ratio_days', 0):.1f}",
        'FINRA_Daily%': f"{row.get('daily_short_ratio', 0):.1f}%" if row.get('daily_short_ratio', 0) > 0 else "N/A",
        'Tech_Score': f"{row['Buy_Signal_Score']}/100",
        'Total_Score': f"{row['Total_Investment_Score']}/120",
        'Signal': '💚' if row['Total_Investment_Score'] >= 90 else '💛' if row['Total_Investment_Score'] >= 75 else '💙'
    })

df_summary = pd.DataFrame(summary_table)
print("\n")
print(df_summary.to_string(index=False))
print("=" * 150)

print("\n📖 컬럼 설명:")
print("  • VWAP_Diff: 현재가의 VWAP 대비 위치")
print("  • Q_Return: 현재 분기 수익률")
print("  • YF_Short%: Yahoo Finance 공매도 잔고 비율")
print("  • Days_Cover: 공매도 청산 소요일")
print("  • FINRA_Daily%: FINRA 일일 공매도 거래 비율")
print("  • Tech_Score: 기술적 분석 점수 (VWAP 기반)")
print("  • Total_Score: 종합 점수 (기술적 + 공매도)")
print("  • Signal: 💚=최우선 매수, 💛=강력 매수, 💙=관찰")

# 공매도 분석 요약
low_short = df_results[df_results['short_percent_float'] < 5]
print(f"\n✅ 낮은 공매도 (<5%): {len(low_short)}개")
if not low_short.empty:
    print(f"   → {', '.join(low_short['Ticker'].tolist())}")

high_short = df_results[df_results['short_percent_float'] >= 10]
print(f"⚠️ 높은 공매도 (≥10%): {len(high_short)}개")
if not high_short.empty:
    print(f"   → {', '.join(high_short['Ticker'].tolist())}")

# 종합 추천
top_score = df_results[df_results['Total_Investment_Score'] >= 90]
print(f"\n💚 최우선 매수 (90점↑): {len(top_score)}개")
if not top_score.empty:
    print(f"   → {', '.join(top_score['Ticker'].tolist())}")

strong_buy = df_results[(df_results['Total_Investment_Score'] >= 75) &
                        (df_results['Total_Investment_Score'] < 90)]
print(f"💛 강력 매수 (75-90점): {len(strong_buy)}개")
if not strong_buy.empty:
    print(f"   → {', '.join(strong_buy['Ticker'].tolist())}")

print("\n🏆 TOP 3 추천 (공매도 고려):")
for idx in range(min(3, len(df_results))):
    row = df_results.iloc[idx]
    print(f"  {idx+1}. {row['Ticker']} - {row['Company']}")
    print(f"     💰 시총: ${row['Market_Cap_Trillion']:.2f}T | " +
          f"종합점수: {row['Total_Investment_Score']:.0f}/120")
    print(f"     📊 VWAP: {row['Price_vs_VWAP_%']:+.2f}% | " +
          f"분기수익: {row['Quarter_Return_%']:+.2f}% | " +
          f"공매도: {row['short_percent_float']:.2f}%")

print("\n💡 투자 가이드 (공매도 포함):")
print("""
1. 💚 최우선 매수 (90점 이상):
   - VWAP 위 + 낮은 공매도 비율
   - 즉시 매수 검토, 강한 상승 모멘텀 예상

2. 💛 강력 매수 (75-90점):
   - 기술적으로 양호하나 공매도 약간 존재
   - VWAP 근처 조정 시 매수 기회

3. 💙 눌림목 대기 (60-75점):
   - 공매도 비율 확인 필수
   - VWAP 돌파 확인 후 매수

4. ⚠️ 공매도 주의사항:
   - 10% 이상: Short Squeeze 가능성 주의
   - 5% 미만: 시장의 신뢰 높음
   - Days to Cover 3일 이상: 변동성 증가 가능
""")

print("\n📊 공매도 데이터 출처:")
for _, row in df_results.iterrows():
    if pd.notna(row.get('data_source')):
        print(f"  • {row['Ticker']}: {row['data_source']}")

print("\n" + "=" * 80)
print("✨ MAGNIFICENT SEVEN 종합 분석 완료!")
print("=" * 80)

🌟 MAGNIFICENT SEVEN (MAG 7) 실시간 종합 분석 + 공매도 데이터

빅테크 7대 기업 리스트:
  • AAPL   - Apple Inc.                     | 아이폰, 생태계, 온디바이스 AI
  • MSFT   - Microsoft Corporation          | 클라우드(Azure), 생성형 AI (OpenAI 대주주)
  • GOOGL  - Alphabet Inc.                  | 구글 검색, 유튜브, AI (Gemini)
  • AMZN   - Amazon.com Inc.                | 전자상거래, 클라우드(AWS) 1위
  • NVDA   - NVIDIA Corporation             | AI 반도체(GPU) 독점적 지배자
  • META   - Meta Platforms Inc.            | 페이스북, 인스타그램, AI(Llama)
  • TSLA   - Tesla Inc.                     | 전기차, 자율주행, 로봇

🎯 Anchored VWAP 분석: 2025 Q4
📍 Anchor Point: 2025-10-01
📅 현재까지 거래일: 58일

🔍 MAG 7 종목 분석 중...

분석 중: AAPL (Apple Inc.)... ✓
분석 중: MSFT (Microsoft Corporation)... ✓
분석 중: GOOGL (Alphabet Inc.)... ✓
분석 중: AMZN (Amazon.com Inc.)... ✓
분석 중: NVDA (NVIDIA Corporation)... ✓
분석 중: META (Meta Platforms Inc.)... ✓
분석 중: TSLA (Tesla Inc.)... ✓

📊 공매도(Short Interest) 데이터 수집 중...
  📊 AAPL 공매도 데이터 수집... ✓
  📊 MSFT 공매도 데이터 수집... ✓
  📊 GOOGL 공매도 데이터 수집... ✓
  📊 AMZN 공매도 데이터 


📈 MAG 7 투자 전략 최종 요약 (공매도 분석 포함)

📍 분석 기준: Anchored VWAP (2025 Q4)
📅 Anchor Point: 2025-10-01
🌟 분석 대상: MAGNIFICENT SEVEN (7개 종목)

📊 MAG 7 종합 요약표 (기술적 분석 + 공매도)


 Rank Ticker   Price VWAP_Diff Q_Return YF_Short% Days_Cover FINRA_Daily% Tech_Score Total_Score Signal
    1  GOOGL $319.95    +15.1%   +30.6%     1.07%        1.9        31.6%     80/100     100/120      💚
    2   AAPL $277.55     +4.9%    +8.8%     0.78%        2.3        51.6%     75/100      95/120      💚
    3   TSLA $426.58     -1.5%    -7.2%     2.45%        0.8        50.7%     30/100      50/120      💙
    4   NVDA $180.26     -4.4%    -3.7%     1.02%        1.3        46.6%     25/100      45/120      💙
    5   AMZN $229.16     -0.2%    +3.9%     0.73%        1.4        36.5%     25/100      45/120      💙
    6   MSFT $485.50     -4.5%    -6.4%     0.65%        2.1        48.6%     15/100      35/120      💙
    7   META $633.61     -4.0%   -11.7%     1.39%        1.4        53.8%     10/100      30/120      💙

📖 컬럼 

In [6]:
# 필요한 라이브러리 설치 및 임포트
!pip install yfinance pandas numpy plotly requests -q

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
import requests
from io import StringIO
import time

warnings.filterwarnings('ignore')
pio.renderers.default = 'colab'

# ==================== FINRA 공매도 데이터 수집 함수 ====================

def get_finra_short_interest(ticker, retries=2):
    """
    FINRA에서 특정 종목의 공매도 잔고(Short Interest) 데이터 수집
    실제 CSV 파일 다운로드 방식 사용
    """
    try:
        # 1. 일별 공매도 거래량 데이터 수집
        daily_data = get_finra_short_volume_csv(ticker, days_back=10)

        # 2. 월별 공매도 잔고 데이터 수집
        monthly_data = get_finra_monthly_short_interest(ticker)

        # 데이터 통합
        result = {}

        if daily_data:
            result.update({
                'daily_short_ratio': daily_data['latest_short_ratio'],
                'avg_daily_short_ratio': daily_data['avg_short_ratio_10d'],
                'daily_data_date': daily_data['latest_date'],
                'daily_data_points': daily_data['data_points']
            })

        if monthly_data:
            result.update({
                'monthly_short_interest': monthly_data.get('short_interest', 0),
                'days_to_cover': monthly_data.get('days_to_cover', 0),
                'settlement_date': monthly_data.get('settlement_date', 'N/A')
            })

        return result if result else None

    except Exception as e:
        return None


def get_finra_short_volume_csv(ticker, days_back=10):
    """
    FINRA에서 일별 공매도 거래량 CSV 파일 다운로드 및 분석
    URL: https://cdn.finra.org/equity/regsho/daily/CNMSshvol[YYYYMMDD].txt
    """
    try:
        today = datetime.now()
        short_volume_data = []

        for days in range(days_back):
            check_date = today - timedelta(days=days)

            # 주말 건너뛰기
            if check_date.weekday() >= 5:  # 토요일(5), 일요일(6)
                continue

            date_str = check_date.strftime('%Y%m%d')

            # FINRA Daily Short Volume URL
            url = f"https://cdn.finra.org/equity/regsho/daily/CNMSshvol{date_str}.txt"

            try:
                response = requests.get(url, timeout=10)

                if response.status_code == 200:
                    # 파이프(|) 구분자로 CSV 읽기
                    df = pd.read_csv(StringIO(response.text), sep='|')

                    # 컬럼명 확인 및 정리
                    if 'Symbol' in df.columns or 'symbol' in df.columns:
                        # 대소문자 무관하게 처리
                        df.columns = df.columns.str.strip()
                        symbol_col = 'Symbol' if 'Symbol' in df.columns else 'symbol'

                        # 해당 종목 데이터 찾기
                        ticker_data = df[df[symbol_col].str.upper() == ticker.upper()]

                        if not ticker_data.empty:
                            row = ticker_data.iloc[0]

                            short_vol = row.get('ShortVolume', row.get('shortVolume', 0))
                            total_vol = row.get('TotalVolume', row.get('totalVolume', 0))

                            if pd.notna(short_vol) and pd.notna(total_vol) and total_vol > 0:
                                short_volume_data.append({
                                    'date': check_date.strftime('%Y-%m-%d'),
                                    'short_volume': int(short_vol),
                                    'total_volume': int(total_vol),
                                    'short_ratio': round(short_vol / total_vol * 100, 2)
                                })
            except:
                continue

        if short_volume_data:
            # 데이터프레임으로 변환
            df_short = pd.DataFrame(short_volume_data)

            # 최근 데이터 통계
            avg_short_ratio = df_short['short_ratio'].mean()
            latest_short_ratio = df_short.iloc[0]['short_ratio'] if len(df_short) > 0 else 0

            return {
                'ticker': ticker,
                'latest_date': df_short.iloc[0]['date'] if len(df_short) > 0 else 'N/A',
                'latest_short_ratio': latest_short_ratio,
                'avg_short_ratio_10d': round(avg_short_ratio, 2),
                'data_points': len(df_short),
                'historical_data': df_short
            }

        return None

    except Exception as e:
        return None


def get_finra_monthly_short_interest(ticker):
    """
    FINRA 월별 공매도 잔고 데이터 수집
    URL: https://cdn.finra.org/equity/regsho/monthly/[FILENAME].txt
    """
    try:
        # 최근 6개월 시도
        today = datetime.now()

        for months_back in range(6):
            check_date = today - timedelta(days=30 * months_back)

            # 월별 파일명 형식: CNMSshvol{YYYYMM}a.txt, CNMSshvol{YYYYMM}b.txt
            year_month = check_date.strftime('%Y%m')

            for suffix in ['a', 'b']:
                url = f"https://cdn.finra.org/equity/regsho/monthly/CNMSshvol{year_month}{suffix}.txt"

                try:
                    response = requests.get(url, timeout=10)

                    if response.status_code == 200:
                        df = pd.read_csv(StringIO(response.text), sep='|')

                        if 'Symbol' in df.columns or 'symbol' in df.columns:
                            df.columns = df.columns.str.strip()
                            symbol_col = 'Symbol' if 'Symbol' in df.columns else 'symbol'

                            ticker_data = df[df[symbol_col].str.upper() == ticker.upper()]

                            if not ticker_data.empty:
                                row = ticker_data.iloc[0]

                                return {
                                    'ticker': ticker,
                                    'settlement_date': row.get('SettlementDate', 'N/A'),
                                    'short_interest': row.get('ShortInterest', 0),
                                    'avg_daily_volume': row.get('AverageDailyVolume', 0),
                                    'days_to_cover': row.get('DaysToCover', 0)
                                }
                except:
                    continue

        return None

    except Exception as e:
        return None


def get_short_interest_from_yfinance(ticker):
    """
    Yahoo Finance에서 공매도 관련 데이터 수집 (대체 방법)
    """
    try:
        stock = yf.Ticker(ticker)
        info = stock.info

        short_data = {
            'ticker': ticker,
            'short_ratio': info.get('shortRatio', 0),  # Days to Cover
            'short_percent_float': info.get('shortPercentOfFloat', 0) * 100 if info.get('shortPercentOfFloat') else 0,
            'shares_short': info.get('sharesShort', 0),
            'shares_short_prior_month': info.get('sharesShortPriorMonth', 0),
            'short_percent_shares_out': info.get('sharesPercentSharesOut', 0) * 100 if info.get('sharesPercentSharesOut') else 0
        }

        # 변화율 계산
        if short_data['shares_short_prior_month'] > 0:
            short_data['short_change_pct'] = ((short_data['shares_short'] - short_data['shares_short_prior_month']) /
                                               short_data['shares_short_prior_month'] * 100)
        else:
            short_data['short_change_pct'] = 0

        return short_data

    except Exception as e:
        print(f"  ⚠️ Yahoo Finance 오류 ({ticker}): {str(e)}")
        return None


def get_comprehensive_short_data(ticker):
    """
    여러 소스에서 공매도 데이터 종합 수집
    - Yahoo Finance: 공매도 잔고 (Short Interest)
    - FINRA CSV: 일별 공매도 거래량 비율
    """
    # Yahoo Finance 데이터 (공매도 잔고)
    yf_data = get_short_interest_from_yfinance(ticker)

    # FINRA CSV 데이터 (일별 거래량 비율)
    finra_data = get_finra_short_interest(ticker)

    # 데이터 통합
    combined_data = {
        'ticker': ticker,
        'short_ratio_days': 0,
        'short_percent_float': 0,
        'shares_short_millions': 0,
        'short_change_pct': 0,
        'daily_short_ratio': 0,
        'avg_daily_short_ratio_10d': 0,
        'finra_latest_date': 'N/A',
        'data_source': []
    }

    if yf_data:
        combined_data.update({
            'short_ratio_days': round(yf_data.get('short_ratio', 0), 2),
            'short_percent_float': round(yf_data.get('short_percent_float', 0), 2),
            'shares_short_millions': round(yf_data.get('shares_short', 0) / 1e6, 2),
            'short_change_pct': round(yf_data.get('short_change_pct', 0), 2),
        })
        combined_data['data_source'].append('Yahoo Finance')

    if finra_data:
        # FINRA 일별 데이터 추가
        if 'daily_short_ratio' in finra_data:
            combined_data['daily_short_ratio'] = finra_data['daily_short_ratio']
            combined_data['avg_daily_short_ratio_10d'] = finra_data['avg_daily_short_ratio']
            combined_data['finra_latest_date'] = finra_data.get('daily_data_date', 'N/A')
            combined_data['data_source'].append(f"FINRA Daily ({finra_data.get('daily_data_points', 0)}일)")

        # FINRA 월별 데이터로 Days to Cover 업데이트
        if 'days_to_cover' in finra_data and finra_data['days_to_cover'] > 0:
            combined_data['short_ratio_days'] = round(finra_data['days_to_cover'], 2)
            combined_data['data_source'].append('FINRA Monthly')

    # 데이터 소스를 문자열로 변환
    combined_data['data_source'] = ' + '.join(combined_data['data_source']) if combined_data['data_source'] else 'N/A'

    return combined_data


# ==================== Magnificent Seven 정의 ====================

MAG7_STOCKS = {
    'AAPL': {
        'name': 'Apple Inc.',
        'description': '아이폰, 생태계, 온디바이스 AI',
        'sector': 'Technology',
        'industry': 'Consumer Electronics'
    },
    'MSFT': {
        'name': 'Microsoft Corporation',
        'description': '클라우드(Azure), 생성형 AI (OpenAI 대주주)',
        'sector': 'Technology',
        'industry': 'Software'
    },
    'GOOGL': {
        'name': 'Alphabet Inc.',
        'description': '구글 검색, 유튜브, AI (Gemini)',
        'sector': 'Communication Services',
        'industry': 'Internet Content & Information'
    },
    'AMZN': {
        'name': 'Amazon.com Inc.',
        'description': '전자상거래, 클라우드(AWS) 1위',
        'sector': 'Consumer Cyclical',
        'industry': 'Internet Retail'
    },
    'NVDA': {
        'name': 'NVIDIA Corporation',
        'description': 'AI 반도체(GPU) 독점적 지배자',
        'sector': 'Technology',
        'industry': 'Semiconductors'
    },
    'META': {
        'name': 'Meta Platforms Inc.',
        'description': '페이스북, 인스타그램, AI(Llama)',
        'sector': 'Communication Services',
        'industry': 'Internet Content & Information'
    },
    'TSLA': {
        'name': 'Tesla Inc.',
        'description': '전기차, 자율주행, 로봇',
        'sector': 'Consumer Cyclical',
        'industry': 'Auto Manufacturers'
    }
}

print("=" * 80)
print("🌟 MAGNIFICENT SEVEN (MAG 7) 실시간 종합 분석 + 공매도 데이터")
print("=" * 80)
print(f"\n📅 데이터 수집 일시: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🌍 시간대: UTC+9 (한국 시간)")
print("\n빅테크 7대 기업 리스트:")
for ticker, info in MAG7_STOCKS.items():
    print(f"  • {ticker:6} - {info['name']:30} | {info['description']}")
print("=" * 80)


def get_current_quarter_start():
    """현재 분기 시작일 계산"""
    now = datetime.now()
    quarter = (now.month - 1) // 3
    quarter_start_month = quarter * 3 + 1
    quarter_start = datetime(now.year, quarter_start_month, 1)
    return quarter_start


def calculate_anchored_vwap(df):
    """Anchored VWAP 계산"""
    df = df.copy()
    df['Typical_Price'] = (df['High'] + df['Low'] + df['Close']) / 3
    df['TP_Volume'] = df['Typical_Price'] * df['Volume']
    df['Cumulative_TP_Volume'] = df['TP_Volume'].cumsum()
    df['Cumulative_Volume'] = df['Volume'].cumsum()
    df['Anchored_VWAP'] = df['Cumulative_TP_Volume'] / df['Cumulative_Volume']
    return df


def get_quarterly_vwap_analysis(ticker):
    """분기별 Anchored VWAP 분석"""
    try:
        quarter_start = get_current_quarter_start()
        end_date = datetime.now()
        quarter_num = (quarter_start.month - 1) // 3 + 1

        stock = yf.Ticker(ticker)
        df = stock.history(start=quarter_start, end=end_date)

        if df.empty or len(df) < 5:
            return None

        df = calculate_anchored_vwap(df)

        current_price = df['Close'].iloc[-1]
        current_vwap = df['Anchored_VWAP'].iloc[-1]
        above_vwap_ratio = (df['Close'] > df['Anchored_VWAP']).sum() / len(df) * 100
        recent_5days_avg = df['Close'].tail(5).mean()
        recent_10days_avg = df['Close'].tail(10).mean()

        recent_20 = df['Close'].tail(min(20, len(df)))
        uptrend_strength = (recent_20.diff() > 0).sum() / len(recent_20) * 100 if len(recent_20) > 1 else 50

        recent_volume = df['Volume'].tail(5).mean()
        avg_volume = df['Volume'].mean()
        volume_ratio = recent_volume / avg_volume if avg_volume > 0 else 1

        info = stock.info
        quarter_start_price = df['Close'].iloc[0]
        quarter_return = ((current_price - quarter_start_price) / quarter_start_price * 100)

        return {
            'Ticker': ticker,
            'Company': MAG7_STOCKS[ticker]['name'],
            'Description': MAG7_STOCKS[ticker]['description'],
            'Sector': MAG7_STOCKS[ticker]['sector'],
            'Quarter': f'{quarter_start.year} Q{quarter_num}',
            'Quarter_Start_Date': quarter_start.strftime('%Y-%m-%d'),
            'Trading_Days': len(df),
            'Current_Price': round(current_price, 2),
            'Anchored_VWAP': round(current_vwap, 2),
            'Quarter_Start_Price': round(quarter_start_price, 2),
            'Quarter_Return_%': round(quarter_return, 2),
            'Price_vs_VWAP_%': round((current_price - current_vwap) / current_vwap * 100, 2),
            'Above_VWAP_Days_%': round(above_vwap_ratio, 1),
            'Recent_5D_Avg': round(recent_5days_avg, 2),
            'Recent_10D_Avg': round(recent_10days_avg, 2),
            'Uptrend_Strength_%': round(uptrend_strength, 1),
            'Volume_Ratio': round(volume_ratio, 2),
            'Is_Above_VWAP': current_price > current_vwap,
            'Market_Cap': info.get('marketCap', 0),
            'Buy_Signal_Score': 0
        }

    except Exception as e:
        print(f"Error processing {ticker}: {str(e)}")
        return None


# ==================== 메인 분석 시작 ====================

quarter_start = get_current_quarter_start()
quarter_num = (quarter_start.month - 1) // 3 + 1

print("\n" + "=" * 80)
print(f"🎯 Anchored VWAP 분석: {quarter_start.year} Q{quarter_num}")
print(f"📍 Anchor Point: {quarter_start.strftime('%Y-%m-%d')}")
print(f"📅 현재까지 거래일: {(datetime.now() - quarter_start).days}일")
print("=" * 80)

# MAG 7 분석
results = []
mag7_tickers = list(MAG7_STOCKS.keys())

print("\n🔍 MAG 7 종목 분석 중...\n")

for ticker in mag7_tickers:
    print(f"분석 중: {ticker} ({MAG7_STOCKS[ticker]['name']})...", end=" ")
    result = get_quarterly_vwap_analysis(ticker)
    if result:
        results.append(result)
        print("✓")
    else:
        print("✗")

df_results = pd.DataFrame(results)

# 매수 신호 점수 계산
def calculate_buy_score(row):
    score = 0
    if row['Is_Above_VWAP']:
        score += 30

    price_diff = row['Price_vs_VWAP_%']
    if 0 < price_diff <= 5:
        score += 20
    elif 5 < price_diff <= 10:
        score += 10
    elif price_diff > 10:
        score += 5

    if row['Above_VWAP_Days_%'] >= 80:
        score += 20
    elif row['Above_VWAP_Days_%'] >= 60:
        score += 15
    elif row['Above_VWAP_Days_%'] >= 40:
        score += 10

    if row['Uptrend_Strength_%'] >= 60:
        score += 15
    elif row['Uptrend_Strength_%'] >= 50:
        score += 10

    if row['Volume_Ratio'] >= 1.2:
        score += 15
    elif row['Volume_Ratio'] >= 1.0:
        score += 10

    return min(score, 100)

df_results['Buy_Signal_Score'] = df_results.apply(calculate_buy_score, axis=1)

# 시가총액 추가
df_results['Market_Cap_Trillion'] = (df_results['Market_Cap'] / 1e12).round(3)

# ==================== 공매도 데이터 수집 ====================

print("\n" + "=" * 80)
print("📊 공매도(Short Interest) 데이터 수집 중...")
print("=" * 80)

short_data_list = []

for ticker in mag7_tickers:
    print(f"  📊 {ticker} 공매도 데이터 수집...", end=" ")
    short_data = get_comprehensive_short_data(ticker)
    if short_data:
        short_data_list.append(short_data)
        print("✓")
    else:
        print("✗")

df_short = pd.DataFrame(short_data_list)

# 공매도 데이터를 메인 데이터프레임에 병합
df_results = df_results.merge(df_short, left_on='Ticker', right_on='ticker', how='left')

# 공매도 스코어 계산 (낮을수록 좋음)
def calculate_short_score(row):
    """
    공매도 스코어: 공매도 비율이 낮을수록 높은 점수
    - 5% 미만: 매우 좋음 (20점)
    - 5-10%: 양호 (15점)
    - 10-20%: 보통 (10점)
    - 20% 이상: 주의 (5점)
    """
    short_pct = row.get('short_percent_float', 0)

    if short_pct < 5:
        return 20
    elif short_pct < 10:
        return 15
    elif short_pct < 20:
        return 10
    else:
        return 5

df_results['Short_Score'] = df_results.apply(calculate_short_score, axis=1)

# 종합 투자 점수 (기존 Buy Signal + Short Score)
df_results['Total_Investment_Score'] = df_results['Buy_Signal_Score'] + df_results['Short_Score']
df_results = df_results.sort_values('Total_Investment_Score', ascending=False)

# ==================== 결과 출력 ====================

print("\n" + "=" * 80)
print("📊 MAGNIFICENT SEVEN - 종합 분석 결과 (공매도 포함)")
print("=" * 80)

display_cols = ['Ticker', 'Company', 'Current_Price', 'Anchored_VWAP', 'Price_vs_VWAP_%',
                'Quarter_Return_%', 'short_percent_float', 'short_ratio_days',
                'daily_short_ratio', 'Buy_Signal_Score', 'Short_Score', 'Total_Investment_Score']

print(df_results[display_cols].to_string(index=False))

print("\n" + "=" * 80)
print("🏆 MAG 7 투자 추천 순위 (공매도 분석 포함)")
print("=" * 80)

for idx, row in df_results.iterrows():
    rank = df_results.index.get_loc(idx) + 1

    if rank == 1:
        medal = "🥇"
    elif rank == 2:
        medal = "🥈"
    elif rank == 3:
        medal = "🥉"
    else:
        medal = f"  {rank}"

    print(f"\n{medal} {row['Ticker']} - {row['Company']}")
    print(f"    🎯 {row['Description']}")
    print(f"    💰 시가총액: ${row['Market_Cap_Trillion']:.2f}T")
    print(f"    📈 현재가: ${row['Current_Price']} | VWAP: ${row['Anchored_VWAP']}")
    print(f"    📊 VWAP 대비: {row['Price_vs_VWAP_%']:+.2f}% | 분기 수익률: {row['Quarter_Return_%']:+.2f}%")
    print(f"    🎲 VWAP 위 거래일: {row['Above_VWAP_Days_%']}% | 추세 강도: {row['Uptrend_Strength_%']}%")

    # 공매도 정보
    short_pct = row.get('short_percent_float', 0)
    short_days = row.get('short_ratio_days', 0)
    daily_short = row.get('daily_short_ratio', 0)

    print(f"    🔴 공매도 비율: {short_pct:.2f}% of float | 커버 소요일: {short_days:.1f}일")

    if daily_short > 0:
        print(f"    📊 최근 일평균 공매도 거래비율: {daily_short:.1f}%")

    if short_pct < 5:
        short_status = "✅ 매우 낮음 (긍정)"
    elif short_pct < 10:
        short_status = "🟢 양호"
    elif short_pct < 20:
        short_status = "🟡 보통"
    else:
        short_status = "🔴 높음 (주의)"

    print(f"    🎯 공매도 상태: {short_status}")

    print(f"    ⭐ 기술적 점수: {row['Buy_Signal_Score']}/100 | 공매도 점수: {row['Short_Score']}/20", end="")
    print(f" | 종합: {row['Total_Investment_Score']}/120")

    if row['Total_Investment_Score'] >= 90:
        print("       → 💚 최우선 매수 추천")
    elif row['Total_Investment_Score'] >= 75:
        print("       → 💛 강력 매수")
    elif row['Total_Investment_Score'] >= 60:
        print("       → 💙 눌림목 대기")
    else:
        print("       → ⚪ 관찰 필요")

# ==================== Yahoo Finance vs FINRA 비교 테이블 ====================

print("\n" + "=" * 80)
print("📊 Yahoo Finance vs FINRA 공매도 데이터 상세 비교")
print("=" * 80)

comparison_data = []

for idx, row in df_results.iterrows():
    ticker = row['Ticker']

    comparison_data.append({
        'Ticker': ticker,
        'Company': row['Company'][:25],

        # Yahoo Finance 데이터
        'YF_Short_%_Float': f"{row.get('short_percent_float', 0):.2f}%",
        'YF_Days_to_Cover': f"{row.get('short_ratio_days', 0):.2f}",
        'YF_Shares_Short_M': f"{row.get('shares_short_millions', 0):.1f}M",
        'YF_Change_MoM': f"{row.get('short_change_pct', 0):+.1f}%",

        # FINRA 데이터
        'FINRA_Daily_Short_%': f"{row.get('daily_short_ratio', 0):.1f}%" if row.get('daily_short_ratio', 0) > 0 else "N/A",
        'FINRA_10D_Avg_%': f"{row.get('avg_daily_short_ratio_10d', 0):.1f}%" if row.get('avg_daily_short_ratio_10d', 0) > 0 else "N/A",
        'FINRA_Data_Date': row.get('finra_latest_date', 'N/A'),

        # 데이터 소스
        'Data_Source': row.get('data_source', 'N/A')
    })

df_comparison = pd.DataFrame(comparison_data)

print("\n📋 상세 비교표:")
print("=" * 150)

# 컬럼 순서 및 너비 조정을 위한 포맷팅
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 150)
pd.set_option('display.max_colwidth', 30)

print(df_comparison.to_string(index=False))
print("=" * 150)

print("\n" + "=" * 120)
print("📖 컬럼 설명:")
print("=" * 120)
print("""
[Yahoo Finance 데이터 - 공매도 잔고 (Short Interest)]
  • YF_Short_%_Float     : 유통주식(Float) 대비 공매도 비율
                           → 낮을수록 좋음 (일반적으로 5% 미만이 건강)

  • YF_Days_to_Cover     : 공매도 청산 소요일 (Short Interest / Avg Daily Volume)
                           → 3일 이상이면 Short Squeeze 가능성

  • YF_Shares_Short_M    : 현재 공매도된 총 주식 수 (백만 주)

  • YF_Change_MoM        : 전월 대비 공매도 변화율
                           → (+)는 공매도 증가 (약세 신호)
                           → (-)는 공매도 감소 (강세 신호)

[FINRA 데이터 - 일별 공매도 거래량 (Daily Short Volume)]
  • FINRA_Daily_Short_%  : 최근 거래일의 공매도 거래량 비율
                           → 해당일 전체 거래량 중 공매도 거래 비중

  • FINRA_10D_Avg_%      : 최근 10거래일 평균 공매도 거래 비율
                           → 30-40%는 정상 (마켓메이킹 포함)
                           → 50% 이상은 강한 약세 압력

  • FINRA_Data_Date      : FINRA 데이터의 최근 수집 날짜
                           → 실제 공매도 거래 발생일

[중요한 차이점]
  ✓ Yahoo Finance : 공매도 "잔고" (누적된 공매도 포지션) - 월 2회 업데이트
  ✓ FINRA Daily   : 공매도 "거래량" (해당일의 신규 공매도 거래) - 매일 업데이트

  → 두 지표는 다른 의미이므로 함께 분석해야 정확!
""")

# ==================== 데이터 품질 분석 ====================

print("\n" + "=" * 80)
print("🔍 데이터 품질 및 신뢰도 분석")
print("=" * 80)

quality_analysis = []

for idx, row in df_results.iterrows():
    ticker = row['Ticker']

    # 데이터 완성도 평가
    yf_complete = (
        row.get('short_percent_float', 0) > 0 and
        row.get('shares_short_millions', 0) > 0
    )

    finra_complete = row.get('daily_short_ratio', 0) > 0

    data_quality = "⭐⭐⭐" if (yf_complete and finra_complete) else \
                   "⭐⭐" if (yf_complete or finra_complete) else "⭐"

    # 공매도 신호 강도
    short_pct = row.get('short_percent_float', 0)
    daily_short = row.get('daily_short_ratio', 0)

    if short_pct < 3 and daily_short < 40:
        signal = "💚 매우 긍정"
    elif short_pct < 5 and daily_short < 45:
        signal = "🟢 긍정"
    elif short_pct < 10 and daily_short < 50:
        signal = "🟡 중립"
    else:
        signal = "🔴 약세 압력"

    quality_analysis.append({
        'Ticker': ticker,
        'YF_Complete': '✓' if yf_complete else '✗',
        'FINRA_Complete': '✓' if finra_complete else '✗',
        'Data_Quality': data_quality,
        'Short_Signal': signal,
        'Interpretation': f"잔고 {short_pct:.1f}% / 일거래 {daily_short:.0f}%"
    })

df_quality = pd.DataFrame(quality_analysis)

print("\n📊 종목별 데이터 품질 및 공매도 신호:")
print("=" * 100)
print(df_quality.to_string(index=False))
print("=" * 100)

print("\n" + "=" * 80)
print("💡 통합 해석 가이드")
print("=" * 80)
print("""
[최적 투자 신호] 💚
  ✓ Yahoo: Short % < 3% + FINRA: Daily < 40%
  → 공매도 잔고도 낮고, 일일 공매도 거래도 적음
  → 시장의 강한 신뢰, 매수 추천

[양호한 신호] 🟢
  ✓ Yahoo: Short % < 5% + FINRA: Daily < 45%
  → 전반적으로 건강한 수준
  → 적극 매수 고려

[주의 필요] 🟡
  ✓ Yahoo: Short % 5-10% + FINRA: Daily 45-50%
  → 약간의 약세 포지션 존재
  → 기술적 분석 병행 필수

[약세 압력] 🔴
  ✓ Yahoo: Short % > 10% 또는 FINRA: Daily > 50%
  → 강한 공매도 압력
  → Short Squeeze 가능성은 있으나 위험도 높음
""")

# ==================== 공매도 관련 차트 ====================

print("\n📈 공매도 분석 차트 생성 중...")

# 차트 0: Yahoo Finance vs FINRA 비교 차트
fig_compare = make_subplots(
    rows=2, cols=1,
    subplot_titles=(
        'Yahoo Finance: Short % of Float (공매도 잔고)',
        'FINRA: Daily Short Volume Ratio (일일 공매도 거래 비율)'
    ),
    vertical_spacing=0.15,
    specs=[[{"type": "bar"}], [{"type": "bar"}]]
)

# Yahoo Finance 공매도 비율
fig_compare.add_trace(
    go.Bar(
        x=df_results['Ticker'],
        y=df_results['short_percent_float'],
        name='YF Short %',
        marker=dict(
            color=df_results['short_percent_float'],
            colorscale='RdYlGn_r',
            showscale=False
        ),
        text=df_results['short_percent_float'].round(2),
        textposition='auto',
        hovertemplate='<b>%{x}</b><br>공매도 잔고: %{y:.2f}%<extra></extra>'
    ),
    row=1, col=1
)

# FINRA 일일 공매도 비율
fig_compare.add_trace(
    go.Bar(
        x=df_results['Ticker'],
        y=df_results['daily_short_ratio'],
        name='FINRA Daily %',
        marker=dict(
            color=df_results['daily_short_ratio'],
            colorscale='Blues',
            showscale=False
        ),
        text=df_results['daily_short_ratio'].round(1),
        textposition='auto',
        hovertemplate='<b>%{x}</b><br>일일 공매도: %{y:.1f}%<extra></extra>'
    ),
    row=2, col=1
)

# 참고선 추가
fig_compare.add_hline(y=5, line_dash="dash", line_color="green", row=1, col=1,
                      annotation_text="건강 기준선 (5%)", annotation_position="right")
fig_compare.add_hline(y=40, line_dash="dash", line_color="orange", row=2, col=1,
                      annotation_text="정상 범위 (40%)", annotation_position="right")

fig_compare.update_xaxes(title_text="종목", row=1, col=1)
fig_compare.update_xaxes(title_text="종목", row=2, col=1)
fig_compare.update_yaxes(title_text="Short % of Float", row=1, col=1)
fig_compare.update_yaxes(title_text="Daily Short %", row=2, col=1)

fig_compare.update_layout(
    title_text='📊 Yahoo Finance vs FINRA 공매도 데이터 비교',
    height=800,
    showlegend=False,
    template='plotly_white'
)

fig_compare.show()

# 차트 0-2: 산점도로 두 지표 관계 분석
fig_scatter_short = px.scatter(
    df_results,
    x='short_percent_float',
    y='daily_short_ratio',
    size='Market_Cap_Trillion',
    color='Total_Investment_Score',
    hover_data=['Ticker', 'Company'],
    text='Ticker',
    color_continuous_scale='RdYlGn',
    title='📊 공매도 잔고 vs 일일 공매도 거래 비율 관계',
    labels={
        'short_percent_float': 'Yahoo Finance: Short % of Float',
        'daily_short_ratio': 'FINRA: Daily Short Volume %',
        'Total_Investment_Score': '종합 점수'
    }
)

# 참고선 추가
fig_scatter_short.add_hline(y=40, line_dash="dash", line_color="gray",
                            annotation_text="FINRA 정상 기준", annotation_position="right")
fig_scatter_short.add_vline(x=5, line_dash="dash", line_color="gray",
                            annotation_text="YF 건강 기준", annotation_position="top")

fig_scatter_short.update_traces(textposition='top center', textfont_size=12)
fig_scatter_short.update_layout(height=600)
fig_scatter_short.show()

# 차트 1: 공매도 비율 비교
fig_short = go.Figure()

colors_short = ['#00C853' if x < 5 else '#FFD600' if x < 10 else '#FF5722'
                for x in df_results['short_percent_float']]

fig_short.add_trace(go.Bar(
    y=df_results['Ticker'],
    x=df_results['short_percent_float'],
    orientation='h',
    marker=dict(color=colors_short),
    text=df_results['short_percent_float'].round(2),
    textposition='auto',
    hovertemplate='<b>%{y}</b><br>공매도 비율: %{x:.2f}%<extra></extra>'
))

fig_short.update_layout(
    title='🔴 MAG 7 - 공매도 비율 (Short % of Float)',
    xaxis_title='공매도 비율 (%)',
    yaxis_title='종목',
    height=500,
    template='plotly_white'
)

# 참고선 추가
fig_short.add_vline(x=5, line_dash="dash", line_color="green",
                    annotation_text="낮음", annotation_position="top")
fig_short.add_vline(x=10, line_dash="dash", line_color="orange",
                    annotation_text="보통", annotation_position="top")
fig_short.add_vline(x=20, line_dash="dash", line_color="red",
                    annotation_text="높음", annotation_position="top")

fig_short.show()

# 차트 2: 종합 투자 점수 (기술적 + 공매도)
fig_total = make_subplots(
    rows=1, cols=2,
    subplot_titles=('기술적 분석 점수', '종합 투자 점수'),
    specs=[[{"type": "bar"}, {"type": "bar"}]]
)

fig_total.add_trace(
    go.Bar(
        y=df_results['Ticker'],
        x=df_results['Buy_Signal_Score'],
        orientation='h',
        name='기술적 점수',
        marker_color='#2196F3',
        text=df_results['Buy_Signal_Score'],
        textposition='auto'
    ),
    row=1, col=1
)

fig_total.add_trace(
    go.Bar(
        y=df_results['Ticker'],
        x=df_results['Total_Investment_Score'],
        orientation='h',
        name='종합 점수',
        marker_color='#4CAF50',
        text=df_results['Total_Investment_Score'],
        textposition='auto'
    ),
    row=1, col=2
)

fig_total.update_xaxes(title_text="점수", row=1, col=1)
fig_total.update_xaxes(title_text="점수", row=1, col=2)
fig_total.update_yaxes(title_text="종목", row=1, col=1)

fig_total.update_layout(
    title_text='📊 MAG 7 - 투자 점수 비교 (기술적 vs 종합)',
    height=500,
    showlegend=False,
    template='plotly_white'
)

fig_total.show()

# 차트 3: 공매도 vs 분기 수익률
fig_scatter = px.scatter(
    df_results,
    x='short_percent_float',
    y='Quarter_Return_%',
    size='Market_Cap_Trillion',
    color='Total_Investment_Score',
    hover_data=['Ticker', 'Company'],
    text='Ticker',
    color_continuous_scale='RdYlGn',
    title='📊 MAG 7 - 공매도 비율 vs 분기 수익률',
    labels={
        'short_percent_float': '공매도 비율 (%)',
        'Quarter_Return_%': '분기 수익률 (%)',
        'Total_Investment_Score': '종합 점수'
    }
)

fig_scatter.update_traces(textposition='top center', textfont_size=12)
fig_scatter.update_layout(height=600)
fig_scatter.show()

# ==================== 최종 요약 ====================

print("\n" + "=" * 80)
print("📈 MAG 7 투자 전략 최종 요약 (공매도 분석 포함)")
print("=" * 80)

print(f"\n📍 분석 기준: Anchored VWAP ({quarter_start.year} Q{quarter_num})")
print(f"📅 Anchor Point: {quarter_start.strftime('%Y-%m-%d')}")
print(f"🌟 분석 대상: MAGNIFICENT SEVEN (7개 종목)")

# ==================== 최종 요약 비교표 ====================

print("\n" + "=" * 150)
print("📊 MAG 7 종합 요약표 (기술적 분석 + 공매도)")
print("=" * 150)

summary_table = []

for idx, row in df_results.iterrows():
    rank = df_results.index.get_loc(idx) + 1

    summary_table.append({
        'Rank': rank,
        'Ticker': row['Ticker'],
        'Price': f"${row['Current_Price']:.2f}",
        'VWAP_Diff': f"{row['Price_vs_VWAP_%']:+.1f}%",
        'Q_Return': f"{row['Quarter_Return_%']:+.1f}%",
        'YF_Short%': f"{row.get('short_percent_float', 0):.2f}%",
        'Days_Cover': f"{row.get('short_ratio_days', 0):.1f}",
        'FINRA_Daily%': f"{row.get('daily_short_ratio', 0):.1f}%" if row.get('daily_short_ratio', 0) > 0 else "N/A",
        'Tech_Score': f"{row['Buy_Signal_Score']}/100",
        'Total_Score': f"{row['Total_Investment_Score']}/120",
        'Signal': '💚' if row['Total_Investment_Score'] >= 90 else '💛' if row['Total_Investment_Score'] >= 75 else '💙'
    })

df_summary = pd.DataFrame(summary_table)
print("\n")
print(df_summary.to_string(index=False))
print("=" * 150)

print("\n📖 컬럼 설명:")
print("  • VWAP_Diff: 현재가의 VWAP 대비 위치")
print("  • Q_Return: 현재 분기 수익률")
print("  • YF_Short%: Yahoo Finance 공매도 잔고 비율")
print("  • Days_Cover: 공매도 청산 소요일")
print("  • FINRA_Daily%: FINRA 일일 공매도 거래 비율")
print("  • Tech_Score: 기술적 분석 점수 (VWAP 기반)")
print("  • Total_Score: 종합 점수 (기술적 + 공매도)")
print("  • Signal: 💚=최우선 매수, 💛=강력 매수, 💙=관찰")

# 공매도 분석 요약
low_short = df_results[df_results['short_percent_float'] < 5]
print(f"\n✅ 낮은 공매도 (<5%): {len(low_short)}개")
if not low_short.empty:
    print(f"   → {', '.join(low_short['Ticker'].tolist())}")

high_short = df_results[df_results['short_percent_float'] >= 10]
print(f"⚠️ 높은 공매도 (≥10%): {len(high_short)}개")
if not high_short.empty:
    print(f"   → {', '.join(high_short['Ticker'].tolist())}")

# 종합 추천
top_score = df_results[df_results['Total_Investment_Score'] >= 90]
print(f"\n💚 최우선 매수 (90점↑): {len(top_score)}개")
if not top_score.empty:
    print(f"   → {', '.join(top_score['Ticker'].tolist())}")

strong_buy = df_results[(df_results['Total_Investment_Score'] >= 75) &
                        (df_results['Total_Investment_Score'] < 90)]
print(f"💛 강력 매수 (75-90점): {len(strong_buy)}개")
if not strong_buy.empty:
    print(f"   → {', '.join(strong_buy['Ticker'].tolist())}")

print("\n🏆 TOP 3 추천 (공매도 고려):")
for idx in range(min(3, len(df_results))):
    row = df_results.iloc[idx]
    print(f"  {idx+1}. {row['Ticker']} - {row['Company']}")
    print(f"     💰 시총: ${row['Market_Cap_Trillion']:.2f}T | " +
          f"종합점수: {row['Total_Investment_Score']:.0f}/120")
    print(f"     📊 VWAP: {row['Price_vs_VWAP_%']:+.2f}% | " +
          f"분기수익: {row['Quarter_Return_%']:+.2f}% | " +
          f"공매도: {row['short_percent_float']:.2f}%")

print("\n💡 투자 가이드 (공매도 포함):")
print("""
1. 💚 최우선 매수 (90점 이상):
   - VWAP 위 + 낮은 공매도 비율
   - 즉시 매수 검토, 강한 상승 모멘텀 예상

2. 💛 강력 매수 (75-90점):
   - 기술적으로 양호하나 공매도 약간 존재
   - VWAP 근처 조정 시 매수 기회

3. 💙 눌림목 대기 (60-75점):
   - 공매도 비율 확인 필수
   - VWAP 돌파 확인 후 매수

4. ⚠️ 공매도 주의사항:
   - 10% 이상: Short Squeeze 가능성 주의
   - 5% 미만: 시장의 신뢰 높음
   - Days to Cover 3일 이상: 변동성 증가 가능
""")

print("\n📊 공매도 데이터 출처 및 수집 날짜:")
print("=" * 80)
for _, row in df_results.iterrows():
    finra_date = row.get('finra_latest_date', 'N/A')
    data_source = row.get('data_source', 'N/A')

    print(f"  • {row['Ticker']:6}")
    print(f"    - 데이터 소스: {data_source}")
    if finra_date != 'N/A':
        print(f"    - FINRA 최근 데이터: {finra_date}")
    print()

print("\n" + "=" * 80)
print("✨ MAGNIFICENT SEVEN 종합 분석 완료!")
print("=" * 80)

🌟 MAGNIFICENT SEVEN (MAG 7) 실시간 종합 분석 + 공매도 데이터

📅 데이터 수집 일시: 2025-11-28 12:23:47
🌍 시간대: UTC+9 (한국 시간)

빅테크 7대 기업 리스트:
  • AAPL   - Apple Inc.                     | 아이폰, 생태계, 온디바이스 AI
  • MSFT   - Microsoft Corporation          | 클라우드(Azure), 생성형 AI (OpenAI 대주주)
  • GOOGL  - Alphabet Inc.                  | 구글 검색, 유튜브, AI (Gemini)
  • AMZN   - Amazon.com Inc.                | 전자상거래, 클라우드(AWS) 1위
  • NVDA   - NVIDIA Corporation             | AI 반도체(GPU) 독점적 지배자
  • META   - Meta Platforms Inc.            | 페이스북, 인스타그램, AI(Llama)
  • TSLA   - Tesla Inc.                     | 전기차, 자율주행, 로봇

🎯 Anchored VWAP 분석: 2025 Q4
📍 Anchor Point: 2025-10-01
📅 현재까지 거래일: 58일

🔍 MAG 7 종목 분석 중...

분석 중: AAPL (Apple Inc.)... ✓
분석 중: MSFT (Microsoft Corporation)... ✓
분석 중: GOOGL (Alphabet Inc.)... ✓
분석 중: AMZN (Amazon.com Inc.)... ✓
분석 중: NVDA (NVIDIA Corporation)... ✓
분석 중: META (Meta Platforms Inc.)... ✓
분석 중: TSLA (Tesla Inc.)... ✓

📊 공매도(Short Interest) 데이터 수집 중...
  📊 AAPL 공매도 데이터 수집... ✓
  📊 MSFT 공매도 


📈 MAG 7 투자 전략 최종 요약 (공매도 분석 포함)

📍 분석 기준: Anchored VWAP (2025 Q4)
📅 Anchor Point: 2025-10-01
🌟 분석 대상: MAGNIFICENT SEVEN (7개 종목)

📊 MAG 7 종합 요약표 (기술적 분석 + 공매도)


 Rank Ticker   Price VWAP_Diff Q_Return YF_Short% Days_Cover FINRA_Daily% Tech_Score Total_Score Signal
    1  GOOGL $319.95    +15.1%   +30.6%     1.07%        1.9        31.6%     80/100     100/120      💚
    2   AAPL $277.55     +4.9%    +8.8%     0.78%        2.3        51.6%     75/100      95/120      💚
    3   TSLA $426.58     -1.5%    -7.2%     2.45%        0.8        50.7%     30/100      50/120      💙
    4   NVDA $180.26     -4.4%    -3.7%     1.02%        1.3        46.6%     25/100      45/120      💙
    5   AMZN $229.16     -0.2%    +3.9%     0.73%        1.4        36.5%     25/100      45/120      💙
    6   MSFT $485.50     -4.5%    -6.4%     0.65%        2.1        48.6%     15/100      35/120      💙
    7   META $633.61     -4.0%   -11.7%     1.39%        1.4        53.8%     10/100      30/120      💙

📖 컬럼 

In [7]:
# 필요한 라이브러리 설치 및 임포트
!pip install yfinance pandas numpy plotly requests -q

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
import requests
from io import StringIO
import time

warnings.filterwarnings('ignore')
pio.renderers.default = 'colab'

# ==================== FINRA 공매도 데이터 수집 함수 ====================

def get_finra_short_interest(ticker, retries=2):
    """
    FINRA에서 특정 종목의 공매도 잔고(Short Interest) 데이터 수집
    실제 CSV 파일 다운로드 방식 사용
    """
    try:
        # 1. 일별 공매도 거래량 데이터 수집
        daily_data = get_finra_short_volume_csv(ticker, days_back=10)

        # 2. 월별 공매도 잔고 데이터 수집
        monthly_data = get_finra_monthly_short_interest(ticker)

        # 데이터 통합
        result = {}

        if daily_data:
            result.update({
                'daily_short_ratio': daily_data['latest_short_ratio'],
                'avg_daily_short_ratio': daily_data['avg_short_ratio_10d'],
                'daily_data_date': daily_data['latest_date'],
                'daily_data_points': daily_data['data_points']
            })

        if monthly_data:
            result.update({
                'monthly_short_interest': monthly_data.get('short_interest', 0),
                'days_to_cover': monthly_data.get('days_to_cover', 0),
                'settlement_date': monthly_data.get('settlement_date', 'N/A')
            })

        return result if result else None

    except Exception as e:
        return None


def get_finra_short_volume_csv(ticker, days_back=10):
    """
    FINRA에서 일별 공매도 거래량 CSV 파일 다운로드 및 분석
    URL: https://cdn.finra.org/equity/regsho/daily/CNMSshvol[YYYYMMDD].txt
    """
    try:
        today = datetime.now()
        short_volume_data = []

        for days in range(days_back):
            check_date = today - timedelta(days=days)

            # 주말 건너뛰기
            if check_date.weekday() >= 5:  # 토요일(5), 일요일(6)
                continue

            date_str = check_date.strftime('%Y%m%d')

            # FINRA Daily Short Volume URL
            url = f"https://cdn.finra.org/equity/regsho/daily/CNMSshvol{date_str}.txt"

            try:
                response = requests.get(url, timeout=10)

                if response.status_code == 200:
                    # 파이프(|) 구분자로 CSV 읽기
                    df = pd.read_csv(StringIO(response.text), sep='|')

                    # 컬럼명 확인 및 정리
                    if 'Symbol' in df.columns or 'symbol' in df.columns:
                        # 대소문자 무관하게 처리
                        df.columns = df.columns.str.strip()
                        symbol_col = 'Symbol' if 'Symbol' in df.columns else 'symbol'

                        # 해당 종목 데이터 찾기
                        ticker_data = df[df[symbol_col].str.upper() == ticker.upper()]

                        if not ticker_data.empty:
                            row = ticker_data.iloc[0]

                            short_vol = row.get('ShortVolume', row.get('shortVolume', 0))
                            total_vol = row.get('TotalVolume', row.get('totalVolume', 0))

                            if pd.notna(short_vol) and pd.notna(total_vol) and total_vol > 0:
                                short_volume_data.append({
                                    'date': check_date.strftime('%Y-%m-%d'),
                                    'short_volume': int(short_vol),
                                    'total_volume': int(total_vol),
                                    'short_ratio': round(short_vol / total_vol * 100, 2)
                                })
            except:
                continue

        if short_volume_data:
            # 데이터프레임으로 변환
            df_short = pd.DataFrame(short_volume_data)

            # 최근 데이터 통계
            avg_short_ratio = df_short['short_ratio'].mean()
            latest_short_ratio = df_short.iloc[0]['short_ratio'] if len(df_short) > 0 else 0

            return {
                'ticker': ticker,
                'latest_date': df_short.iloc[0]['date'] if len(df_short) > 0 else 'N/A',
                'latest_short_ratio': latest_short_ratio,
                'avg_short_ratio_10d': round(avg_short_ratio, 2),
                'data_points': len(df_short),
                'historical_data': df_short  # 시계열 데이터 저장
            }

        return None

    except Exception as e:
        return None


def get_extended_finra_history(ticker, days_back=60):
    """
    FINRA 데이터 60일 수집 (시계열 차트용)
    """
    return get_finra_short_volume_csv(ticker, days_back=days_back)


def get_finra_monthly_short_interest(ticker):
    """
    FINRA 월별 공매도 잔고 데이터 수집
    URL: https://cdn.finra.org/equity/regsho/monthly/[FILENAME].txt
    """
    try:
        # 최근 6개월 시도
        today = datetime.now()

        for months_back in range(6):
            check_date = today - timedelta(days=30 * months_back)

            # 월별 파일명 형식: CNMSshvol{YYYYMM}a.txt, CNMSshvol{YYYYMM}b.txt
            year_month = check_date.strftime('%Y%m')

            for suffix in ['a', 'b']:
                url = f"https://cdn.finra.org/equity/regsho/monthly/CNMSshvol{year_month}{suffix}.txt"

                try:
                    response = requests.get(url, timeout=10)

                    if response.status_code == 200:
                        df = pd.read_csv(StringIO(response.text), sep='|')

                        if 'Symbol' in df.columns or 'symbol' in df.columns:
                            df.columns = df.columns.str.strip()
                            symbol_col = 'Symbol' if 'Symbol' in df.columns else 'symbol'

                            ticker_data = df[df[symbol_col].str.upper() == ticker.upper()]

                            if not ticker_data.empty:
                                row = ticker_data.iloc[0]

                                return {
                                    'ticker': ticker,
                                    'settlement_date': row.get('SettlementDate', 'N/A'),
                                    'short_interest': row.get('ShortInterest', 0),
                                    'avg_daily_volume': row.get('AverageDailyVolume', 0),
                                    'days_to_cover': row.get('DaysToCover', 0)
                                }
                except:
                    continue

        return None

    except Exception as e:
        return None


def get_short_interest_from_yfinance(ticker):
    """
    Yahoo Finance에서 공매도 관련 데이터 수집 (대체 방법)
    """
    try:
        stock = yf.Ticker(ticker)
        info = stock.info

        short_data = {
            'ticker': ticker,
            'short_ratio': info.get('shortRatio', 0),  # Days to Cover
            'short_percent_float': info.get('shortPercentOfFloat', 0) * 100 if info.get('shortPercentOfFloat') else 0,
            'shares_short': info.get('sharesShort', 0),
            'shares_short_prior_month': info.get('sharesShortPriorMonth', 0),
            'short_percent_shares_out': info.get('sharesPercentSharesOut', 0) * 100 if info.get('sharesPercentSharesOut') else 0
        }

        # 변화율 계산
        if short_data['shares_short_prior_month'] > 0:
            short_data['short_change_pct'] = ((short_data['shares_short'] - short_data['shares_short_prior_month']) /
                                               short_data['shares_short_prior_month'] * 100)
        else:
            short_data['short_change_pct'] = 0

        return short_data

    except Exception as e:
        print(f"  ⚠️ Yahoo Finance 오류 ({ticker}): {str(e)}")
        return None


def get_comprehensive_short_data(ticker):
    """
    여러 소스에서 공매도 데이터 종합 수집
    - Yahoo Finance: 공매도 잔고 (Short Interest)
    - FINRA CSV: 일별 공매도 거래량 비율
    """
    # Yahoo Finance 데이터 (공매도 잔고)
    yf_data = get_short_interest_from_yfinance(ticker)

    # FINRA CSV 데이터 (일별 거래량 비율)
    finra_data = get_finra_short_interest(ticker)

    # 데이터 통합
    combined_data = {
        'ticker': ticker,
        'short_ratio_days': 0,
        'short_percent_float': 0,
        'shares_short_millions': 0,
        'short_change_pct': 0,
        'daily_short_ratio': 0,
        'avg_daily_short_ratio_10d': 0,
        'finra_latest_date': 'N/A',
        'data_source': []
    }

    if yf_data:
        combined_data.update({
            'short_ratio_days': round(yf_data.get('short_ratio', 0), 2),
            'short_percent_float': round(yf_data.get('short_percent_float', 0), 2),
            'shares_short_millions': round(yf_data.get('shares_short', 0) / 1e6, 2),
            'short_change_pct': round(yf_data.get('short_change_pct', 0), 2),
        })
        combined_data['data_source'].append('Yahoo Finance')

    if finra_data:
        # FINRA 일별 데이터 추가
        if 'daily_short_ratio' in finra_data:
            combined_data['daily_short_ratio'] = finra_data['daily_short_ratio']
            combined_data['avg_daily_short_ratio_10d'] = finra_data['avg_daily_short_ratio']
            combined_data['finra_latest_date'] = finra_data.get('daily_data_date', 'N/A')
            combined_data['data_source'].append(f"FINRA Daily ({finra_data.get('daily_data_points', 0)}일)")

        # FINRA 월별 데이터로 Days to Cover 업데이트
        if 'days_to_cover' in finra_data and finra_data['days_to_cover'] > 0:
            combined_data['short_ratio_days'] = round(finra_data['days_to_cover'], 2)
            combined_data['data_source'].append('FINRA Monthly')

    # 데이터 소스를 문자열로 변환
    combined_data['data_source'] = ' + '.join(combined_data['data_source']) if combined_data['data_source'] else 'N/A'

    return combined_data


# ==================== Magnificent Seven + Crypto/Bitcoin Exposure 정의 ====================

MAG7_STOCKS = {
    'AAPL': {
        'name': 'Apple Inc.',
        'description': '아이폰, 생태계, 온디바이스 AI',
        'sector': 'Technology',
        'industry': 'Consumer Electronics'
    },
    'MSFT': {
        'name': 'Microsoft Corporation',
        'description': '클라우드(Azure), 생성형 AI (OpenAI 대주주)',
        'sector': 'Technology',
        'industry': 'Software'
    },
    'GOOGL': {
        'name': 'Alphabet Inc.',
        'description': '구글 검색, 유튜브, AI (Gemini)',
        'sector': 'Communication Services',
        'industry': 'Internet Content & Information'
    },
    'AMZN': {
        'name': 'Amazon.com Inc.',
        'description': '전자상거래, 클라우드(AWS) 1위',
        'sector': 'Consumer Cyclical',
        'industry': 'Internet Retail'
    },
    'NVDA': {
        'name': 'NVIDIA Corporation',
        'description': 'AI 반도체(GPU) 독점적 지배자',
        'sector': 'Technology',
        'industry': 'Semiconductors'
    },
    'META': {
        'name': 'Meta Platforms Inc.',
        'description': '페이스북, 인스타그램, AI(Llama)',
        'sector': 'Communication Services',
        'industry': 'Internet Content & Information'
    },
    'TSLA': {
        'name': 'Tesla Inc.',
        'description': '전기차, 자율주행, 로봇',
        'sector': 'Consumer Cyclical',
        'industry': 'Auto Manufacturers'
    },
    'COIN': {
        'name': 'Coinbase Global Inc.',
        'description': '미국 최대 암호화폐 거래소, 비트코인 직접 노출',
        'sector': 'Financial Services',
        'industry': 'Cryptocurrency Exchange'
    },
    'IBIT': {
        'name': 'iShares Bitcoin Trust ETF',
        'description': 'BlackRock 비트코인 현물 ETF, 순수 BTC 노출',
        'sector': 'ETF',
        'industry': 'Bitcoin Spot ETF'
    }
}

print("=" * 80)
print("🌟 MAGNIFICENT SEVEN + BITCOIN EXPOSURE (MAG 7 + 2) 실시간 종합 분석 + 공매도 데이터")
print("=" * 80)
print(f"\n📅 데이터 수집 일시: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🌍 시간대: UTC+9 (한국 시간)")
print("\n빅테크 7대 기업 + 비트코인 관련 2종목:")
print("\n[Magnificent Seven]")
for ticker, info in list(MAG7_STOCKS.items())[:7]:
    print(f"  • {ticker:6} - {info['name']:30} | {info['description']}")
print("\n[Bitcoin Exposure]")
for ticker, info in list(MAG7_STOCKS.items())[7:]:
    print(f"  • {ticker:6} - {info['name']:30} | {info['description']}")
print("=" * 80)


def get_current_quarter_start():
    """현재 분기 시작일 계산"""
    now = datetime.now()
    quarter = (now.month - 1) // 3
    quarter_start_month = quarter * 3 + 1
    quarter_start = datetime(now.year, quarter_start_month, 1)
    return quarter_start


def calculate_anchored_vwap(df):
    """Anchored VWAP 계산"""
    df = df.copy()
    df['Typical_Price'] = (df['High'] + df['Low'] + df['Close']) / 3
    df['TP_Volume'] = df['Typical_Price'] * df['Volume']
    df['Cumulative_TP_Volume'] = df['TP_Volume'].cumsum()
    df['Cumulative_Volume'] = df['Volume'].cumsum()
    df['Anchored_VWAP'] = df['Cumulative_TP_Volume'] / df['Cumulative_Volume']
    return df


def get_quarterly_vwap_analysis(ticker):
    """분기별 Anchored VWAP 분석"""
    try:
        quarter_start = get_current_quarter_start()
        end_date = datetime.now()
        quarter_num = (quarter_start.month - 1) // 3 + 1

        stock = yf.Ticker(ticker)
        df = stock.history(start=quarter_start, end=end_date)

        if df.empty or len(df) < 5:
            return None

        df = calculate_anchored_vwap(df)

        current_price = df['Close'].iloc[-1]
        current_vwap = df['Anchored_VWAP'].iloc[-1]
        above_vwap_ratio = (df['Close'] > df['Anchored_VWAP']).sum() / len(df) * 100
        recent_5days_avg = df['Close'].tail(5).mean()
        recent_10days_avg = df['Close'].tail(10).mean()

        recent_20 = df['Close'].tail(min(20, len(df)))
        uptrend_strength = (recent_20.diff() > 0).sum() / len(recent_20) * 100 if len(recent_20) > 1 else 50

        recent_volume = df['Volume'].tail(5).mean()
        avg_volume = df['Volume'].mean()
        volume_ratio = recent_volume / avg_volume if avg_volume > 0 else 1

        info = stock.info
        quarter_start_price = df['Close'].iloc[0]
        quarter_return = ((current_price - quarter_start_price) / quarter_start_price * 100)

        return {
            'Ticker': ticker,
            'Company': MAG7_STOCKS[ticker]['name'],
            'Description': MAG7_STOCKS[ticker]['description'],
            'Sector': MAG7_STOCKS[ticker]['sector'],
            'Quarter': f'{quarter_start.year} Q{quarter_num}',
            'Quarter_Start_Date': quarter_start.strftime('%Y-%m-%d'),
            'Trading_Days': len(df),
            'Current_Price': round(current_price, 2),
            'Anchored_VWAP': round(current_vwap, 2),
            'Quarter_Start_Price': round(quarter_start_price, 2),
            'Quarter_Return_%': round(quarter_return, 2),
            'Price_vs_VWAP_%': round((current_price - current_vwap) / current_vwap * 100, 2),
            'Above_VWAP_Days_%': round(above_vwap_ratio, 1),
            'Recent_5D_Avg': round(recent_5days_avg, 2),
            'Recent_10D_Avg': round(recent_10days_avg, 2),
            'Uptrend_Strength_%': round(uptrend_strength, 1),
            'Volume_Ratio': round(volume_ratio, 2),
            'Is_Above_VWAP': current_price > current_vwap,
            'Market_Cap': info.get('marketCap', 0),
            'Buy_Signal_Score': 0
        }

    except Exception as e:
        print(f"Error processing {ticker}: {str(e)}")
        return None


# ==================== 메인 분석 시작 ====================

quarter_start = get_current_quarter_start()
quarter_num = (quarter_start.month - 1) // 3 + 1

print("\n" + "=" * 80)
print(f"🎯 Anchored VWAP 분석: {quarter_start.year} Q{quarter_num}")
print(f"📍 Anchor Point: {quarter_start.strftime('%Y-%m-%d')}")
print(f"📅 현재까지 거래일: {(datetime.now() - quarter_start).days}일")
print("=" * 80)

# MAG 7 + 2 분석
results = []
mag7_tickers = list(MAG7_STOCKS.keys())

print("\n🔍 MAG 7 + Bitcoin Exposure 종목 분석 중...\n")

for ticker in mag7_tickers:
    print(f"분석 중: {ticker} ({MAG7_STOCKS[ticker]['name']})...", end=" ")
    result = get_quarterly_vwap_analysis(ticker)
    if result:
        results.append(result)
        print("✓")
    else:
        print("✗")

df_results = pd.DataFrame(results)

# 매수 신호 점수 계산
def calculate_buy_score(row):
    score = 0
    if row['Is_Above_VWAP']:
        score += 30

    price_diff = row['Price_vs_VWAP_%']
    if 0 < price_diff <= 5:
        score += 20
    elif 5 < price_diff <= 10:
        score += 10
    elif price_diff > 10:
        score += 5

    if row['Above_VWAP_Days_%'] >= 80:
        score += 20
    elif row['Above_VWAP_Days_%'] >= 60:
        score += 15
    elif row['Above_VWAP_Days_%'] >= 40:
        score += 10

    if row['Uptrend_Strength_%'] >= 60:
        score += 15
    elif row['Uptrend_Strength_%'] >= 50:
        score += 10

    if row['Volume_Ratio'] >= 1.2:
        score += 15
    elif row['Volume_Ratio'] >= 1.0:
        score += 10

    return min(score, 100)

df_results['Buy_Signal_Score'] = df_results.apply(calculate_buy_score, axis=1)

# 시가총액 추가
df_results['Market_Cap_Trillion'] = (df_results['Market_Cap'] / 1e12).round(3)

# ==================== 공매도 데이터 수집 ====================

print("\n" + "=" * 80)
print("📊 공매도(Short Interest) 데이터 수집 중...")
print("=" * 80)

short_data_list = []

for ticker in mag7_tickers:
    print(f"  📊 {ticker} 공매도 데이터 수집...", end=" ")
    short_data = get_comprehensive_short_data(ticker)
    if short_data:
        short_data_list.append(short_data)
        print("✓")
    else:
        print("✗")

df_short = pd.DataFrame(short_data_list)

# 공매도 데이터를 메인 데이터프레임에 병합
df_results = df_results.merge(df_short, left_on='Ticker', right_on='ticker', how='left')

# 공매도 스코어 계산 (낮을수록 좋음)
def calculate_short_score(row):
    """
    공매도 스코어: 공매도 비율이 낮을수록 높은 점수
    - 5% 미만: 매우 좋음 (20점)
    - 5-10%: 양호 (15점)
    - 10-20%: 보통 (10점)
    - 20% 이상: 주의 (5점)
    """
    short_pct = row.get('short_percent_float', 0)

    if short_pct < 5:
        return 20
    elif short_pct < 10:
        return 15
    elif short_pct < 20:
        return 10
    else:
        return 5

df_results['Short_Score'] = df_results.apply(calculate_short_score, axis=1)

# 종합 투자 점수 (기존 Buy Signal + Short Score)
df_results['Total_Investment_Score'] = df_results['Buy_Signal_Score'] + df_results['Short_Score']
df_results = df_results.sort_values('Total_Investment_Score', ascending=False)

# ==================== 결과 출력 ====================

print("\n" + "=" * 80)
print("📊 MAGNIFICENT SEVEN + BITCOIN - 종합 분석 결과 (공매도 포함)")
print("=" * 80)

display_cols = ['Ticker', 'Company', 'Current_Price', 'Anchored_VWAP', 'Price_vs_VWAP_%',
                'Quarter_Return_%', 'short_percent_float', 'short_ratio_days',
                'daily_short_ratio', 'Buy_Signal_Score', 'Short_Score', 'Total_Investment_Score']

print(df_results[display_cols].to_string(index=False))

print("\n" + "=" * 80)
print("🏆 MAG 7+2 투자 추천 순위 (공매도 분석 포함)")
print("=" * 80)

for idx, row in df_results.iterrows():
    rank = df_results.index.get_loc(idx) + 1

    if rank == 1:
        medal = "🥇"
    elif rank == 2:
        medal = "🥈"
    elif rank == 3:
        medal = "🥉"
    else:
        medal = f"  {rank}"

    print(f"\n{medal} {row['Ticker']} - {row['Company']}")
    print(f"    🎯 {row['Description']}")
    print(f"    💰 시가총액: ${row['Market_Cap_Trillion']:.2f}T")
    print(f"    📈 현재가: ${row['Current_Price']} | VWAP: ${row['Anchored_VWAP']}")
    print(f"    📊 VWAP 대비: {row['Price_vs_VWAP_%']:+.2f}% | 분기 수익률: {row['Quarter_Return_%']:+.2f}%")
    print(f"    🎲 VWAP 위 거래일: {row['Above_VWAP_Days_%']}% | 추세 강도: {row['Uptrend_Strength_%']}%")

    # 공매도 정보
    short_pct = row.get('short_percent_float', 0)
    short_days = row.get('short_ratio_days', 0)
    daily_short = row.get('daily_short_ratio', 0)

    print(f"    🔴 공매도 비율: {short_pct:.2f}% of float | 커버 소요일: {short_days:.1f}일")

    if daily_short > 0:
        print(f"    📊 최근 일평균 공매도 거래비율: {daily_short:.1f}%")

    if short_pct < 5:
        short_status = "✅ 매우 낮음 (긍정)"
    elif short_pct < 10:
        short_status = "🟢 양호"
    elif short_pct < 20:
        short_status = "🟡 보통"
    else:
        short_status = "🔴 높음 (주의)"

    print(f"    🎯 공매도 상태: {short_status}")

    print(f"    ⭐ 기술적 점수: {row['Buy_Signal_Score']}/100 | 공매도 점수: {row['Short_Score']}/20", end="")
    print(f" | 종합: {row['Total_Investment_Score']}/120")

    if row['Total_Investment_Score'] >= 90:
        print("       → 💚 최우선 매수 추천")
    elif row['Total_Investment_Score'] >= 75:
        print("       → 💛 강력 매수")
    elif row['Total_Investment_Score'] >= 60:
        print("       → 💙 눌림목 대기")
    else:
        print("       → ⚪ 관찰 필요")

# ==================== Yahoo Finance vs FINRA 비교 테이블 ====================

print("\n" + "=" * 80)
print("📊 Yahoo Finance vs FINRA 공매도 데이터 상세 비교")
print("=" * 80)

comparison_data = []

for idx, row in df_results.iterrows():
    ticker = row['Ticker']

    comparison_data.append({
        'Ticker': ticker,
        'Company': row['Company'][:25],

        # Yahoo Finance 데이터
        'YF_Short_%_Float': f"{row.get('short_percent_float', 0):.2f}%",
        'YF_Days_to_Cover': f"{row.get('short_ratio_days', 0):.2f}",
        'YF_Shares_Short_M': f"{row.get('shares_short_millions', 0):.1f}M",
        'YF_Change_MoM': f"{row.get('short_change_pct', 0):+.1f}%",

        # FINRA 데이터
        'FINRA_Daily_Short_%': f"{row.get('daily_short_ratio', 0):.1f}%" if row.get('daily_short_ratio', 0) > 0 else "N/A",
        'FINRA_10D_Avg_%': f"{row.get('avg_daily_short_ratio_10d', 0):.1f}%" if row.get('avg_daily_short_ratio_10d', 0) > 0 else "N/A",
        'FINRA_Data_Date': row.get('finra_latest_date', 'N/A'),

        # 데이터 소스
        'Data_Source': row.get('data_source', 'N/A')
    })

df_comparison = pd.DataFrame(comparison_data)

print("\n📋 상세 비교표:")
print("=" * 150)

# 컬럼 순서 및 너비 조정을 위한 포맷팅
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 150)
pd.set_option('display.max_colwidth', 30)

print(df_comparison.to_string(index=False))
print("=" * 150)

print("\n" + "=" * 120)
print("📖 컬럼 설명:")
print("=" * 120)
print("""
[Yahoo Finance 데이터 - 공매도 잔고 (Short Interest)]
  • YF_Short_%_Float     : 유통주식(Float) 대비 공매도 비율
                           → 낮을수록 좋음 (일반적으로 5% 미만이 건강)

  • YF_Days_to_Cover     : 공매도 청산 소요일 (Short Interest / Avg Daily Volume)
                           → 3일 이상이면 Short Squeeze 가능성

  • YF_Shares_Short_M    : 현재 공매도된 총 주식 수 (백만 주)

  • YF_Change_MoM        : 전월 대비 공매도 변화율
                           → (+)는 공매도 증가 (약세 신호)
                           → (-)는 공매도 감소 (강세 신호)

[FINRA 데이터 - 일별 공매도 거래량 (Daily Short Volume)]
  • FINRA_Daily_Short_%  : 최근 거래일의 공매도 거래량 비율
                           → 해당일 전체 거래량 중 공매도 거래 비중

  • FINRA_10D_Avg_%      : 최근 10거래일 평균 공매도 거래 비율
                           → 30-40%는 정상 (마켓메이킹 포함)
                           → 50% 이상은 강한 약세 압력

  • FINRA_Data_Date      : FINRA 데이터의 최근 수집 날짜
                           → 실제 공매도 거래 발생일

[중요한 차이점]
  ✓ Yahoo Finance : 공매도 "잔고" (누적된 공매도 포지션) - 월 2회 업데이트
  ✓ FINRA Daily   : 공매도 "거래량" (해당일의 신규 공매도 거래) - 매일 업데이트

  → 두 지표는 다른 의미이므로 함께 분석해야 정확!
""")

# ==================== 데이터 품질 분석 ====================

print("\n" + "=" * 80)
print("🔍 데이터 품질 및 신뢰도 분석")
print("=" * 80)

quality_analysis = []

for idx, row in df_results.iterrows():
    ticker = row['Ticker']

    # 데이터 완성도 평가
    yf_complete = (
        row.get('short_percent_float', 0) > 0 and
        row.get('shares_short_millions', 0) > 0
    )

    finra_complete = row.get('daily_short_ratio', 0) > 0

    data_quality = "⭐⭐⭐" if (yf_complete and finra_complete) else \
                   "⭐⭐" if (yf_complete or finra_complete) else "⭐"

    # 공매도 신호 강도
    short_pct = row.get('short_percent_float', 0)
    daily_short = row.get('daily_short_ratio', 0)

    if short_pct < 3 and daily_short < 40:
        signal = "💚 매우 긍정"
    elif short_pct < 5 and daily_short < 45:
        signal = "🟢 긍정"
    elif short_pct < 10 and daily_short < 50:
        signal = "🟡 중립"
    else:
        signal = "🔴 약세 압력"

    quality_analysis.append({
        'Ticker': ticker,
        'YF_Complete': '✓' if yf_complete else '✗',
        'FINRA_Complete': '✓' if finra_complete else '✗',
        'Data_Quality': data_quality,
        'Short_Signal': signal,
        'Interpretation': f"잔고 {short_pct:.1f}% / 일거래 {daily_short:.0f}%"
    })

df_quality = pd.DataFrame(quality_analysis)

print("\n📊 종목별 데이터 품질 및 공매도 신호:")
print("=" * 100)
print(df_quality.to_string(index=False))
print("=" * 100)

print("\n" + "=" * 80)
print("💡 통합 해석 가이드")
print("=" * 80)
print("""
[최적 투자 신호] 💚
  ✓ Yahoo: Short % < 3% + FINRA: Daily < 40%
  → 공매도 잔고도 낮고, 일일 공매도 거래도 적음
  → 시장의 강한 신뢰, 매수 추천

[양호한 신호] 🟢
  ✓ Yahoo: Short % < 5% + FINRA: Daily < 45%
  → 전반적으로 건강한 수준
  → 적극 매수 고려

[주의 필요] 🟡
  ✓ Yahoo: Short % 5-10% + FINRA: Daily 45-50%
  → 약간의 약세 포지션 존재
  → 기술적 분석 병행 필수

[약세 압력] 🔴
  ✓ Yahoo: Short % > 10% 또는 FINRA: Daily > 50%
  → 강한 공매도 압력
  → Short Squeeze 가능성은 있으나 위험도 높음
""")

# ==================== 공매도 관련 차트 ====================

print("\n📈 공매도 분석 차트 생성 중...")

# 차트 0: Yahoo Finance vs FINRA 비교 차트
fig_compare = make_subplots(
    rows=2, cols=1,
    subplot_titles=(
        'Yahoo Finance: Short % of Float (공매도 잔고)',
        'FINRA: Daily Short Volume Ratio (일일 공매도 거래 비율)'
    ),
    vertical_spacing=0.15,
    specs=[[{"type": "bar"}], [{"type": "bar"}]]
)

# Yahoo Finance 공매도 비율
fig_compare.add_trace(
    go.Bar(
        x=df_results['Ticker'],
        y=df_results['short_percent_float'],
        name='YF Short %',
        marker=dict(
            color=df_results['short_percent_float'],
            colorscale='RdYlGn_r',
            showscale=False
        ),
        text=df_results['short_percent_float'].round(2),
        textposition='auto',
        hovertemplate='<b>%{x}</b><br>공매도 잔고: %{y:.2f}%<extra></extra>'
    ),
    row=1, col=1
)

# FINRA 일일 공매도 비율
fig_compare.add_trace(
    go.Bar(
        x=df_results['Ticker'],
        y=df_results['daily_short_ratio'],
        name='FINRA Daily %',
        marker=dict(
            color=df_results['daily_short_ratio'],
            colorscale='Blues',
            showscale=False
        ),
        text=df_results['daily_short_ratio'].round(1),
        textposition='auto',
        hovertemplate='<b>%{x}</b><br>일일 공매도: %{y:.1f}%<extra></extra>'
    ),
    row=2, col=1
)

# 참고선 추가
fig_compare.add_hline(y=5, line_dash="dash", line_color="green", row=1, col=1,
                      annotation_text="건강 기준선 (5%)", annotation_position="right")
fig_compare.add_hline(y=40, line_dash="dash", line_color="orange", row=2, col=1,
                      annotation_text="정상 범위 (40%)", annotation_position="right")

fig_compare.update_xaxes(title_text="종목", row=1, col=1)
fig_compare.update_xaxes(title_text="종목", row=2, col=1)
fig_compare.update_yaxes(title_text="Short % of Float", row=1, col=1)
fig_compare.update_yaxes(title_text="Daily Short %", row=2, col=1)

fig_compare.update_layout(
    title_text='📊 Yahoo Finance vs FINRA 공매도 데이터 비교',
    height=800,
    showlegend=False,
    template='plotly_white'
)

fig_compare.show()

# 차트 0-2: 산점도로 두 지표 관계 분석
fig_scatter_short = px.scatter(
    df_results,
    x='short_percent_float',
    y='daily_short_ratio',
    size='Market_Cap_Trillion',
    color='Total_Investment_Score',
    hover_data=['Ticker', 'Company'],
    text='Ticker',
    color_continuous_scale='RdYlGn',
    title='📊 공매도 잔고 vs 일일 공매도 거래 비율 관계',
    labels={
        'short_percent_float': 'Yahoo Finance: Short % of Float',
        'daily_short_ratio': 'FINRA: Daily Short Volume %',
        'Total_Investment_Score': '종합 점수'
    }
)

# 참고선 추가
fig_scatter_short.add_hline(y=40, line_dash="dash", line_color="gray",
                            annotation_text="FINRA 정상 기준", annotation_position="right")
fig_scatter_short.add_vline(x=5, line_dash="dash", line_color="gray",
                            annotation_text="YF 건강 기준", annotation_position="top")

fig_scatter_short.update_traces(textposition='top center', textfont_size=12)
fig_scatter_short.update_layout(height=600)
fig_scatter_short.show()

# ==================== 시계열 차트 섹션 ====================

print("\n" + "=" * 80)
print("📈 공매도 시계열 데이터 수집 중 (최근 60일)...")
print("=" * 80)

# 60일 데이터 수집
timeseries_data = {}

for ticker in mag7_tickers:
    print(f"  📊 {ticker} 시계열 수집...", end=" ")
    extended_data = get_extended_finra_history(ticker, days_back=60)

    if extended_data and 'historical_data' in extended_data:
        timeseries_data[ticker] = extended_data['historical_data']
        print(f"✓ ({len(extended_data['historical_data'])}일)")
    else:
        print("✗")

# 차트 A: 전체 MAG 7 공매도 비율 추세 비교
if timeseries_data:
    print("\n📊 시계열 차트 생성 중...")

    fig_ts_all = go.Figure()

    colors_ts = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8', '#F7DC6F', '#BB8FCE', '#E74C3C', '#3498DB']

    for idx, (ticker, df_ts) in enumerate(timeseries_data.items()):
        if df_ts is not None and not df_ts.empty:
            # 날짜순 정렬 (오래된 것부터)
            df_ts_sorted = df_ts.sort_values('date')

            fig_ts_all.add_trace(go.Scatter(
                x=pd.to_datetime(df_ts_sorted['date']),
                y=df_ts_sorted['short_ratio'],
                mode='lines+markers',
                name=ticker,
                line=dict(width=2.5, color=colors_ts[idx % len(colors_ts)]),
                marker=dict(size=6),
                hovertemplate='<b>%{fullData.name}</b><br>날짜: %{x|%Y-%m-%d}<br>공매도 비율: %{y:.1f}%<extra></extra>'
            ))

    # 참고선 추가
    fig_ts_all.add_hline(y=40, line_dash="dash", line_color="gray",
                         annotation_text="정상 범위 (40%)", annotation_position="right")
    fig_ts_all.add_hline(y=50, line_dash="dash", line_color="red",
                         annotation_text="약세 압력 (50%)", annotation_position="right")

    fig_ts_all.update_layout(
        title='📈 MAG 7+2 - FINRA 일별 공매도 거래 비율 추세 (최근 60일)',
        xaxis_title='날짜',
        yaxis_title='공매도 거래 비율 (%)',
        hovermode='x unified',
        height=600,
        template='plotly_white',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )

    fig_ts_all.show()

    # 차트 B: 개별 종목 상세 시계열 (서브플롯)
    print("\n📊 개별 종목 시계열 차트 생성 중...")

    # 3x3 그리드로 배치 (9개 종목)
    fig_ts_individual = make_subplots(
        rows=3, cols=3,
        subplot_titles=[ticker for ticker in mag7_tickers],
        vertical_spacing=0.10,
        horizontal_spacing=0.08
    )

    for idx, ticker in enumerate(mag7_tickers):
        row = idx // 3 + 1
        col = idx % 3 + 1

        if ticker in timeseries_data:
            df_ts = timeseries_data[ticker]
            df_ts_sorted = df_ts.sort_values('date')

            # 공매도 비율 라인
            fig_ts_individual.add_trace(
                go.Scatter(
                    x=pd.to_datetime(df_ts_sorted['date']),
                    y=df_ts_sorted['short_ratio'],
                    mode='lines',
                    name=ticker,
                    line=dict(width=2, color=colors_ts[idx % len(colors_ts)]),
                    fill='tozeroy',
                    fillcolor=f'rgba{tuple(list(int(colors_ts[idx % len(colors_ts)][i:i+2], 16) for i in (1, 3, 5)) + [0.2])}',
                    showlegend=False,
                    hovertemplate='%{y:.1f}%<extra></extra>'
                ),
                row=row, col=col
            )

            # 이동평균선 추가 (7일)
            if len(df_ts_sorted) >= 7:
                ma7 = df_ts_sorted['short_ratio'].rolling(window=7).mean()
                fig_ts_individual.add_trace(
                    go.Scatter(
                        x=pd.to_datetime(df_ts_sorted['date']),
                        y=ma7,
                        mode='lines',
                        name=f'{ticker} MA7',
                        line=dict(width=1.5, color='red', dash='dash'),
                        showlegend=False,
                        hovertemplate='MA7: %{y:.1f}%<extra></extra>'
                    ),
                    row=row, col=col
                )

    fig_ts_individual.update_xaxes(title_text="날짜", row=3)
    fig_ts_individual.update_yaxes(title_text="공매도 비율 (%)")

    fig_ts_individual.update_layout(
        title_text='📊 MAG 7+2 개별 종목 - 일별 공매도 비율 추세 (with 7일 이동평균)',
        height=1000,
        template='plotly_white',
        showlegend=False
    )

    fig_ts_individual.show()

    # 차트 C: 거래량과 공매도 비율 관계
    print("\n📊 거래량 vs 공매도 비율 차트 생성 중...")

    # 데이터 통합
    all_ts_data = []
    for ticker, df_ts in timeseries_data.items():
        if df_ts is not None and not df_ts.empty:
            df_temp = df_ts.copy()
            df_temp['ticker'] = ticker
            all_ts_data.append(df_temp)

    if all_ts_data:
        df_all_ts = pd.concat(all_ts_data, ignore_index=True)
        df_all_ts['date'] = pd.to_datetime(df_all_ts['date'])

        # 최근 30일 데이터만 사용
        recent_date = df_all_ts['date'].max() - timedelta(days=30)
        df_recent = df_all_ts[df_all_ts['date'] >= recent_date]

        fig_vol_short = px.scatter(
            df_recent,
            x='total_volume',
            y='short_ratio',
            color='ticker',
            size='short_volume',
            hover_data=['date'],
            title='📊 거래량 vs 공매도 비율 관계 (최근 30일)',
            labels={
                'total_volume': '전체 거래량',
                'short_ratio': '공매도 비율 (%)',
                'ticker': '종목'
            },
            color_discrete_sequence=colors_ts
        )

        fig_vol_short.update_layout(height=600, template='plotly_white')
        fig_vol_short.show()

    # 차트 D: 공매도 변동성 분석
    print("\n📊 공매도 변동성 분석 차트 생성 중...")

    volatility_data = []

    for ticker, df_ts in timeseries_data.items():
        if df_ts is not None and not df_ts.empty and len(df_ts) > 1:
            short_ratios = df_ts['short_ratio'].values

            volatility_data.append({
                'Ticker': ticker,
                'Avg_Short_Ratio': df_ts['short_ratio'].mean(),
                'Std_Dev': df_ts['short_ratio'].std(),
                'Min': df_ts['short_ratio'].min(),
                'Max': df_ts['short_ratio'].max(),
                'Range': df_ts['short_ratio'].max() - df_ts['short_ratio'].min()
            })

    df_volatility = pd.DataFrame(volatility_data)

    fig_volatility = go.Figure()

    # 박스 플롯
    for idx, ticker in enumerate(mag7_tickers):
        if ticker in timeseries_data:
            df_ts = timeseries_data[ticker]

            fig_volatility.add_trace(go.Box(
                y=df_ts['short_ratio'],
                name=ticker,
                marker_color=colors_ts[idx % len(colors_ts)],
                boxmean='sd'  # 평균과 표준편차 표시
            ))

    fig_volatility.update_layout(
        title='📊 MAG 7+2 - 공매도 비율 변동성 분석 (Box Plot)',
        yaxis_title='공매도 비율 (%)',
        xaxis_title='종목',
        height=600,
        template='plotly_white',
        showlegend=False
    )

    fig_volatility.show()

    # 변동성 통계 테이블
    print("\n" + "=" * 80)
    print("📊 공매도 변동성 통계")
    print("=" * 80)
    print("\n")
    print(df_volatility.round(2).to_string(index=False))
    print("\n💡 해석:")
    print("  • Std_Dev (표준편차): 높을수록 변동성이 큼")
    print("  • Range (범위): 최대-최소 차이, 높을수록 불안정")
    print("  • 변동성이 낮고 평균이 40% 미만이면 안정적")

else:
    print("\n⚠️ 시계열 데이터를 충분히 수집하지 못했습니다.")

# 차트 1: 공매도 비율 비교
fig_short = go.Figure()

colors_short = ['#00C853' if x < 5 else '#FFD600' if x < 10 else '#FF5722'
                for x in df_results['short_percent_float']]

fig_short.add_trace(go.Bar(
    y=df_results['Ticker'],
    x=df_results['short_percent_float'],
    orientation='h',
    marker=dict(color=colors_short),
    text=df_results['short_percent_float'].round(2),
    textposition='auto',
    hovertemplate='<b>%{y}</b><br>공매도 비율: %{x:.2f}%<extra></extra>'
))

fig_short.update_layout(
    title='🔴 MAG 7 - 공매도 비율 (Short % of Float)',
    xaxis_title='공매도 비율 (%)',
    yaxis_title='종목',
    height=500,
    template='plotly_white'
)

# 참고선 추가
fig_short.add_vline(x=5, line_dash="dash", line_color="green",
                    annotation_text="낮음", annotation_position="top")
fig_short.add_vline(x=10, line_dash="dash", line_color="orange",
                    annotation_text="보통", annotation_position="top")
fig_short.add_vline(x=20, line_dash="dash", line_color="red",
                    annotation_text="높음", annotation_position="top")

fig_short.show()

# 차트 2: 종합 투자 점수 (기술적 + 공매도)
fig_total = make_subplots(
    rows=1, cols=2,
    subplot_titles=('기술적 분석 점수', '종합 투자 점수'),
    specs=[[{"type": "bar"}, {"type": "bar"}]]
)

fig_total.add_trace(
    go.Bar(
        y=df_results['Ticker'],
        x=df_results['Buy_Signal_Score'],
        orientation='h',
        name='기술적 점수',
        marker_color='#2196F3',
        text=df_results['Buy_Signal_Score'],
        textposition='auto'
    ),
    row=1, col=1
)

fig_total.add_trace(
    go.Bar(
        y=df_results['Ticker'],
        x=df_results['Total_Investment_Score'],
        orientation='h',
        name='종합 점수',
        marker_color='#4CAF50',
        text=df_results['Total_Investment_Score'],
        textposition='auto'
    ),
    row=1, col=2
)

fig_total.update_xaxes(title_text="점수", row=1, col=1)
fig_total.update_xaxes(title_text="점수", row=1, col=2)
fig_total.update_yaxes(title_text="종목", row=1, col=1)

fig_total.update_layout(
    title_text='📊 MAG 7 - 투자 점수 비교 (기술적 vs 종합)',
    height=500,
    showlegend=False,
    template='plotly_white'
)

fig_total.show()

# 차트 3: 공매도 vs 분기 수익률
fig_scatter = px.scatter(
    df_results,
    x='short_percent_float',
    y='Quarter_Return_%',
    size='Market_Cap_Trillion',
    color='Total_Investment_Score',
    hover_data=['Ticker', 'Company'],
    text='Ticker',
    color_continuous_scale='RdYlGn',
    title='📊 MAG 7 - 공매도 비율 vs 분기 수익률',
    labels={
        'short_percent_float': '공매도 비율 (%)',
        'Quarter_Return_%': '분기 수익률 (%)',
        'Total_Investment_Score': '종합 점수'
    }
)

fig_scatter.update_traces(textposition='top center', textfont_size=12)
fig_scatter.update_layout(height=600)
fig_scatter.show()

# ==================== 최종 요약 ====================

print("\n" + "=" * 80)
print("📈 MAG 7+2 투자 전략 최종 요약 (공매도 분석 포함)")
print("=" * 80)

print(f"\n📍 분석 기준: Anchored VWAP ({quarter_start.year} Q{quarter_num})")
print(f"📅 Anchor Point: {quarter_start.strftime('%Y-%m-%d')}")
print(f"🌟 분석 대상: MAGNIFICENT SEVEN + BITCOIN EXPOSURE (9개 종목)")

# ==================== 최종 요약 비교표 ====================

print("\n" + "=" * 150)
print("📊 MAG 7+2 종합 요약표 (기술적 분석 + 공매도)")
print("=" * 150)

summary_table = []

for idx, row in df_results.iterrows():
    rank = df_results.index.get_loc(idx) + 1

    summary_table.append({
        'Rank': rank,
        'Ticker': row['Ticker'],
        'Price': f"${row['Current_Price']:.2f}",
        'VWAP_Diff': f"{row['Price_vs_VWAP_%']:+.1f}%",
        'Q_Return': f"{row['Quarter_Return_%']:+.1f}%",
        'YF_Short%': f"{row.get('short_percent_float', 0):.2f}%",
        'Days_Cover': f"{row.get('short_ratio_days', 0):.1f}",
        'FINRA_Daily%': f"{row.get('daily_short_ratio', 0):.1f}%" if row.get('daily_short_ratio', 0) > 0 else "N/A",
        'Tech_Score': f"{row['Buy_Signal_Score']}/100",
        'Total_Score': f"{row['Total_Investment_Score']}/120",
        'Signal': '💚' if row['Total_Investment_Score'] >= 90 else '💛' if row['Total_Investment_Score'] >= 75 else '💙'
    })

df_summary = pd.DataFrame(summary_table)
print("\n")
print(df_summary.to_string(index=False))
print("=" * 150)

print("\n📖 컬럼 설명:")
print("  • VWAP_Diff: 현재가의 VWAP 대비 위치")
print("  • Q_Return: 현재 분기 수익률")
print("  • YF_Short%: Yahoo Finance 공매도 잔고 비율")
print("  • Days_Cover: 공매도 청산 소요일")
print("  • FINRA_Daily%: FINRA 일일 공매도 거래 비율")
print("  • Tech_Score: 기술적 분석 점수 (VWAP 기반)")
print("  • Total_Score: 종합 점수 (기술적 + 공매도)")
print("  • Signal: 💚=최우선 매수, 💛=강력 매수, 💙=관찰")

# 공매도 분석 요약
low_short = df_results[df_results['short_percent_float'] < 5]
print(f"\n✅ 낮은 공매도 (<5%): {len(low_short)}개")
if not low_short.empty:
    print(f"   → {', '.join(low_short['Ticker'].tolist())}")

high_short = df_results[df_results['short_percent_float'] >= 10]
print(f"⚠️ 높은 공매도 (≥10%): {len(high_short)}개")
if not high_short.empty:
    print(f"   → {', '.join(high_short['Ticker'].tolist())}")

# 종합 추천
top_score = df_results[df_results['Total_Investment_Score'] >= 90]
print(f"\n💚 최우선 매수 (90점↑): {len(top_score)}개")
if not top_score.empty:
    print(f"   → {', '.join(top_score['Ticker'].tolist())}")

strong_buy = df_results[(df_results['Total_Investment_Score'] >= 75) &
                        (df_results['Total_Investment_Score'] < 90)]
print(f"💛 강력 매수 (75-90점): {len(strong_buy)}개")
if not strong_buy.empty:
    print(f"   → {', '.join(strong_buy['Ticker'].tolist())}")

print("\n🏆 TOP 5 추천 (공매도 고려):")
for idx in range(min(5, len(df_results))):
    row = df_results.iloc[idx]
    print(f"  {idx+1}. {row['Ticker']} - {row['Company']}")
    print(f"     💰 시총: ${row['Market_Cap_Trillion']:.2f}T | " +
          f"종합점수: {row['Total_Investment_Score']:.0f}/120")
    print(f"     📊 VWAP: {row['Price_vs_VWAP_%']:+.2f}% | " +
          f"분기수익: {row['Quarter_Return_%']:+.2f}% | " +
          f"공매도: {row['short_percent_float']:.2f}%")

print("\n💡 투자 가이드 (공매도 포함):")
print("""
1. 💚 최우선 매수 (90점 이상):
   - VWAP 위 + 낮은 공매도 비율
   - 즉시 매수 검토, 강한 상승 모멘텀 예상

2. 💛 강력 매수 (75-90점):
   - 기술적으로 양호하나 공매도 약간 존재
   - VWAP 근처 조정 시 매수 기회

3. 💙 눌림목 대기 (60-75점):
   - 공매도 비율 확인 필수
   - VWAP 돌파 확인 후 매수

4. ⚠️ 공매도 주의사항:
   - 10% 이상: Short Squeeze 가능성 주의
   - 5% 미만: 시장의 신뢰 높음
   - Days to Cover 3일 이상: 변동성 증가 가능
""")

print("\n📊 공매도 데이터 출처 및 수집 날짜:")
print("=" * 80)
for _, row in df_results.iterrows():
    finra_date = row.get('finra_latest_date', 'N/A')
    data_source = row.get('data_source', 'N/A')

    print(f"  • {row['Ticker']:6}")
    print(f"    - 데이터 소스: {data_source}")
    if finra_date != 'N/A':
        print(f"    - FINRA 최근 데이터: {finra_date}")
    print()

print("\n" + "=" * 80)
print("✨ MAGNIFICENT SEVEN + BITCOIN EXPOSURE 종합 분석 완료!")
print("=" * 80)

🌟 MAGNIFICENT SEVEN + BITCOIN EXPOSURE (MAG 7 + 2) 실시간 종합 분석 + 공매도 데이터

📅 데이터 수집 일시: 2025-11-28 12:35:18
🌍 시간대: UTC+9 (한국 시간)

빅테크 7대 기업 + 비트코인 관련 2종목:

[Magnificent Seven]
  • AAPL   - Apple Inc.                     | 아이폰, 생태계, 온디바이스 AI
  • MSFT   - Microsoft Corporation          | 클라우드(Azure), 생성형 AI (OpenAI 대주주)
  • GOOGL  - Alphabet Inc.                  | 구글 검색, 유튜브, AI (Gemini)
  • AMZN   - Amazon.com Inc.                | 전자상거래, 클라우드(AWS) 1위
  • NVDA   - NVIDIA Corporation             | AI 반도체(GPU) 독점적 지배자
  • META   - Meta Platforms Inc.            | 페이스북, 인스타그램, AI(Llama)
  • TSLA   - Tesla Inc.                     | 전기차, 자율주행, 로봇

[Bitcoin Exposure]
  • COIN   - Coinbase Global Inc.           | 미국 최대 암호화폐 거래소, 비트코인 직접 노출
  • IBIT   - iShares Bitcoin Trust ETF      | BlackRock 비트코인 현물 ETF, 순수 BTC 노출

🎯 Anchored VWAP 분석: 2025 Q4
📍 Anchor Point: 2025-10-01
📅 현재까지 거래일: 58일

🔍 MAG 7 + Bitcoin Exposure 종목 분석 중...

분석 중: AAPL (Apple Inc.)... ✓
분석 중: MSFT (Microsoft Corporation)... ✓


📈 공매도 시계열 데이터 수집 중 (최근 60일)...
  📊 AAPL 시계열 수집... ✓ (42일)
  📊 MSFT 시계열 수집... ✓ (42일)
  📊 GOOGL 시계열 수집... ✓ (42일)
  📊 AMZN 시계열 수집... ✓ (42일)
  📊 NVDA 시계열 수집... ✓ (42일)
  📊 META 시계열 수집... ✓ (42일)
  📊 TSLA 시계열 수집... ✓ (42일)
  📊 COIN 시계열 수집... ✓ (42일)
  📊 IBIT 시계열 수집... ✓ (42일)

📊 시계열 차트 생성 중...



📊 개별 종목 시계열 차트 생성 중...



📊 거래량 vs 공매도 비율 차트 생성 중...



📊 공매도 변동성 분석 차트 생성 중...



📊 공매도 변동성 통계


Ticker  Avg_Short_Ratio  Std_Dev   Min   Max  Range
  AAPL            42.60     7.84 28.26 58.85  30.59
  MSFT            34.38     7.95 22.22 59.92  37.70
 GOOGL            31.96     8.55 19.25 46.35  27.10
  AMZN            33.83     5.44 23.86 46.85  22.99
  NVDA            39.43     5.14 31.04 52.19  21.15
  META            36.89     6.09 28.69 54.35  25.66
  TSLA            47.83     5.90 33.12 56.78  23.66
  COIN            51.39     6.09 40.80 63.81  23.01
  IBIT            39.29     9.51 22.56 57.31  34.75

💡 해석:
  • Std_Dev (표준편차): 높을수록 변동성이 큼
  • Range (범위): 최대-최소 차이, 높을수록 불안정
  • 변동성이 낮고 평균이 40% 미만이면 안정적



📈 MAG 7+2 투자 전략 최종 요약 (공매도 분석 포함)

📍 분석 기준: Anchored VWAP (2025 Q4)
📅 Anchor Point: 2025-10-01
🌟 분석 대상: MAGNIFICENT SEVEN + BITCOIN EXPOSURE (9개 종목)

📊 MAG 7+2 종합 요약표 (기술적 분석 + 공매도)


 Rank Ticker   Price VWAP_Diff Q_Return YF_Short% Days_Cover FINRA_Daily% Tech_Score Total_Score Signal
    1  GOOGL $319.95    +15.1%   +30.6%     1.07%        1.9        31.6%     80/100     100/120      💚
    2   AAPL $277.55     +4.9%    +8.8%     0.78%        2.3        51.6%     75/100      95/120      💚
    3   TSLA $426.58     -1.5%    -7.2%     2.45%        0.8        50.7%     30/100      50/120      💙
    4   NVDA $180.26     -4.4%    -3.7%     1.02%        1.3        46.6%     25/100      45/120      💙
    5   AMZN $229.16     -0.2%    +3.9%     0.73%        1.4        36.5%     25/100      45/120      💙
    6   MSFT $485.50     -4.5%    -6.4%     0.65%        2.1        48.6%     15/100      35/120      💙
    7   IBIT  $51.03    -14.3%   -23.5%     0.00%        0.0        39.5%     15/100   